# Project1 PNLF Postproduction <a class="tocSkip">
    
After running `PNLF production.ipynb`, this notebook can be used to create plots that contain the results of all galaxies and the LaTeX output tables that are used in the paper.

## Preparation
     
First we load a bunch of common packages that are used across the project. More specific packages that are only used in one section are loaded later to make it clear where they belong to (this also applies to all custom moduls that were written for this project).

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from pnlf.packages import *

from pnlf.constants import tab10, single_column, two_column
from pnlf.auxiliary import filter_table

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

we use the `logging` module to handle informations and warnings (this does not always work as expected in jupyter notebooks).

In [ ]:
logging.basicConfig(stream=sys.stdout,format='%(levelname)s: %(message)s',level=logging.INFO)
logger = logging.getLogger(__name__)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

In [ ]:
# first we need to specify the path to the raw data
basedir = Path('..')
data_raw = Path('a:')/'Archive'
data_ext = Path('a:')/'Archive'
    
# the table with the measured distances
results = ascii.read(basedir/'data'/'interim'/ 'results.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
results.add_index('name')

# the parameters that were used
with open(basedir / 'data' / 'interim' / 'parameters.yml') as yml_file:
    parameters = yaml.load(yml_file,Loader=yaml.FullLoader)    
    
# some general parameters of the galaxy sample
sample_table = ascii.read(basedir/'data'/'interim'/'sample.txt')
sample_table.add_index('name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])

# the catalogue with the PNe (and SNRs)
with fits.open(basedir/'data'/'catalogues'/'nebulae.fits') as hdul:
    catalogue = Table(hdul[1].data)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])

In [ ]:
for row in results[np.abs(results['(m-M)']-results['(m-M)_SNR'])>0.05]:
    print(f"{row['name']}: d(m-M) = {row['(m-M)']-row['(m-M)_SNR']:.2f}, sigma = {(row['(m-M)']-row['(m-M)_SNR'])/row['err-(m-M)']:.2f}")

## Tables

### LaTeX sample table

In [ ]:
from astropy.table import join 

abundance_gradients = ascii.read(basedir/'data'/'external'/'radial_abundance_gradients.txt',
                                names=['name','R0','g_r25'])
abundance_gradients.add_index('name')

extinction = ascii.read(basedir/'data'/'external'/'extinction.txt')
extinction.add_index('name')

# 12+logOH at mean position of PNe
logOH = []
rmean = []
for name in abundance_gradients['name']:
    completeness = parameters[name]['completeness_limit']
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
    catalogue['overluminous'] = catalogue['overluminous'].astype(bool)
    catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
    catalogue = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude']) & (~catalogue['overluminous']) & (catalogue['mOIII']<completeness)]

    center = sample_table.loc[name]['SkyCoord']
    posang = sample_table.loc[name]['posang']
    inclination = sample_table.loc[name]['Inclination']
    eccentricity = np.sin(inclination*u.deg).value
    r25 = sample_table.loc[name]['r25']*u.arcmin
    catalogue['r'] = catalogue['SkyCoord'].separation(center)
    rmean.append(np.mean(catalogue['r']/r25).decompose())
        
abundance_gradients['rmean'] = rmean
abundance_gradients['12+logOH'] = abundance_gradients['R0'] + abundance_gradients['rmean']*abundance_gradients['g_r25']

In [ ]:
filename = basedir / 'data' / 'external' / 'phangs_sample_table_v1p6.fits'
with fits.open(filename) as hdul:
    sample = Table(hdul[1].data)

#galaxies = sample_table[sample_table['HAS_MUSE']==1]['NAME']
sample = sample[sample['survey_muse_status']!='not_in_survey']
sample['name'] = [x.upper() for x in sample['name']]


sample.rename_columns(['morph_string','orient_incl','orient_posang','size_r25'],
                      ['Type','Inclination','posang','r25'])

coord = SkyCoord(sample['orient_ra']*u.degree,sample['orient_dec']*u.degree)
sample['R.A.'],sample['Dec.'] = zip(*[x.split(' ') for x in coord.to_string(style='hmsdms',precision=2)])
sample['mass'] = np.log10(sample['props_mstar'])
sample['SFR'] = np.log10(sample['props_sfr'])

err_up = 10 ** (np.log10(sample['dist']) + sample['dist_unc']) - sample['dist']
err_down = sample['dist'] - 10 ** (np.log10(sample['dist']) - sample['dist_unc'])
sample['(m-M)'] = Distance(sample['dist']*u.Mpc).distmod.value
sample['err(m-M)'] = 5/np.log(10)*err_up / sample['dist']
sample['r25']/=60

sample['(m-M)'].info.format = '%.2f' 
sample['err(m-M)'].info.format = '%.2f' 
sample['r25'].info.format = '%.2f' 
sample['mass'].info.format = '%.2f' 
sample['SFR'].info.format = '%.2f' 

columns = ['name','Type','R.A.','Dec.','(m-M)','err(m-M)','Inclination','posang','r25','mass','SFR']
tbl = join(sample[columns],abundance_gradients,keys='name')
tbl['E(B-V)'] = [extinction.loc[name]['E(B-V)'] for name in tbl['name']]
# from sample table
tbl['PSF'] = [0.72,0.73,0.74,0.63,0.82,0.49,0.65,0.8,0.64,0.72,
                  0.85,0.74,0.77,0.58,0.58,0.64,0.44,0.73,0.79]
tbl['AO'] = ['e' if parameters[name]['power_index']==2.3 else '' for name in tbl['name'] ]
tbl['PSF'] = [f'${psf}$' if e else f'${psf}^\mathrm{{AO}}$' for psf,e in zip(tbl['PSF'],tbl['AO'])] 
tbl['12+logOH'].info.format = '%.2f' 

with open(basedir / 'data' / 'interim' / 'sample.txt','w',newline='\n') as f:
    ascii.write(tbl,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')
    
del tbl[['R0','rmean','g_r25','mass','SFR','AO']]


In [ ]:
latexdict = {'tabletype': 'table*',
'header_start': '\\toprule\\toprule',
'header_end': '\\midrule',
'data_end': '\\bottomrule',
'caption': f'Galaxy sample',
'units': {'R.A.':'(J2000)','Dec.':'(J2000)','Inclination':'deg','Distance':'$\si{\mega\parsec}$',
          'r25':'arcmin'},
'preamble': '\\centering',
'tablefoot': f'\\label{{tbl:sample}}'
            }
 
tbl['name'] = [f'\\galaxyname{{{row["name"][:-4]}}}{{{row["name"][-4:]}}}' for row in tbl]
#tbl['(m-M)'] = [f'{row["(m-M)"]:.2f}\pm{row["err(m-M)"]:.2f}' for row in tbl]
    
with open(basedir / 'data' / 'interim' /'sample.tex','w',newline='\n') as f:
    ascii.write(tbl,f,Writer=ascii.Latex, latexdict=latexdict,overwrite=True,exclude_names=['Name'])
    

### LaTeX result table

this uses the result table and prints out a LaTeX table (only the data part) that can be used in the final document. In another step, we merge the individual catalogues for PN and SNR identifications. 

In [ ]:
# with SI units
tbl_out = ''
results.sort('name')
for row in results:
    name = f'\\galaxyname{{{row["name"][:-4]}}}{{{row["name"][-4:]}}}'
    tbl_out += f'{name} & {row["N_PN"]} & {row["N_SNRorPN"]} '
    tbl_out += f'& $\\uncertainty{{{row["(m-M)"]:.2f}}}{{{row["err+(m-M)"]:.2f}}}{{{row["err-(m-M)"]:.2f}}}$ '
    tbl_out += f'& $\\uncertainty{{{row["(m-M)_SNR"]:.2f}}}{{{row["err+(m-M)_SNR"]:.2f}}}{{{row["err-(m-M)_SNR"]:.2f}}}$ '
    tbl_out += f'& $\\uncertainty{{{row["d/Mpc"]:.2f}}}{{{row["err+d/Mpc"]:.2f}}}{{{row["err-d/Mpc"]:.2f}}}$ \\\\\n'
    #tbl_out += f'& {row["alpha"]:.2f}\\\\\n'
    #tbl_out += f'& $\\uncertainty{{{row["dis_SNR"]:.2f}}}{{{row["dis_SNR_plus"]:.2f}}}{{{row["dis_SNR_minus"]:.2f}}}$ '
    #tbl_out += f'& $\\uncertainty{{{row["dis_SNR_Mpc"]:.2f}}}{{{row["dis_SNR_Mpc_plus"]:.2f}}}{{{row["dis_SNR_Mpc_minus"]:.2f}}}$\\\\\n'
    
print(tbl_out)    

In [ ]:
# without SI units
tbl_out = ''
results.sort('name')
for row in results:
    print(f'\\setvalue{{mag/{row["name"]} =\\SI[parse-numbers=false]{{\\uncertainty{{{row["(m-M)"]:.2f}}}{{{row["err+(m-M)"]:.2f}}}{{{row["err-(m-M)"]:.2f}}}}}{{\\mag}}}}')
print(' ')
for row in results:
    print(f'\\setvalue{{Mpc/{row["name"]} =\\SI[parse-numbers=false]{{\\uncertainty{{{row["d/Mpc"]:.2f}}}{{{row["err+d/Mpc"]:.2f}}}{{{row["err-d/Mpc"]:.2f}}}}}{{\\mega\\parsec}}}}')
    

In [ ]:
tbl_out = ''
results.sort('name')
for row in results:
    print(f'\\setvalue{{mag/{row["name"]} = \\uncertainty{{{row["(m-M)"]:.2f}}}{{{row["err+(m-M)"]:.2f}}}{{{row["err-(m-M)"]:.2f}}}}}')
print(' ')
for row in results:
    print(f'\\setvalue{{Mpc/{row["name"]} = \\uncertainty{{{row["d/Mpc"]:.2f}}}{{{row["err+d/Mpc"]:.2f}}}{{{row["err-d/Mpc"]:.2f}}}}}')
    

In [ ]:
# save to file
for col in ['(m-M)','err+(m-M)','err-(m-M)','d/Mpc','err+d/Mpc','err-d/Mpc']:
    results[col].info.format = '%.2f'

distance_modulus = []
distance_parsec = []
for row in results:
    distance_modulus.append(f'{row["(m-M)"]:.2f} + {row["err-(m-M)"]:.2f} - {row["err+(m-M)"]:.2f}')
    distance_parsec.append(f'{row["d/Mpc"]:.2f} + {row["err+d/Mpc"]:.2f} - {row["err-d/Mpc"]:.2f}')
results['mu'] = distance_modulus
results['d/Mpc'] = distance_parsec
                           
filename = basedir / 'data' / 'interim' / f'distances.txt'
with open(filename,'w',newline='\n') as f:
    ascii.write(results[['name','mu','d/Mpc']],
                f,format='fixed_width',delimiter='\t',overwrite=True)

### Combine Catalogues to single file

In [ ]:
tbl_lst = []
for name in results['name']:
    with fits.open(basedir/'data'/'catalogues'/f'{name}_classifications.fits') as hdul:
        tmp = Table(hdul[1].data)    
        tmp.add_column(name,name='gal_name',index=0)
    tbl_lst.append(tmp)
catalogue = vstack(tbl_lst)

# the object in NGC0628 is excluded in our sample
matched_objects = [('NGC0628',886,'Kreckel-SNR-8'),
                   ('NGC3351',1216,'Ciardullo-PN-5'),
                   ('NGC3627',749,'Ciardullo-PN-35')]

notes = []
for row in catalogue:
    note = []
    if row['SNRorPN']:
        pass
        #note.append('PN')
    if row['overluminous']:
        note.append('OL')
    if row['exclude']:
        note.append('EX')

    for name,idx,label in matched_objects:
        if row['gal_name']==name and row['id']==idx:
            note.append(label)
        
    notes.append(','.join(note))
catalogue['note'] = notes


# save complete catalogue
columns = ['gal_name','id','type','x','y','RA','DEC','fwhm','mOIII','dmOIII',
           'HB4861','HB4861_err', 'OIII5006','OIII5006_err', 'HA6562','HA6562_err', 
           'NII6583','NII6583_err', 'SII6716','SII6716_err', 'SII6730','SII6730_err',
           'note']

# we save all objects (nice table for collaborators)
hdu = fits.BinTableHDU(catalogue,name='nebulae catalogue')
hdu.writeto(basedir/'data'/'catalogues'/f'nebulae.fits',overwrite=True)

the catalogue for the journal

In [ ]:
from datetime import date

columns = ['gal_name','id','type','RA','DEC','mOIII','dmOIII','logOIII/Ha','d(logOIII/Ha)',
           'logNII/Ha','d(logNII/Ha)','logSII/Ha','d(logSII/Ha)','note']

catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue['RA'],catalogue['DEC'] = zip(*[x.split(' ') for x in catalogue['SkyCoord'].to_string(style='hmsdms',precision=2)])

catalogue['logOIII/Ha']   = np.log10(catalogue['OIII5006_flux']/catalogue['HA6562_flux'])
catalogue['d(logOIII/Ha)']= 1/np.log(10) * np.sqrt( (catalogue['HA6562_flux_err'] / catalogue['HA6562_flux'])**2 + (catalogue['OIII5006_flux_err'] / catalogue['OIII5006_flux'])**2)

#catalogue['Ha/NII']    = catalogue['HA6562']/catalogue['NII6583']
#catalogue['d(Ha/NII)'] = catalogue['HA6562'] / catalogue['NII6583'] * np.sqrt( (catalogue['NII6583_err'] / catalogue['NII6583'])**2 + (catalogue['HA6562_err'] / catalogue['HA6562'])**2)

#catalogue['Ha/SII']    = catalogue['HA6562']/catalogue['SII']
#catalogue['d(Ha/SII)'] = catalogue['HA6562'] / catalogue['SII'] * np.sqrt( (catalogue['SII_err'] / catalogue['SII'])**2 + (catalogue['HA6562_err'] / catalogue['HA6562'])**2)

catalogue['logNII/Ha']    = np.log10(catalogue['NII6583_flux']/catalogue['HA6562_flux'])
catalogue['d(logNII/Ha)'] = 1/np.log(10) * np.sqrt( (catalogue['NII6583_flux_err'] / catalogue['NII6583_flux'])**2 + (catalogue['HA6562_flux_err'] / catalogue['HA6562_flux'])**2)

catalogue['logSII/Ha']    = np.log10(catalogue['SII_flux']/catalogue['HA6562_flux'])
catalogue['d(logSII/Ha)'] = 1/np.log(10) * np.sqrt( (catalogue['SII_flux_err'] / catalogue['SII_flux'])**2 + (catalogue['HA6562_flux_err'] / catalogue['HA6562_flux'])**2)

for col in ['x','y','mOIII','dmOIII','logOIII/Ha','d(logOIII/Ha)','logNII/Ha','d(logNII/Ha)','logSII/Ha','d(logSII/Ha)']:
    catalogue[col].info.format = '%.2f' 
catalogue.sort(['gal_name','mOIII']) 

# we only use PN or SNR that could be PN
SNRorPN =  ((catalogue['type']=='PN')|((catalogue['type']=='SNR')&(catalogue['SNRorPN'])))
criteria = SNRorPN & (catalogue['mOIII']<28) & catalogue['OIII5006_detection'] & (~catalogue['exclude']) # | catalogue['overluminous'])


for name in np.unique(catalogue['gal_name']):
    N = len(catalogue[(criteria) & (catalogue['gal_name']==name)])
    catalogue['id'][(criteria) & (catalogue['gal_name']==name)] = np.arange(1,N+1)
    
# save only PNe
export = catalogue[columns][criteria].copy()
# adjust the completness limit for those two galaxies
export = export[~(np.isin(export['gal_name'],['NGC2835','NGC3627']) & (export['mOIII']>=27.5))]
with open(basedir/'data'/'catalogues'/'PN_candidates.txt','w',newline='\n') as f:
    ascii.write(export,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')

doc = f'''PNe catalogue for Scheuermann et al. (2021)

This catalogue was created with the following code:
https://github.com/fschmnn/pnlf
last update: {date.today().strftime("%b %d, %Y")}
'''    

primary_hdu = fits.PrimaryHDU()
for i,comment in enumerate(doc.split('\n')):
    if i==0:
        primary_hdu.header['COMMENT'] = comment
    else:
        primary_hdu.header[''] = comment
#table_hdu   = fits.BinTableHDU(export,name='catalogue')
#hdul = fits.HDUList([primary_hdu, table_hdu])
#hdul.writeto(basedir/'data'/'catalogues'/f'PN_catalogue.fits',overwrite=True)

In [ ]:
# create LaTeX table for a single galaxy (for paper)
tmp = ascii.read(basedir/'data'/'catalogues'/'PN_candidates.txt')
tmp = tmp[tmp['gal_name']=='NGC0628']
#with open(basedir / 'data' / 'catalogues' /'NGC0628_nebulae.tex','w',newline='\n') as f:
ascii.write(tmp[-5:],sys.stdout,Writer=ascii.Latex,overwrite=True,exclude_names=['x','y','fwhm'])


In [ ]:
columns = ['gal_name','id','type','x','y','RA','DEC','fwhm','mOIII','dmOIII','MOIII','dMOIII','HB4861_flux','HB4861_flux_err','HB4861_SIGMA','OIII5006_flux','OIII5006_flux_err','OIII5006_SIGMA','HA6562_flux','HA6562_flux_err','HA6562_SIGMA','NII6583_flux','NII6583_flux_err','NII6583_SIGMA','SII6716_flux','SII6716_flux_err','SII6716_SIGMA','SII6730_flux','SII6730_flux_err','SII6730_SIGMA','EBV_balmer','HB4861_flux_corr','HB4861_flux_corr_err','OIII5006_flux_corr','OIII5006_flux_corr_err','HA6562_flux_corr','HA6562_flux_corr_err','NII6583_flux_corr','NII6583_flux_corr_err','SII6716_flux_corr','SII6716_flux_corr_err','SII6730_flux_corr','SII6730_flux_corr_err','note']

primary_hdu = fits.PrimaryHDU()
for i,comment in enumerate(doc.split('\n')):
    if i==0:
        primary_hdu.header['COMMENT'] = comment
    else:
        primary_hdu.header[''] = comment
table_hdu   = fits.BinTableHDU(export[columns],name='catalogue')
hdul = fits.HDUList([primary_hdu, table_hdu])
hdul.writeto(basedir/'data'/'catalogues'/f'PN_catalogue_Enrico.fits',overwrite=True)



### id of other surveys in note column

the referee suggested to add a remark in the note column when an object is classified differently. Here I compare the catalogue to the literature and add this note.

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)
from astropy.table import vstack

from pnlf.load_references import NGC628, \
                                   pn_NGC628_kreckel, \
                                   snr_NGC628_kreckel, \
                                   pn_NGC628_herrmann, \
                                   NGC628_kreckel, \
                                   pn_NGC5068_herrmann, \
                                   pn_NGC3351_ciardullo, \
                                   pn_NGC3627_ciardullo,\
                                   pn_NGC0628_roth


In [ ]:
matchcoord = catalogue[catalogue['gal_name']=='NGC0628'].copy()

for catalogcoord, label in zip([pn_NGC628_herrmann,pn_NGC628_kreckel,snr_NGC628_kreckel,pn_NGC0628_roth],
                               ['Hpn','Kpn','Ksnr','Rpn']):

    idx, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogcoord['SkyCoord'])
    matchcoord[label] = catalogcoord[idx]['ID']
    matchcoord[label][sep>0.5*u.arcsec] = ''
    
matchcoord[['id','mOIII','type','Hpn','Kpn','Ksnr','Rpn']][(matchcoord['type']=='PN') & (matchcoord['mOIII']<28)]

In [ ]:
matchcoord = catalogue[catalogue['gal_name']=='NGC3627'].copy()

idx, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],pn_NGC3627_ciardullo['SkyCoord'])
matchcoord['idx'] = idx
matchcoord['sep'] = sep.to(u.arcsec)
matchcoord['id_ci'] = pn_NGC3627_ciardullo['ID'][idx]
matchcoord['mOIII_ci'] = pn_NGC3627_ciardullo['mOIII'][idx]
    
matchcoord.sort('mOIII')
matchcoord[['id','id_ci','type','mOIII','mOIII_ci','sep']][matchcoord['sep']<=1*u.arcsec]

In [ ]:
matchcoord = catalogue[catalogue['gal_name']=='NGC3351'].copy()

idx, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],pn_NGC3351_ciardullo['SkyCoord'])
matchcoord['idx'] = idx
matchcoord['sep'] = sep.to(u.arcsec)
matchcoord['id_ci'] = pn_NGC3351_ciardullo['ID'][idx]
matchcoord['mOIII_ci'] = pn_NGC3351_ciardullo['mOIII'][idx]

    
matchcoord.sort('mOIII')
matchcoord[['id','id_ci','type','mOIII','mOIII_ci','sep']][matchcoord['sep']<=1*u.arcsec]

In [ ]:
matchcoord = catalogue[catalogue['gal_name']=='NGC5068'].copy()

idx, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],pn_NGC5068_herrmann['SkyCoord'])
matchcoord['idx'] = idx
matchcoord['sep'] = sep.to(u.arcsec)
matchcoord['id_h'] = pn_NGC5068_herrmann['ID'][idx]
matchcoord['mOIII_h'] = pn_NGC5068_herrmann['mOIII'][idx]

#matchcoord['id_ciardullo'] = pn_NGC3627_ciardullo[idx]['ID'].astype(str)
#matchcoord['id_ciardullo'][sep>1*u.arcsec] = ''
    
matchcoord.sort('mOIII')
matchcoord[['id','id_h','type','mOIII','mOIII_h','sep']][matchcoord['sep']<=1*u.arcsec]

In [ ]:
np.char.lstrip(matchcoord['Hpn'],'M74-')

In [ ]:
matchcoord.sort('mOIII')

matchcoord[(matchcoord['type']=='PN')][['id','mOIII','type','Hpn','Kpn','Ksnr','Rpn']][:20]

## Combined plots

this section creates plots where all galaxies are combined into one figure

### Single PNLF

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, pnlf, cdf
from pnlf.plot.pnlf import plot_pnlf
from pnlf.auxiliary import mu_to_parsec
from scipy.stats import kstest

name = 'NGC4254'

catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)

binsize = 0.4 #parameters[name]['binsize']
completeness_limit = parameters[name]['completeness_limit']
mu = parameters[name]['mu']

# ----------------------------------------------------------------------
# fit the data
# ----------------------------------------------------------------------

data = catalogue['mOIII'][(catalogue['type']=='PN') & (catalogue['mOIII']<completeness_limit)]
err = catalogue['dmOIII'][(catalogue['type']=='PN') & (catalogue['mOIII']<completeness_limit)]

fitter = MaximumLikelihood1D(pnlf,data,err=err,mhigh=completeness_limit,Mmax=-4.47)
mu,mu_p,mu_m = fitter([29])

d,(dp,dm)=mu_to_parsec(mu,[mu_p,mu_m])
print('{:.2f} + {:.2f} - {:.2f}'.format(mu,mu_p,mu_m))
print('{:.2f} + {:.2f} - {:.2f}'.format(d,dp,dm))

ks,pv = kstest(data,cdf,args=(mu,completeness_limit))
print(f'{name}: statistic={ks:.3f}, pvalue={pv:.3f}')

# ----------------------------------------------------------------------
#plot PNLF
# ----------------------------------------------------------------------
filename = None #basedir / 'reports' / f'{galaxy.name}' / f'{galaxy.name}_PNLF'
ax1,ax2 = plot_pnlf(catalogue['mOIII'][(catalogue['type']=='PN')],mu,completeness_limit,
                 binsize=binsize,mhigh=29,filename=filename,color=tab10[0])
label = f'$(m-M)={mu:.2f}^{{+{mu_p:.2f}}}_{{-{mu_m:.2f}}}$'
ax1.text(0.07,0.8,label, transform=ax1.transAxes,fontsize=8)

plt.show()

### Combined PNLF

In [ ]:
from pnlf.plot.pnlf import _plot_pnlf

nbins = {'IC5332':5,'NGC0628':6,'NGC1087':3,'NGC1300':2,'NGC1365':3,
         'NGC1385':4,'NGC1433':3,'NGC1512':3,'NGC1566':3,'NGC1672':2,
         'NGC2835':3,'NGC3351':6,'NGC3627':3,'NGC4254':5,'NGC4303':4,
         'NGC4321':4,'NGC4535':3,'NGC5068':5,'NGC7496':3}

names = results['name']
nrows = 4
ncols = 5
filename = basedir / 'reports' / f'all_galaxies_PNLF_presentation'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
if nrows*ncols<len(names):
    raise ValueError('not enough subplots for selected objects') 
width = 8.5 #two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())
Mmax = -4.47

# loop over the galaxies we want to plot
for name in names:  

    with fits.open(basedir / 'data' / 'catalogues' / f'{name}_classifications.fits') as hdul:
        sub = Table(hdul[1].data)
    sub['SkyCoord'] = SkyCoord(sub['RaDec'])
    sub = sub[~np.isnan(sub['mOIII']) & sub['OIII5006_detection']]
    
    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0
        
    # pre-process the data for the plot and read additional parameters
    data = sub[(sub['type']=='PN') & (~sub['exclude'])]['mOIII']
    mask = sub[(sub['type']=='PN') & (~sub['exclude'])]['overluminous']
    
    mu = results.loc[name]['(m-M)']
    mu_p = results.loc[name]['err+(m-M)']
    mu_m = results.loc[name]['err-(m-M)']

    completeness = parameters[name]['completeness_limit']

    mlow = Mmax+mu
    binsize = (completeness-mlow) / nbins[name]
    mhigh = completeness+1.5*binsize
    
    ax=_plot_pnlf(data,mu,completeness,mask,binsize=binsize,mhigh=mhigh,ax=ax,ms=3)

        
    ylim=ax.get_ylim()
    y2 = ylim[1]*1.7
    if y2>100:y2=99
    ax.set_ylim([0.7,y2])
    if name in ['NGC1433','NGC1512']:
        ax.set_ylim([None,99])
    if name=='NGC1385':
        ax.set_ylim([None,12])        
    
    if name=='NGC2835':
        ax.text(0.2,0.07,f'{name}', transform=ax.transAxes,fontsize=7)        
    else:
        ax.text(0.63,0.07,f'{name}', transform=ax.transAxes,fontsize=7)
    
    label = f'$(m-M)={mu:.2f}^{{+{mu_p:.2f}}}_{{-{mu_m:.2f}}}$'
    ax.text(0.05,0.88,label, transform=ax.transAxes,fontsize=6)
    
    #ax.set_xlim([mu-5,completeness+0.5])
    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'$N_\mathrm{PN}$')
    #ax.set_title(name)
    #ax.set(xlim=[24,28.5])
    
axes[3,3].set_xlabel(r'$m_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ / mag')
ax = next(axes_iter)
#ax.remove()
h,l = fig.axes[0].get_legend_handles_labels()
ax.axis('off')
ax.legend(h,l,fontsize=7,loc='center left',frameon=False)

plt.subplots_adjust(wspace=0.15, hspace=0.15)
#plt.tight_layout()
#plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

### Combined cumulative PNLF

In [ ]:
from pnlf.plot.pnlf import _plot_cum_pnlf
from pnlf.analyse import cdf
from scipy.stats import kstest

names = results['name']
nrows = 4
ncols = 5
filename = basedir / 'reports' / f'all_galaxies_PNLF_cum_presentation'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
if nrows*ncols<len(names):
    raise ValueError('not enough subplots for selected objects')
width = 8.5 #two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

binsize=0.1
Mmax = -4.47
color = 'tab:red'

for name in names:  
        
    with fits.open(basedir / 'data' / 'catalogues' / f'{name}_classifications.fits') as hdul:
        sub = Table(hdul[1].data)
    sub['SkyCoord'] = SkyCoord(sub['RaDec'])
    sub = sub[~np.isnan(sub['mOIII']) & sub['OIII5006_detection']]
    
    # get the next axis
    ax = next(axes_iter)
    # find current position on the grid
    i, j = np.where(axes == ax)
    i,j=i[0],j[0]
    
    data = sub[(sub['type']=='PN') & (~sub['exclude']) & (~sub['overluminous'])]['mOIII']
    mu = results.loc[name]['(m-M)']
    completeness = parameters[name]['completeness_limit']

    print(name,len(data[data<completeness]))

    ks,pv = kstest(data[data<completeness],cdf,args=(mu,completeness))
    #print(f'{name}: {len(data[data<completeness])}, {ks:.2f}, {pv:.2f}')
    ks,pv = results.loc[name]['KS'],results.loc[name]['pvalue']
    
    ax=_plot_cum_pnlf(data,mu,completeness,ax=ax,binsize=None,ms=1.5)
    if name=='NGC1300':
        ax.set(xlim=[27,None])
        ax.text(0.03,0.07,f'{name}', transform=ax.transAxes,fontsize=7)
    else:
        ax.text(0.61,0.07,f'{name}', transform=ax.transAxes,fontsize=7)

    ax.text(0.10,0.88,f'$D_{{max}}={ks:.3f}$', transform=ax.transAxes,fontsize=7)
    ax.text(0.06,0.78,f'$p$-value$\ ={pv:.2f}$',transform=ax.transAxes,fontsize=7)
    
    if name in ['NGC1087','NGC1300']:
        ax.set_yticks([0,5,10,15])
        
    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'Cumulative N')
    
    
    #ax.set_title(name)
axes[3,3].set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
ax = next(axes_iter)
#ax.remove()

#h,l = fig.axes[0].get_legend_handles_labels()
ax.axis('off')
#ax.legend(h,l,fontsize=7,loc='center left',frameon=False)
    
plt.subplots_adjust(wspace=0.22, hspace=0.22)
#plt.tight_layout()
#plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

together with SNR (and without loading each catalogue by itself)

In [ ]:
nebulae = ascii.read(basedir/'data'/'catalogues'/'nebulae.txt')
nebulae['SkyCoord'] = SkyCoord(nebulae['RA'],nebulae['DEC'])
nebulae['SNRorPN'] = [True if 'PN' in row else False for row in nebulae['note']]
nebulae['OL'] = [True if 'OL' in row else False for row in nebulae['note']]
nebulae = nebulae[~nebulae['OL']]

In [ ]:
from scipy.stats import ks_2samp
from pnlf.analyse import cdf

names = results['name']
names = ['NGC1087','NGC1365','NGC1385','NGC1512','NGC4321']
nrows = 2
ncols = 3
filename = basedir / 'reports' / f'all_galaxies_PNLF_SNR_cum'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
if nrows*ncols<len(names):
    raise ValueError('not enough subplots for selected objects')
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

binsize=0.1
Mmax = -4.47
color = 'tab:red'

for name in names:  
    
    # get the next axis
    ax = next(axes_iter)
    # find current position on the grid
    i, j = np.where(axes == ax)
    i,j=i[0],j[0]
    
    mu = results.loc[name]['(m-M)']
    completeness = parameters[name]['completeness_limit']
    
    data1 = nebulae[(nebulae['gal_name']==name) & (nebulae['type']=='PN')]['mOIII']
    data2 = nebulae[(nebulae['gal_name']==name) & ((nebulae['type']=='PN')|((nebulae['type']=='SNR')&nebulae['SNRorPN']))]['mOIII']

    data1 = data1[data1<completeness]
    data2 = data2[data2<completeness]
    N1,N2 = len(data1),len(data2)      
    #print(f'{name}: PN: {N1}, PN and SNR: {N2}')

    ax.plot(data1,np.arange(1,N1+1,1),ls='-',mfc=tab10[0],mec=tab10[0],ms=1,marker='o',label='PN')
    ax.plot(data2,np.arange(1,N2+1,1),ls='-',mfc=tab10[1],mec=tab10[1],ms=1,marker='o',label='PN+SNR')
    
    # plot the fit
    ax.plot(data1,N1*cdf(data1,mu,completeness),ls=':',color='k',label='fit')
    ax.plot(data2,N2*cdf(data2,results.loc[name]['mu_SNR'],completeness),ls=':',color='k')
    
    ks,pv = ks_2samp(data1,data2)
    #ks,pv = kstest(data[data<completeness],cdf,args=(mu,completeness))
    ax.text(0.55,0.08,f'{name}', transform=ax.transAxes,fontsize=7)
    ax.text(0.1,0.78,f'$p$-value$={pv:.2f}$',transform=ax.transAxes,fontsize=7)
    ax.text(0.1,0.88,f'$D_{{max}}={ks:.3f}$', transform=ax.transAxes,fontsize=7)

    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'Cumulative N')
    
    #ax.set_title(name)
axes[0,2].set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
ax = next(axes_iter)
#ax.remove()

h,l = fig.axes[0].get_legend_handles_labels()
ax.axis('off')
ax.legend(h,l,fontsize=7,loc='center left',frameon=False)
    
plt.subplots_adjust(wspace=0.22, hspace=0.22)
#plt.tight_layout()
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
#plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

### Combine line diagnostics

In [ ]:
from pnlf.analyse import PNLF

style = {'SNR':{"marker":'o',"ms":3,"mfc":'None',"mec":tab10[0],'ls':'none','ecolor':tab10[0]},
         'SNRorPN':{"marker":'o',"ms":4,"mfc":'white',"mec":'tab:green','ls':'none','ecolor':'tab:green'},
         'HII':{"marker":'+',"ms":3,"mec":tab10[1],'ls':'none'},
         'PN':{"marker":'o',"ms":2,"mfc":'black','mec':'black','ls':'none','ecolor':'black'}
        }
Mmax=-4.47
color = 'tab:red'

# define the figure with the number of subplots
nrows = 5
ncols = 4
width = two_column
fig, axes_arr = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes = iter(axes_arr.flatten())


names = ['IC5332','NGC0628','NGC1566','NGC3351','NGC3627','NGC5068']

# loop over the galaxies we want to plot
for name in results['name']:  
       
    # read in the data
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
        for col in catalogue.columns:
            if col.endswith('detection'):
                catalogue[col] = catalogue[col]=='True'
    else:
        print(f'no catalogue for {name}')
        continue
        
    # get the next axis
    ax = next(axes)
    # find current position on the grid
    i, j = np.where(axes_arr == ax)
    i,j=i[0],j[0]
    
    # pre-process the data for the plot and read additional parameters
    mu = results.loc[name]['(m-M)']
    completeness = parameters[name]['completeness_limit']
    
    # draw line that we use seperate PN from HII
    MOIII = np.linspace(-5,1)
    OIII_Ha = 10**(-0.37*(MOIII)-1.16)
    ax.plot(MOIII,OIII_Ha,c='black',lw=0.6)
    ax.axhline(10**4)

    if completeness:
        ax.axvline(completeness-mu,ls='--',c='grey',lw=0.5)
    ax.axvline(Mmax,ls='--',c='grey',lw=0.5)

    for t in ['HII','PN','SNR']:
        tbl = catalogue[catalogue['type']==t]        
        ax.errorbar(tbl['mOIII']-mu,tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']),**style[t],label=t) 

        if t=='PN':
            # indicate for which PN we don't have a detection in HA6562
            tbl = tbl[~tbl['HA6562_detection']]
            ax.errorbar(tbl['mOIII']-mu,1.11*tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']),
                         marker=r'$\uparrow$',ms=4,mec='black',ls='none') 
        if t=='SNR':
            #tbl = tbl[tbl['SNRorPN']] 
            ax.errorbar(tbl['mOIII']-mu,tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']), marker='x',ms=2,mec=tab10[0],ls='none') 
   
    # objects that were rejeceted by eye
    tbl = catalogue[catalogue['exclude']]
    ax.errorbar(tbl['mOIII']-mu,tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']),marker='o',ms=3,ls='none',color='tab:green',label='rejected') 
    
    
    # configure axes 
    ax.set(xlim=[-5,np.ceil(completeness-mu)],
           ylim=[0.03,200],
           yscale='log')
    
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:.16g}'.format(y)))

    axt = ax.twiny()
    xlim1,xlim2 = ax.get_xlim()
    axt.set_xticks(np.arange(np.ceil(xlim1+mu),np.floor(xlim2+mu)+1),minor=False)
    axt.set(xlim   = [xlim1+mu,xlim2+mu])
    
    if i==0:
        axt.set(xlabel = r'$m_{\mathrm{[OIII]}}$')
    
    if i==nrows-1:
        ax.set(xlabel=r'$M_{\mathrm{[OIII]}}$')
    if j==0:
        ax.set(ylabel=r'[OIII] / $(\mathrm{H}\alpha + \mathrm{[NII]})$')
    ax.set_title(name)
    
    ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    
    
h,l = fig.axes[0].get_legend_handles_labels()
fig.legend(h, l, bbox_to_anchor=(0., 1.01, 1., .051),loc = 'upper center',ncol=4)

plt.tight_layout()
filename = basedir / 'reports' / f'all_galaxies_line_diagnostics_new'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
#plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

In [ ]:
from pnlf.analyse import PNLF

style = {'SNR':{"marker":'o',"ms":3,"mfc":'None',"mec":tab10[0],'ls':'none','ecolor':tab10[0]},
         'SNRorPN':{"marker":'o',"ms":4,"mfc":'white',"mec":'tab:green','ls':'none','ecolor':'tab:green'},
         'HII':{"marker":'+',"ms":3,"mec":tab10[1],'ls':'none'},
         'PN':{"marker":'o',"ms":2,"mfc":'black','mec':'black','ls':'none','ecolor':'black'}
        }
Mmax=-4.47
color = 'tab:red'

# define the figure with the number of subplots
nrows = 5
ncols = 4
width = two_column
fig, axes_arr = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes = iter(axes_arr.flatten())


names = ['IC5332','NGC0628','NGC1566','NGC3351','NGC3627','NGC5068']

# loop over the galaxies we want to plot
for name in results['name']:  
       
    # read in the data
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
        for col in catalogue.columns:
            if col.endswith('detection'):
                catalogue[col] = catalogue[col]=='True'
    else:
        print(f'no catalogue for {name}')
        continue
        
    # get the next axis
    ax = next(axes)
    # find current position on the grid
    i, j = np.where(axes_arr == ax)
    i,j=i[0],j[0]
    
    # pre-process the data for the plot and read additional parameters
    mu = results.loc[name]['(m-M)']
    completeness = parameters[name]['completeness_limit']
    
    # draw line that we use seperate PN from HII
    ax.axhline(-0.3979)

    if completeness:
        ax.axvline(completeness-mu,ls='--',c='grey',lw=0.5)
    ax.axvline(Mmax,ls='--',c='grey',lw=0.5)

    for t in ['HII','PN','SNR']:
        tbl = catalogue[catalogue['type']==t]        
        ax.errorbar(tbl['mOIII']-mu,np.log10(tbl['SII']/tbl['HA6562']),
                    **style[t],label=t) 

    # objects that were rejeceted by eye
    tbl = catalogue[catalogue['exclude']]
    ax.errorbar(tbl['mOIII']-mu,np.log10(tbl['SII']/tbl['HA6562']),
                 marker='o',ms=3,mfc='tab:green',ls='none',label='rejected') 

    # configure axes 
    ax.set(xlim=[-5,np.ceil(completeness-mu)],
           ylim=[-1.5,1],
           )
    
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:.16g}'.format(y)))

    axt = ax.twiny()
    xlim1,xlim2 = ax.get_xlim()
    axt.set_xticks(np.arange(np.ceil(xlim1+mu),np.floor(xlim2+mu)+1),minor=False)
    axt.set(xlim   = [xlim1+mu,xlim2+mu])
    
    if i==0:
        axt.set(xlabel = r'$m_{\mathrm{[OIII]}}$')
    
    if i==nrows-1:
        ax.set(xlabel=r'$M_{\mathrm{[OIII]}}$')
    if j==0:
        ax.set(ylabel=r'$\log_{10} \left(I_{[\mathrm{S}\,\textsc{ii}]} \; /\; I_{\mathrm{H}\,\alpha} \right)$')
    ax.set_title(name)
    
    ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    
    
h,l = fig.axes[0].get_legend_handles_labels()
fig.legend(h, l, bbox_to_anchor=(0., 1.01, 1., .051),loc = 'upper center',ncol=4)

plt.tight_layout()
filename = basedir / 'reports' / f'all_galaxies_line_diagnostics_SII'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
#plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

### Combined Completeness limit

In [ ]:
path = basedir/'data'/'interim'
files = [x for x in path.iterdir() if x.stem.endswith('mock_sources')]

limit   = 0.8
max_sep = 0.3

ncols = 5
nrows = int(np.ceil(len(files)/ncols))

width = 2*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

for file in files:
    
    ax = next(axes_iter)
    i, j = np.where(axes == ax)
    i,j=i[0],j[0]
    
    name = file.stem.split('_')[0]
    mock_sources = ascii.read(file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
  
    hist = []
    width = 0.5
    bins = np.arange(26,30,width)

    for center in bins:
        tmp = mock_sources[(mock_sources['magnitude']>center-width/2) & (mock_sources['magnitude']<=center+width/2)]
        if len(tmp)>0:
            hist.append(np.sum(tmp['sep']<max_sep)/len(tmp))
        else:
            hist.append(0)
    hist = np.array(hist)
    
    completeness_limit = np.max(bins[hist>=limit])
    
    ax.axhline(100*limit,color='black',lw=0.6)

    ax.bar(bins[hist>=limit],hist[hist>=limit]*100,width=width*0.9,color=tab10[0])
    ax.bar(bins[hist<limit],hist[hist<limit]*100,width=width*0.9,fc='white',ec=tab10[0])
    
    for b,h in zip(bins,hist):
        if b>=26.5:
            if h>=0.8:
                ax.text(b,5,f'{h*100:.0f}\%',horizontalalignment='center',color='white',fontsize=6)
            else:
                ax.text(b,5,f'{h*100:.0f}\%',horizontalalignment='center',color=tab10[0],fontsize=6)
    t = ax.text(0.75,0.92,f'{name}', transform=ax.transAxes,color='black',fontsize=7)
    t = ax.text(0.75,0.82,f'cl={completeness_limit:.0f}', transform=ax.transAxes,color='black',fontsize=7)

    ax.set(xlim=[26.2,29.8],
           ylim=[0,100])
    
    if i==nrows-1:
        ax.set(xlabel='m$_{[\mathrm{OIII}]}$')
    else:
        ax.set_xticklabels([])
    if j==0:
        ax.set(ylabel='percentage of recovered objects')
    else:
        ax.set_yticklabels([])
    
    #ax.set_title(f'{name}: cl = {completeness_limit}')
    
for i in range(nrows*ncols-len(files)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=0.05, hspace=0.05)

filename = basedir / 'reports' / f'all_galaxies_completeness'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

In [ ]:
from pnlf.detection import plot_completeness_limit

name = 'NGC0628'

file = basedir/'data'/'interim'/f'{name}_mock_sources.txt'

limit   = 0.8
max_sep = 0.3

mock_sources = ascii.read(file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')

filename = basedir / 'reports' / name / f'{name}_completness.pdf'

plot_completeness_limit(mock_sources,max_sep=max_sep,limit=limit,filename=filename)

### RGB image

In [ ]:
from pnlf.auxiliary import circular_mask
from pnlf.plot.plot import create_RGB
from pnlf.io import ReadLineMaps

nrows = 5
ncols = 4
width = two_column
fig, axes_arr = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes = iter(axes_arr.flatten())

# loop over the galaxies we want to plot
for name in results['name']:  
        
    # get the next axis
    ax = next(axes)
    # find current position on the grid
    i, j = np.where(axes_arr == ax)
    i,j=i[0],j[0]
    
    #galaxy = galaxies[name]
    galaxy = ReadLineMaps(data_raw,name,**parameters[name])
    
    # define masks as slices
    masks = {
     'NGC1300' : circular_mask(*galaxy.shape,radius=50),
     'NGC1365' : circular_mask(*galaxy.shape,(720,420),radius=200),
     #'NGC1433' : circular_mask(*galaxy.shape,radius=100),
     'NGC1512' : circular_mask(*galaxy.shape,radius=70),
     'NGC1566' : circular_mask(*galaxy.shape,(450,450),radius=100)|circular_mask(*galaxy.shape,(350,150),radius=180),
     'NGC1672' : circular_mask(*galaxy.shape,(600,310),radius=60),
     #'NGC3627' : circular_mask(*galaxy.shape,(330,740),radius=100),
     'NGC3351' : circular_mask(*galaxy.shape,radius=200),
     'NGC4321' : circular_mask(*galaxy.shape,(550,450),radius=60),
     'NGC4535' : circular_mask(*galaxy.shape,(300,520),radius=100)
    }
    
    mask = np.zeros(galaxy.shape,dtype=bool)
    mask |= galaxy.star_mask.astype(bool)
    mask[masks.get(galaxy.name,(slice(-1,0),slice(-1,0)))] = True

    #img = galaxy.OIII5006_DAP.copy()
    img = create_RGB(galaxy.HA6562,galaxy.OIII5006_DAP,galaxy.SII6716,weights=[0.6,1,0.6],percentile=[95,99.,95])
    img[mask,...] = (1,1,1) #(1, 165/255, 1/255) 

    ax.imshow(img,origin='lower')
    ax.set_title(name)
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
filename = basedir / 'reports' / f'all_objects_rgb'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
#plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

## Overluminous sources



In [ ]:
name='NGC7496'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
    catalogue['overluminous'] = catalogue['overluminous'].astype(bool)
catalogue[catalogue['overluminous']][['id','mOIII','type','SNRorPN','overluminous']]

### RGB image

In [ ]:
from pnlf.io import ReadLineMaps
from pnlf.auxiliary import filter_table
from pnlf.plot.utils import radial_profile, growth_curve, create_RGB

filename = basedir / 'reports' / f'all_galaxies_overluminous'

size = 40

with fits.open(basedir/'data'/'catalogues'/'PN_candidates.fits') as hdul:
    catalogue = Table(hdul[1].data)
sample = catalogue[catalogue['note']=='OL']

n_objects = len(sample)
print(f'plotting cutouts for {n_objects} objects')
ncols = 5
nrows = int(np.ceil(n_objects/ncols))

width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,1.05*width/ncols*nrows))
axes_iter = iter(axes.flatten())

failed = 0
for row in sample:
    
    name = row['gal_name']
    galaxy = ReadLineMaps(data_raw/'MUSE'/'DR2.1'/'MUSEDAP',name,**parameters[name])
    
    print(row['type'])

    ax = next(axes_iter)

    x,y = row[['x','y']]
    aperture_size=2.5*row['fwhm']/2

    star = Cutout2D(galaxy.OIII5006, (x,y), u.Quantity((size, size), u.pixel),wcs=galaxy.wcs)


    rgb = create_RGB(galaxy.HA6562,galaxy.OIII5006,galaxy.SII6716,percentile=99)
    yslice = slice(int(x-size/2),int(x+size/2))
    xslice = slice(int(y-size/2),int(y+size/2))

    ax.set_yticks([])
    ax.set_xticks([])

    try:
        im = ax.imshow(rgb[xslice,yslice,:],origin='lower')
    except:
        text = f'{name}: {id_change[row["id"]]}'
        t = ax.text(0.06,0.87,text, transform=ax.transAxes,color='black',fontsize=7)
        continue

    '''
    norm = simple_norm(star.data,clip=False,percent=99)
    im = ax.imshow(star.data,norm=norm,origin='lower',cmap=plt.cm.Reds)
    ax.set_yticks([])
    ax.set_xticks([])
    '''

    aperture = CircularAperture((size/2+(x-int(x)),size/2+(y-int(y))),aperture_size)
    aperture.plot(color='tab:red',lw=0.8,axes=ax)

    profile = radial_profile(star.data,star.input_position_cutout)

    ax2 = ax.inset_axes([0.02, 0.02, 0.32, 0.25])
    ax2.set_yticks([])
    ax2.set_xticks([])  

    ax2.plot(profile,color='black')
    ax2.axvline(aperture_size,color='tab:red',lw=0.5)

    text = f'{name}, {id_change.get(row["id"],row["id"])} ({row["type"]})'
    #t = ax.text(0.07,0.87,text, transform=ax.transAxes,color='black',fontsize=5)
    ax.set_title(text,loc='left',pad=3,fontsize=6)
    ax.set(aspect='equal')
    #t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))


    if (row['mOIII']<results.loc[name]['(m-M)']-3.97) and False:
        for loc in ['bottom','top','right','left']:
            ax.spines[loc].set_color('tab:orange')
            ax.spines[loc].set_linewidth(1)
    #t = ax.text(0.05,0.8,f'mOIII={row["mOIII"]:.1f}', transform=ax.transAxes,color='black',fontsize=8)
    #t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

for i in range(nrows*ncols-n_objects+failed):
    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()
    
plt.subplots_adjust(wspace=0.05,hspace=0.17)
#plt.tight_layout()
if filename:
    plt.savefig(filename.with_suffix('.png'),dpi=600)
    plt.savefig(filename.with_suffix('.pdf'),dpi=600)
plt.show()

In [ ]:
from pnlf.photometry import light_in_moffat

alpha = 2.3
fwhm  = 0.9
gamma = fwhm / 2 / np.sqrt(2**(1/alpha)-1)

light_in_moffat(3*fwhm/2,alpha,gamma)

In [ ]:
from pnlf.plot.utils import radial_profile, growth_curve
from astropy.modeling.models import Gaussian2D,Moffat2D,Const2D

y, x = np.mgrid[0:51, 0:51]
gm1 = Gaussian2D(100, 25, 25, 3, 3)
gm2 = Gaussian2D(50, 30, 35, 4, 4)

gm1 = Moffat2D(100, 25, 25, alpha=2.3, gamma=5)
gm2 = Moffat2D(100, 30, 30, alpha=2.3, gamma=5)
gm3 = Const2D(10)

g1 = gm1(x, y) + gm3(x,y) +gm2(x,y)

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(9,3))

ax1.imshow(g1)
ax2.plot(growth_curve(g1,np.array(g1.shape)/2))
ax3.plot(radial_profile(g1))
plt.show()

make a plot with growth curve, RGB, OIII and Halpha

In [ ]:
from pnlf.plot.plot import create_RGB
from pnlf.io import ReadLineMaps
from pnlf.auxiliary import filter_table
from pnlf.plot.plot import radial_profile
from matplotlib.backends.backend_pdf import PdfPages
import datetime

filename = basedir / 'reports' / f'all_galaxies_overluminous_full'

catalogue_file = basedir / 'data' / 'catalogues' / f'nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['overluminous'] = catalogue['note']=='OL'
    catalogue['exclude'] = catalogue['note']=='EX'

size = 40
sample = catalogue[catalogue['overluminous']]

ncols = 4
nrows = 4

width = 8.27
N = len(sample)
Npage = nrows # number we get on each page

with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for i in range(int(np.ceil(N/Npage))):
        print(f'working on page {i+1}')

        sub_sample = sample[i*Npage:(i+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))

        for row, (ax1,ax2,ax3,ax4) in zip(sub_sample,axes):  
            
            name = row['gal_name']
            galaxy = ReadLineMaps(data_raw/'MUSE_DR2'/'MUSEDAP',name,**parameters[name])

            x,y = row[['x','y']]
            aperture_size=2.5*row['fwhm']/2
            aperture = CircularAperture((size/2+(x-int(x)),size/2+(y-int(y))),aperture_size)

            star = Cutout2D(galaxy.OIII5006, (x,y), u.Quantity((size, size), u.pixel),wcs=galaxy.wcs)
            profile = radial_profile(star.data,star.input_position_cutout)

            ax1.plot(profile,color='black')
            ax1.axvline(aperture_size,color='tab:red',lw=0.5)
            text = f'{name}: ID={row["id"]}'
            t = ax1.text(0.07,0.87,text, transform=ax1.transAxes,color='black',fontsize=7)
            t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
            ax1.axis('off')
            
            norm = simple_norm(star.data,clip=False,percent=99)
            im = ax2.imshow(star.data,norm=norm,origin='lower',cmap=plt.cm.Greens)
            aperture.plot(color='black',lw=0.8,axes=ax2)
            ax2.axis('off')

            star = Cutout2D(galaxy.HA6562, (x,y), u.Quantity((size, size), u.pixel),wcs=galaxy.wcs)
            norm = simple_norm(star.data,clip=False,percent=99)
            im = ax3.imshow(star.data,norm=norm,origin='lower',cmap=plt.cm.Reds)
            aperture.plot(color='black',lw=0.8,axes=ax3)
            ax3.axis('off')

            rgb = create_RGB(galaxy.HA6562,galaxy.OIII5006,galaxy.SII6716,percentile=99)
            yslice = slice(int(x-size/2),int(x+size/2))
            xslice = slice(int(y-size/2),int(y+size/2))

            im = ax4.imshow(rgb[xslice,yslice,:],origin='lower')
            aperture.plot(color='tab:red',lw=0.8,axes=ax4)

            ax4.axis('off')
        
        for (ax1,ax2,ax3,ax4) in axes[Npage-len(sub_sample):]:
            print('removing axis')
            ax1.axis('off')    
            ax2.axis('off')    
            ax3.axis('off')    
            ax4.axis('off')    
        
        plt.subplots_adjust(wspace=-0.1, hspace=0)

        

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()
 


### Plot all PNe

In [ ]:
catalogue_file = basedir / 'data' / 'catalogues' / f'nebulae.txt'
catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
catalogue['overluminous'] = catalogue['note']=='OL'
catalogue['exclude'] = catalogue['note']=='EX'
catalogue['SkyCoord'] = SkyCoord(catalogue['RA'],catalogue['DEC'])
group_distances = ascii.read(basedir/'data'/'literature distances'/'group_distances.txt')
group_distances.add_index('Name')

In [ ]:
name = 'NGC7496'

tmp = catalogue[catalogue['gal_name']==name]

print(f"N_PN = {np.sum(tmp['type']=='PN')}")
print(f"overluminous N_PN = {np.sum((tmp['type']=='PN') & (tmp['overluminous']))}")
print(f"excluded N_PN = {np.sum((tmp['type']=='PN') & (tmp['exclude']))}")

print(f"N_SNR = {np.sum(tmp['type']=='SNR')}")
print(f"overluminous N_SNR = {np.sum((tmp['type']=='SNR') & (tmp['overluminous']))}")
print(f"excluded N_SNR = {np.sum((tmp['type']=='SNR') & (tmp['exclude']))}")
print(f"SNR or PN = {np.sum((tmp['type']=='SNR') & (tmp['exclude']))}")

In [ ]:
from pnlf.io import ReadLineMaps
from pnlf.auxiliary import filter_table
from pnlf.plot.cutouts import multipage_cutout_with_profile

name = 'NGC4254'

filename = basedir / 'reports' / name / f'{name}_PN_cutouts'

galaxy = ReadLineMaps(data_raw/'MUSE_DR2'/'MUSEDAP',name,**parameters[name])

catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
    catalogue['overluminous'] = catalogue['overluminous'].astype(bool)
sample = catalogue[(catalogue['mOIII']<28) & (catalogue['type']=='PN')]
sample.sort('mOIII')
sample = sample[:24]

multipage_cutout_with_profile(galaxy,sample,filename)

### Spectrum of those objects
I only downloaded datacubes for the DR1 galaxies (IC5332,NGC0628,NGC1087,NGC1365,NGC1512,NGC1566,NGC1672,NGC2835,NGC3351,NGC3627,NGC4254,NGC4535,NGC5068)

In [ ]:
from pnlf.auxiliary import annulus_mask, circular_mask
from pnlf.io import ReadLineMaps
from pnlf.auxiliary import filter_table

name = 'NGC0628'

filename = basedir / 'reports' / name / f'{name}_exclude'
region_IDs = exclude[name]
xlim=[4750,7000]
size = 40

print(f'plot spectra for {len(region_IDs)} regions')

cube_path = Path('g:\Archive')/'MUSE'/'DR1'/'datacubes'
if name not in [x.stem.split('_')[0] for x in cube_path.iterdir()]:
    raise FileNotFoundError(f'no datacube for {name}')
with fits.open(cube_path / f'{name}_DATACUBE_FINAL.fits' , memmap=True, mode='denywrite') as hdul:
    data_cube   = hdul[1].data
    cube_header = hdul[1].header
    
galaxy = ReadLineMaps(data_raw/'MUSE_DR2'/'MUSEDAP',name,**parameters[name])

catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)

fig = plt.figure(figsize=(two_column,len(region_IDs)*two_column/4)) 
gs = mpl.gridspec.GridSpec(len(region_IDs), 2, width_ratios=[1,3]) 

spectra = {}
for i,region_ID in enumerate(region_IDs):
    
    x,y = filter_table(catalogue,id=region_ID)[['x','y']][0]

    ax1 = fig.add_subplot(gs[2*i])
    ax2 = fig.add_subplot(gs[2*i+1])
    
    r = Cutout2D(galaxy.OIII5006, (x,y), u.Quantity((size, size), u.pixel),wcs=galaxy.wcs)
    norm = simple_norm(r.data,'linear',clip=False,percent=95)
    ax1.imshow(r.data, origin='lower',norm=norm,cmap='Greys')

    aperture = CircularAperture(r.position_cutout,8)
    aperture.plot(color='tab:red',lw=1,axes=ax1)

    t = ax1.text(0.07,0.87,f'{region_ID}', transform=ax1.transAxes,color='black',fontsize=7)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
    ax1.set_xticks([])
    ax1.set_yticks([])
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # there will be NaNs in the subcube that is used for the sigma clipping
        # astropy will issue a warning which we ignore in this enviornment
        circle  = circular_mask(*data_cube.shape[1:],(x,y),4)
        annulus = annulus_mask(*data_cube.shape[1:],(x,y),8,12) 
        _, bkg, _ = sigma_clipped_stats(data_cube[...,annulus],axis=1)
    
    spectrum = np.sum(data_cube[...,circle],axis=1)    
    # the background is the median * the number of non zero pixel
    spectrum_without_bkg = spectrum - bkg * np.sum(circle)
    
    #spectra = np.sum(data_cube[...,int(x)-1:int(x)+1,int(y)-1:int(y)+1],axis=(1,2))    
    # the wavelenght coverage of MUSE
    wavelength = np.linspace(4749.88,9349.88,data_cube.shape[0]) 
    
    ax2.plot(wavelength,spectrum,color=tab10[1],label='with background')
    ax2.plot(wavelength,spectrum_without_bkg,color=tab10[0],label='background subtracted')
    #ax2.legend() 

    ax2.set(xlim=xlim,
            ylabel=r'erg\,/\,s\,/\,\AA')
    if i ==0 :
        ax2.set_title(name)
    if i == len(region_IDs)-1:
        ax2.set(xlabel=r'$\lambda$\,/\,\AA')
    else:
        ax2.set_xticklabels([])
        
    ax2.yaxis.tick_right()
    ax2.yaxis.set_ticks_position('both')
    ax2.yaxis.set_label_position("right")
    
    # save spectra
    spectra[f'{region_ID}_wavelength'] = wavelength
    spectra[f'{region_ID}_spectra'] = spectrum
    spectra[f'{region_ID}_bkg'] = bkg * np.sum(circle)
    
plt.subplots_adjust(wspace=0,hspace=0.05)
    
if filename:
    plt.savefig(filename.with_suffix('.pdf'),dpi=600)
    
plt.show()

In [ ]:
from pnlf.auxiliary import annulus_mask, circular_mask

name = 'NGC0628'

cube_path = Path('g:\Archive')/'MUSE'/'DR1'/'datacubes'
if name not in [x.stem.split('_')[0] for x in cube_path.iterdir()]:
    raise FileNotFoundError(f'no datacube for {name}')
with fits.open(cube_path / f'{name}_DATACUBE_FINAL.fits' , memmap=True, mode='denywrite') as hdul:
    data_cube   = hdul[1].data
    cube_header = hdul[1].header
    
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
    

def extract_spectra(cube,header,positions,region_ID,filename):
    '''extract spectra from a spectral cube at given positions'''
    
    logger.info(f'extracting spectrum for {len(positions)} objects')
    
    wavelength = []
    spectrum   = []
    background = []
    
    radii = [22.48, 21.42, 22.12]
    
    for i,pos in enumerate(positions):
        print(f'{i+1} of {len(positions)}')
        print(radii[i])
        x,y=pos
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # there will be NaNs in the subcube that is used for the sigma clipping
            # astropy will issue a warning which we ignore in this enviornment
            circle  = circular_mask(*cube.shape[1:],(x,y),radii[i])
            annulus = annulus_mask(*cube.shape[1:],(x,y),23,24) 
            _, bkg, _ = sigma_clipped_stats(cube[...,annulus],axis=1)

        spectrum.append(np.sum(data_cube[...,circle],axis=1))  
        background.append(bkg * np.sum(circle))
        wavelength.append(np.linspace(header['CRVAL3'],header['CRVAL3']+header['NAXIS3']*header['CD3_3'],header['NAXIS3']))
    
    spectra = Table(data=[region_ID,wavelength,spectrum,background],
                    names=['region_ID','wavelenght','spectrum','bkg'])

    hdu = fits.BinTableHDU(spectra,name='spectra')
    hdu.writeto(filename,overwrite=True)

In [ ]:
#positions = catalogue[catalogue['type']=='PN'][['x','y']]
#filename = basedir/'data'/'suspicious'/f'{name}_spectra_liz.fits'
#extract_spectra(data_cube,cube_header,positions[:50],catalogue[catalogue['type']=='PN']['id'][:50],filename)
    

In [ ]:
with fits.open(basedir/'data'/'suspicious'/'{}_spectra.fits'.format(name)) as hdul:
    spec = Table(hdul[1].data)
spec.add_index('region_ID')

In [ ]:
dap_path = Path('g:\Archive')/'MUSE'/'DR1'/'MUSEDAP'/'NGC0628_MAPS.fits'
with fits.open(dap_path) as hdul:
    wcs = WCS(hdul['FLUX'].header)
    

ra  = np.array([24.1623,24.1645,24.1888])
dec = np.array([15.7701,15.7958,15.7968])

p_sk = SkyCoord(ra*u.degree,dec*u.degree)

for sk in p_sk:
    p_xy.append(sk.to_pixel(wcs))

In [ ]:
filename = basedir/'data'/'suspicious'/f'{name}_spectra_liz.fits'
extract_spectra(data_cube,cube_header,positions,[1,2,3],filename)


In [ ]:
p_xy[0]

In [ ]:
x,y = p_xy[0]
width = 30
sub_cube = data_cube[:,int(x)-width:int(x)+width,int(y)-width:int(y)+width]

In [ ]:
with fits.open(basedir/'data'/'suspicious'/f'{name}_spectra_liz.fits') as hdul:
    spec = Table(hdul[1].data)
spec.add_index('region_ID')

In [ ]:
p = positions[0]

plt.plot(data_cube[:,int(p[0]),int(p[1])])

In [ ]:
sub = catalogue[(np.isin(catalogue['type'],['PN','SNR'])) & (catalogue['mOIII']<28)][['id','x','y','RaDec','mOIII','type']]

with open(basedir / 'data' / f'{name}_PN_and_SNR.txt','w',newline='\n') as f:
    ascii.write(sub,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')
    

In [ ]:
from spectral_cube import SpectralCube

In [ ]:
for sk in p_sk:
    sep=sk.separation(catalogue['SkyCoord'])
    row = catalogue[np.argmin(sep)]
    print(catalogue[sep.__lt__(Angle('10"'))]['type'])
    print(f'{np.min(sep.to(u.arcsec)):.2f}, {row["id"]}, {row["type"]}')

## zero point

the zeropoint $M*$ of the PNLF must be measured from galaxies with known distances. Ciardullo+2002 suggested a quadratic dependence on metallicity. Here we try to validate this assumption by comparing our measured distances to TRGB distances and infer $M*$. 

From **Ferrares+2000**

The OIII5007 magnitude of the PNLF, m*, uncorrected for foreground extinction, is listed in column (9) of Table 3. Because PNLF distance moduli are calculated by Ðtting the luminosity function with a standard template (e.g., Ciardullo et al. 1989b), only the Ðnal distance moduli are published. From these we derived m* a posteriori by subtracting the zero point (and the extinction correction, if applied) adopted by the authors. The PNLF distances to the SMC, NGC 3109, and NGC 5253, listed in Table 3, are not well constrained. The planetary nebula (PN) sample in NGC 3109 (Richer & McCall 1992) includes only seven objects, and an upper limit to the distance is derived from the brightest of the PNs observed. Jacoby, Walker, & Ciardullo (1990) advise against the use of the PNLF distance to the SMC because of the small number of PNs deÐning the luminosity function. Finally, the small number of PNs detected in NGC 5253, the presence of strong internal dust extinction, and the galaxyÏs very low metal abundance all conjoin to produce a very ill constrained PNLF magnitude cuto†, unsuitable for distance determinations (Phillips et al. 1992). Uncertainties in the values of m* are summarized, for example, in Jacoby, Ciardullo, & Ford (1990). They include a contribution associated with the Ðtting procedure (of the order of 0.10 mag), photometric zero points (D0.05 mag), the Ðlter response calibration (D0.04 mag), and the uncertain deÐnition of the empirical PNLF (D0.05 mag). Errors in the reddening estimate, which are sometimes included, have been removed (in quadrature) from the present analysis, since we only deal with uncorrected magnitudes.

we shift the original prescription from Ciardullo
$$
dM* = 0.928[O/H]^2 +0.225[O/H]+0.014, 
$$
that uses (Grevesse, 12+logOH=8.87) to (Asplund, 12+logOH=8.69). For this we define the difference d=logOH_asplund-logOH_grevesse. With this we can calculate the new parameters as
$$
a2 = a1 \\
b2 = b1+2*a2*d \\
c2 = c1-a2*d^2+b1*d
$$

In [ ]:
# shift formula from Ciardullo+2002 to a new logOH_sun
logOH_sun = 8.69 # from Asplund
logOH_sun_old = 8.87 # from Greeves

d = logOH_sun - 8.87 # difference between Asplund and Greeves
a1,b1,c1 = 0.928,0.225,0.014

a2 = a1
b2 = b1+2*a1*d
c2 = c1-a2*d**2+b2*d

deltaM_old = lambda OH: a1*OH**2+b1*OH+c1
deltaM = lambda OH: a2*OH**2+b2*OH+c2

the tables with the abundance gradients

In [ ]:
catalogue = Table(fits.getdata(basedir/'data'/'catalogues'/f'PN_candidates.fits',ext=1))
catalogue['overluminous'] = catalogue['note']=='OL'
catalogue['exclude'] = catalogue['note']=='EX'
catalogue['SkyCoord'] = SkyCoord(catalogue['RA'],catalogue['DEC'])

abundance_gradients = ascii.read(basedir/'data'/'external'/'radial_abundance_gradients.txt',
                                names=['name','R0','g_r25'])
abundance_gradients.add_index('name')

pilyugin = ascii.read(basedir/'data'/'external'/'Pilyugin2014.txt')
pilyugin.add_index('name')
pilyugin.add_row(['SMC',8.03,0.03,-0.03,0.03]+10*[0])
pilyugin.add_row(['LMC',8.35,0.03,-0.05,0.05]+10*[0])
# missing are
#'NGC3368', 'NGC3627', 'NGC5253'

In [ ]:
from pnlf.auxiliary import project

def calc_r25(name,x,y):
    '''calculate radius in terms of r25 (deprojected)
    
    '''

    # get the pixel position of the centre
    with fits.open(data_ext/'MUSE'/'DR2.1'/'MUSEDAP'/f'{name}_MAPS.fits') as hdul:
        wcs = WCS(hdul['FLUX'].header)
    centre = sample_table.loc[name]['SkyCoord']
    x_cen,y_cen = centre.to_pixel(wcs)
    
    pa  = sample_table.loc[name]['posang']
    inc = sample_table.loc[name]['Inclination']
    r25 = sample_table.loc[name]['r25']*u.arcmin
    
    # deproject
    x_depr,y_depr = project(x-x_cen,y-y_cen,pa,inc)
    skycoord_depr = SkyCoord.from_pixel(x_depr+x_cen,y_depr+y_cen,wcs)
    
    # separation to centre
    sep = skycoord_depr.separation(centre)
    
    return (sep/r25).decompose()

# to get the abundances for all galaxies (used in the sample table)
for name in results['name']:
    tmp = catalogue[(catalogue['gal_name']==name) & 
                    (catalogue['type']=='PN') & 
                    ~catalogue['overluminous'] 
                   ]
    center = sample_table.loc[name]['SkyCoord']


    # the catalogue with the positions

    radii = calc_r25(name,tmp['x'],tmp['y'])
    rmean = np.mean(radii)

    logOH = abundance_gradients.loc[name]['R0'] +rmean*abundance_gradients.loc[name]['g_r25']
    print(f'{name}:{logOH:.2f}')

the table with our measured zero points

In [ ]:
better_Mmax = {'IC5332': (-4.503,0.122,0.169),
 'NGC0628': (-4.527,0.133,0.135),
 'NGC1365':  (-4.624,0.071,0.069),
 'NGC2835': (-4.360,0.165,0.184),
 'NGC3351': (-4.139,0.087,0.103),
 'NGC3627':(-4.540,0.096,0.116),
 'NGC4321': (-4.337,0.211,0.194),
 'NGC5068': (-4.570,0.124,0.204)
                }

# create the table from the results table
sample = results[np.isin(results['name'],list(better_Mmax.keys()))][['name','(m-M)','err+(m-M)','err-(m-M)']].copy()
sample.add_index('name')

# add columns for the fitted zeropoint
sample['M*'] = np.nan
sample['err+M*'] = np.nan
sample['err-M*'] = np.nan
sample['dM*'] = np.nan

# calculate the mean position (for the abundances)
sample['rmean'] = np.nan
sample['rmin'] = np.nan
sample['rmax'] = np.nan


for row in sample:
    name = row['name']
    
    row[['M*','err+M*','err-M*']] = better_Mmax[name]
    
    tmp = catalogue[(catalogue['gal_name']==name) & (catalogue['type']=='PN') & ~catalogue['overluminous']]
    center = sample_table.loc[name]['SkyCoord']

    radii = calc_r25(name,tmp['x'],tmp['y'])
    row['rmean'] = np.mean(radii)
    row['rmin'] = np.min(radii)
    row['rmax'] = np.max(radii)

sample['dM*'] = sample['M*'] + 4.47

sample = join(sample,abundance_gradients,keys='name')
sample['logOH'] = sample['R0'] +sample['rmean']*sample['g_r25']
sample['logOHmin'] = sample['R0'] +sample['rmax']*sample['g_r25']
sample['logOHmax'] = sample['R0'] +sample['rmin']*sample['g_r25']
sample.sort('logOH')

for col in sample.columns[1:]:
    sample[col].info.format = '%.3f'
sample.add_index('name')

the table from Ciardullo+2001 with the zero point based on Cepheids distances

In [ ]:
from astropy.modeling import models, fitting

logOH = np.linspace(7.8,9.5)

# table from Ciardullo+2002
# NGC5253 has dM* =0.33 in paper, NGC300 dM*=0.15
# NGC0224 has M*=-4.66 in paper
cepheids = Table({
 'name':['LMC','SMC','NGC0224','NGC0300','NGC0598','NGC2403','NGC3031','NGC3351','NGC3368','NGC3627','NGC4258','NGC5253','NGC5457'],
 'EBV' : [0.075,0.037,0.062,0.013,0.041,0.040,0.080,0.028,0.025,0.032,0.016,0.056,0.009],
 '(m-M)' : [18.50,19.01 ,24.38 ,26.53 ,24.56 ,27.48 ,27.75 ,29.85 ,29.97 ,29.86 ,29.44 ,27.56 ,29.13],
 'err(m-M)' : [0.0,0.03,0.05,0.07,0.10,0.10,0.08,0.09,0.06,0.08,0.07,0.14,0.11],
 'M*': [-4.56,-4.67,-4.5,-4.21,-4.08,-4.41,-4.52,-4.39,-4.65,-4.44,-4.51,-4.05,-4.28],
 '+M*': [0.13,0.40,0.14,0.67,0.16,0.16,0.12,0.19,0.12,0.12,0.13,0.63,0.15],
 '-M*' : [0.09,0.17,0.11,0.16,0.14,0.13,0.11,0.13,0.11,0.12,0.11,0.16,0.14],
 'logOH': [8.50,8.03,8.98,8.35,8.82,8.80,8.75,9.24,9.20,9.25,8.85,8.15,8.50],
 'dM*' : [0.06,0.48,0,0,0,0,0,0,0,0,0,0.0,0.06]
})
cepheids.add_index('name')
# dM* has a different meaning in the Cepheid table (some correction form Dopita+92)
cepheids['M*'] += cepheids['dM*']
cepheids['dM*'] = cepheids['M*']+4.47


model  = models.Polynomial1D(degree=2,c0=0.014, c1=0.225, c2= 0.928)
fitter = fitting.LinearLSQFitter()
fit = fitter(model,cepheids['logOH']-logOH_sun,cepheids['dM*'],weights=1/cepheids['+M*'])

fig,ax=plt.subplots()
ax.errorbar(cepheids['logOH'],cepheids['M*'],yerr=[cepheids['+M*'],cepheids['-M*']],fmt='o')
ax.plot(logOH,-4.47+deltaM(logOH-logOH_sun),'k:',lw=1.2,label='Ciardullo+2002',color='gray',zorder=1)
for row in cepheids:
    ax.text(row['logOH']+0.01,row['M*'],row['name'],
         ha='left',fontsize='7',zorder=4)

ax.invert_yaxis()
ax.set(xlim=[7.8,9.5],ylim=[-3.6,-5])
plt.locator_params(axis='y',nbins=10)
plt.grid(True)
plt.show()

the abundances in Ciardullo+2001 are measured with different methods. Here we update them with values from Pilyugin+2014 (similar to the prescription that we are using)

In [ ]:
print(f'sample before: {len(cepheids)}')
# use metallicities from Pilyugin for Cepheids distances from Ciardullo
cepheids = join(pilyugin,cepheids)
cepheids.add_index('name')
cepheids['logOH_new'] = cepheids['R0_O']+0.25*cepheids['g_r25_O']
cepheids['errM*'] = cepheids['+M*']
print(f'sample after: {len(cepheids)}')

### measure M* from our data

Fit dM* (from TRGB distances) to log (O/H) to determine the zeropoint

In [ ]:
from astropy.modeling import models, fitting
from pnlf.fit import linearMLE

model  = models.Polynomial1D(degree=2,c0=0.014, c1=0.225, c2= 0.928)
fitter = fitting.LinearLSQFitter(calc_uncertainties=True)

# ----------------------------------------------
# fit a constant line
# ----------------------------------------------

model  = models.Polynomial1D(degree=1,c0=-4.5, c1=0)
model.c1.fixed=True
sub = sample
fit = fitter(model,sub['logOH'],sub['M*'],weights=1/sub['err+M*'])
Mmax_fit = fit.c0.value
Mmax_errp = fit._stds.stds[0]
Mmax_errm = fit._stds.stds[0]
print(f'PHANGS: M*={Mmax_fit:.3f}+-{Mmax_errp:.3f}')

sub = cepheids[~np.isin(cepheids['name'],['SMC','NGC5253','NGC0300'])]
fit = fitter(model,sub['logOH'],sub['M*'],weights=1/sub['+M*'])
print(f'Ciardullo: M*={fit.c0.value:.3f}+-{fit._stds.stds[0]:.3f}')



In [ ]:
from scipy.optimize import minimize

sample.sort('logOH')
x = np.array(sample['logOH'])
y = np.array(sample['M*'])
yerr = np.array(sample['err+M*'])
names = sample['name']

loglikelihood = lambda b: -np.sum(np.log(1/np.sqrt(2*np.pi*yerr**2) * np.exp(-(y-b)**2/(2*yerr**2))))

Mmax_fit = minimize(loglikelihood,-4.5).x[0]

# calculate errors
x_arr =np.linspace(-4.2,-4.8)

likelihood = np.exp([-loglikelihood(b) for b in x_arr])
valid = ~np.isnan(likelihood) 

likelihood /= np.abs(np.trapz(likelihood[valid],x_arr [valid]))
normalization = np.trapz(likelihood,x_arr )
integral = np.array([np.trapz(likelihood[x_arr<=xp],x_arr[x_arr<=xp])/normalization for xp in x_arr[1:]])
# 1 sigma interval for cumulative likelihood
mid = np.argmin(np.abs(integral-0.5))
high = np.argmin(np.abs(integral-0.8415))
low = np.argmin(np.abs(integral-0.1585))

Mmax_errp = x_arr[high]-Mmax_fit
Mmax_errm = Mmax_fit-x_arr[low]
        
print(f'Mmax = {Mmax_fit:.3f}+{Mmax_errp:.3f}-{Mmax_errm:.3f}')

In [ ]:
fig,ax=plt.subplots()

ax.plot(x_arr[:-1],integral)

ax.axhline(0.1585,color='black')
ax.axhline(0.5,color='black')
ax.axhline(0.8415,color='black')

#ax.axvline(x_arr[mid],color='black')
#ax.axvline(x_arr[low],color='gray')
#ax.axvline(x_arr[high],color='gray')

ax.axvline(Mmax_fit,color='black')
ax.axvline(Mmax_fit-Mmax_errm,color='gray')
ax.axvline(Mmax_fit+Mmax_errp,color='gray')

plt.show()

In [ ]:
print(x_arr[low])
print(x_arr[mid])
print(x_arr[high])

or with MCMC

https://dfm.io/posts/mixture-models/

In [ ]:
from pnlf.fit import linearMLE

sample = sample
x = np.array(sample['logOH'])
y = np.array(sample['M*'])
yerr = np.array((sample['err+M*']+sample['err-M*'])/2)
yerr = np.array(sample['err+M*'])

names = list(sample['name'])

bounds = [(-1e-5, 1e-5), (-5,-4), (0, 1), (-5,-4), (-5, 2)]
p0 = np.array([0.0, -4.5, 0.9, np.mean(y), np.log(np.var(y))])

fitter = linearMLE(x,y,yerr,bounds)
result = fitter.fit(p0)
fitter.outlier()
ax = fitter.plot(xlim=[8.3,8.7],ylim=[-5.,-3.8],xlabel='logOH',ylabel='M*')
#ax.text(lit_dist.loc['NGC4535']['logOH']-0.01,lit_dist.loc['NGC4535']['M*']+0.03,'NGC4535',
#         ha='right',va='top',fontsize=6,zorder=4)
#ax.text(lit_dist.loc['NGC3351']['logOH']+0.01,lit_dist.loc['NGC3351']['M*']+0.03,'NGC3351',
#         ha='left',va='top',fontsize=6,zorder=4)
ax.invert_yaxis()

x0 = np.linspace(*[8.3,8.7],100)
A = np.vander(x0, 2)
lines = np.dot(fitter.sampler.flatchain[:, :2], A.T)
quantiles = np.percentile(lines, [16, 84], axis=0)

Mmax_fit  = np.mean(fitter.results["b"][0])
Mmax_errp = np.mean(quantiles[1]) - Mmax_fit
Mmax_errm = Mmax_fit - np.mean(quantiles[0])
Mmax_fit, Mmax_errm, Mmax_errp = fitter.results['b']

print(f'Mmax = {Mmax_fit:.3f}+{Mmax_errp:.3f}-{Mmax_errm:.3f}')

plt.show()

post_prob = fitter.outlier(names)

In [ ]:
fig,ax=plt.subplots()

ax.hist(fitter.sampler.flatchain[:,1],bins=np.linspace(-4.8,-4.4,50))

plt.show()

and now the final plot for the paper

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

cmap1 = LinearSegmentedColormap.from_list("mycmap", ['white','tab:orange'])

logOH = np.linspace(7.8,9.5)
logOH_sun_new = 8.69   # Asplund

fig,(ax1,ax2) = plt.subplots(nrows=2,figsize=(single_column,single_column),sharex='col')

# plot data from Ciardullo+2002
ax1.errorbar(cepheids['logOH_new'],cepheids['M*'],
            yerr=[cepheids['+M*'],cepheids['-M*']],capsize=0,
            fmt='o',color=tab10[4],mec=tab10[4],ms=2.5,elinewidth=0.5,label='Cepheids',zorder=3)
ax1.plot(logOH,-4.47+deltaM(logOH-logOH_sun),'k:',lw=1.2,label='model',color='gray',zorder=1)

ax1.text(cepheids.loc['NGC3351']['logOH_new']-0.01,cepheids.loc['NGC3351']['M*'],'NGC3351',
         ha='right',va='top',fontsize='4',zorder=4)
ax1.text(cepheids.loc['SMC']['logOH_new']+0.01,cepheids.loc['SMC']['M*'],'SMC',
         ha='left',fontsize='4',zorder=4)
ax1.text(cepheids.loc['LMC']['logOH_new']-0.01,cepheids.loc['LMC']['M*'],'LMC',
         ha='right',fontsize='4',zorder=4)

# plot our own data
sample['xerrp'] = list(sample['logOHmax']-sample['logOH'])
sample['xerrm'] = list(sample['logOH']-sample['logOHmin'])
ax2.errorbar(sample['logOH'],sample['M*'],
             xerr=[sample['xerrm'],sample['xerrp']],
             yerr=[sample['err-M*'],sample['err+M*']],
            fmt='o',mew=0.5,color=tab10[2],ms=2.5,elinewidth=0.5,label='TRGB',zorder=3)
ax2.plot(logOH,-4.47+deltaM(logOH-logOH_sun),'k:',lw=1.2,color='gray',zorder=1)

#ax2.scatter(fitter.x, fitter.y, marker="o", s=10,c=fitter.post_prob, cmap=cmap1,
#           edgecolors='tab:orange',linewidths=0.5,vmin=0, vmax=1, zorder=1000)

#print(Mmax_fit)
ax2.fill_between([8,8.8], 2*[Mmax_fit-Mmax_errm],2*[Mmax_fit+Mmax_errp],color="silver",edgecolor='face',alpha=1,zorder=0)    
ax2.plot([8,8.8],[Mmax_fit,Mmax_fit],lw=0.8,color='black',label='fit')

for name in ['NGC3351']:
    ax2.text(sample.loc[name]['logOH']+0.01,sample.loc[name]['M*']-0.05,name,
             ha='left',fontsize='4',zorder=4)

ax1.locator_params(axis='y',nbins=3)
ax2.locator_params(axis='y',nbins=3)

#for x in [-4.47,-4.5]:
#    ax2.axhline(x,color='blue',lw=0.2)

ax1.set(xlim=[8,8.8],ylim=[-5.1,-3.6],ylabel=r'$M^*$ / mag')
ax2.set(xlim=[8,8.8],ylim=[-5.1,-3.6],xlabel=r'$12+\log (\mathrm{O}/\mathrm{H})$',ylabel=r'$M^*$ / mag')
ax1.invert_yaxis()
ax2.invert_yaxis()

lines = []
labels = []
for ax in fig.axes:
    h, l = ax.get_legend_handles_labels()
    lines.extend(h[::-1])
    labels.extend(l[::-1])
myorder = [0,2,1,3]
lines = [lines[i] for i in myorder]
labels = [labels[i] for i in myorder]

#ax1.legend(ncol=2)
ax2.legend(lines, labels,ncol=1,loc=3)
plt.subplots_adjust(wspace=0.05,hspace=0.0)

plt.savefig(basedir/'reports'/'zeropoint.pdf',dpi=600)
plt.show()

the M* that we measure is smaller (fainter) than what we previously used. This would slightly decrease our measured distances

In [ ]:
np.random.seed(1)

N = 15
x0 = np.linspace(0,10)
x = np.random.uniform(0,10,N)
x.sort()
y = -4+2*x+np.random.normal(0,0.51,N)
yerr = np.random.normal(1,0.1,N)
y[5] +=6

bounds = [(-5,5), (-5,5), (0, 1), (-5,5), (-2, 4)]
p0 = np.array([0.0, 1, 0.9, 0, np.log(1.0)])

fitter = linearMLE(x,y,yerr,bounds)
fitter.fit(p0)
print('m={:.2f}+{:.2f}-{:.2f}, b={:.2f}+{:.2f}-{:.2f}'.format(*fitter.results['m'],*fitter.results['b']))
#print(fitter)
outlier=fitter.outlier()
ax = fitter.plot(xlim=[0,10],xlabel='x',ylabel='y')
ax.plot(x0,-4+2*x0,color='black',lw=1)

'''
model  = models.Polynomial1D(degree=1,c0=-4, c1=1)
LinearLSQFitter = fitting.LinearLSQFitter(calc_uncertainties=True)
fit = LinearLSQFitter(model,x,y,weights=1/yerr)
ax.plot(x,fit.c0.value+fit.c1.value*x,color='blue',lw=2)
print(f'm={fit.c1.value:.2f}+-{fit._stds.stds[1]:.2f}, b={fit.c0.value:.2f}+-{fit._stds.stds[0]:.2f}')
ax.set(xlim=[-1,10])
'''

plt.show()

### Literature search

the following galaxies are in Ciardullo+2002 and in Pilyugin

NGC0300, NGC5457 (M101), NGC3031 (M81), NGC2403, NGC0598 (M33), NGC4258, NGC0224 (M31), NGC3351

missing are

LMC,SMC (from ToribioSanCipriano+2017)

NGC5253,NGC3368,NGC3627

**this does not work**: the authors use different $M^*$. To properly do this, one has to go through all the papers by hand and find out what values was used. Since some papers used different values for different galaxies (or did not give any value at all) it becomes extremely tedious.

In [ ]:
# prepare the literature distances
messier_to_ngc = {
     'M001': 'NGC1952','M002': 'NGC7089','M003': 'NGC5272','M004': 'NGC6121','M005': 'NGC5904',
     'M006': 'NGC6405','M007': 'NGC6475','M008': 'NGC6523','M009': 'NGC6333','M010': 'NGC6254',
     'M011': 'NGC6705','M012': 'NGC6218','M013': 'NGC6205','M014': 'NGC6402','M015': 'NGC7078',
     'M016': 'NGC6611','M017': 'NGC6618','M018': 'NGC6613','M019': 'NGC6273','M020': 'NGC6514',
     'M021': 'NGC6531','M022': 'NGC6656','M023': 'NGC6494','M024': 'NGC6603','M026': 'NGC6694',
     'M027': 'NGC6853','M028': 'NGC6626','M029': 'NGC6913','M030': 'NGC7099','M031': 'NGC0224',
     'M032': 'NGC0221','M033': 'NGC0598','M034': 'NGC1039','M035': 'NGC2168','M036': 'NGC1960',
     'M037': 'NGC2099','M038': 'NGC1912','M039': 'NGC7092','M041': 'NGC2287','M042': 'NGC1976',
     'M043': 'NGC1982','M044': 'NGC2632','M046': 'NGC2437','M047': 'NGC2422','M048': 'NGC2548',
     'M049': 'NGC4472','M050': 'NGC2323','M051': 'NGC5194','M052': 'NGC7654','M053': 'NGC5024',
     'M054': 'NGC6715','M055': 'NGC6809','M056': 'NGC6779','M057': 'NGC6720','M058': 'NGC4579',
     'M059': 'NGC4621','M060': 'NGC4649','M061': 'NGC4303','M062': 'NGC6266','M063': 'NGC5055',
     'M064': 'NGC4826','M065': 'NGC3623','M066': 'NGC3627','M067': 'NGC2682','M068': 'NGC4590',
     'M069': 'NGC6637','M070': 'NGC6681','M071': 'NGC6838','M072': 'NGC6981','M073': 'NGC6994',
     'M074': 'NGC0628','M075': 'NGC6864','M076': 'NGC0650','M077': 'NGC1068','M078': 'NGC2068',
     'M079': 'NGC1904','M080': 'NGC6093','M081': 'NGC3031','M082': 'NGC3034','M083': 'NGC5236',
     'M084': 'NGC4374','M085': 'NGC4382','M086': 'NGC4406','M087': 'NGC4486','M088': 'NGC4501',
     'M089': 'NGC4552','M090': 'NGC4569','M091': 'NGC4548','M092': 'NGC6341','M093': 'NGC2447',
     'M094': 'NGC4736','M095': 'NGC3351','M096': 'NGC3368','M097': 'NGC3587','M098': 'NGC4192',
     'M099': 'NGC4254','M100': 'NGC4321','M101': 'NGC5457','M102': 'NGC5866','M103': 'NGC0581',
     'M104': 'NGC4594','M105': 'NGC3379','M106': 'NGC4258','M107': 'NGC6171','M108': 'NGC3556',
     'M109': 'NGC3992','M110': 'NGC0205'}

distances = ascii.read(basedir/'data'/'literature distances'/'latest.csv',delimiter=',',header_start=12,data_start=14)
distances.rename_column('Galaxy ID','gal_name')

def replace_values_in_string(text, args_dict):
    for key in args_dict.keys():
        text = text.replace(key, str(args_dict[key]))
    return text

distances['gal_name'] = [replace_values_in_string(s,{'IC ':'IC','MESSIER ':'M','NGC ':'NGC'}) for s in distances['gal_name']]
distances['gal_name'] = [messier_to_ngc.get(s,s) for s in distances['gal_name']]

In [ ]:
pnlf_sample = set(filter_table(distances,Method='PNLF')['gal_name'])
trgb_sample = set(filter_table(distances,Method='TRGB')['gal_name'])
cepheid_sample = set(filter_table(distances,Method='Cepheids')['gal_name'])

distance_sample = pnlf_sample & (trgb_sample|cepheid_sample)
abundance_sample = set(pilyugin['name'])

sample = distance_sample & abundance_sample

print(f'PNLF: {len(pnlf_sample)}\nTRGB: {len(trgb_sample)}\nCepheids: {len(cepheid_sample)}')
print(f'PNLF & (TRGB|Cepheids): {len(distance_sample)}\nAbundances: {len(abundance_sample)}')
print(f'combined: {len(sample)}')

combine tables with abundances and distances

In [ ]:
lit_dist = Table(names=['name','(m-M)_PNLF','err(m-M)_PNLF','(m-M)_ref','err(m-M)_ref',
                        'R0','g25','N_PN','N_ref','source'],dtype=[str]+8*[float]+[str])
for gal_name in sample-{'NGC0628','NGC3351','NGC5068'}:
    pnlf_dist = filter_table(distances,gal_name=gal_name,Method='PNLF')
    ref_dist = filter_table(distances,gal_name=gal_name,Method=['TRGB','Cepheids'])
    abund = pilyugin.loc[gal_name]
    err_pnlf = np.sqrt(np.sum(pnlf_dist['err']**2)/len(pnlf_dist))
    err_ref = np.sqrt(np.sum(ref_dist['err']**2)/len(ref_dist))
    
    lit_dist.add_row([gal_name,
                      pnlf_dist['m-M'].mean(),err_pnlf,
                      ref_dist['m-M'].mean(),err_ref,
                      abund['R0_O'],abund['g_r25_O'],
                      len(pnlf_dist),len(ref_dist),'literature'])

# we add our own data to the sample
for name in ['IC5332','NGC0628','NGC1365','NGC2835','NGC3351','NGC3627','NGC4321','NGC4535','NGC5068']:  
    lit_dist.add_row([name,
                      results.loc[name]['(m-M)'],results.loc[name]['err-(m-M)'],
                      sample_table.loc[name]['(m-M)'],sample_table.loc[name]['err(m-M)'],
                      abundance_gradients.loc[name]['R0'],abundance_gradients.loc[name]['g_r25'],
                      1,1,'PHANGS'])
    
lit_dist['logOH'] = lit_dist['R0']+0.25*lit_dist['g25']

lit_dist['dM*'] = lit_dist['(m-M)_PNLF']-lit_dist['(m-M)_ref']
lit_dist['M*'] = -4.47+lit_dist['dM*']
lit_dist['errM*'] = np.sqrt(lit_dist['err(m-M)_PNLF']**2 + lit_dist['err(m-M)_ref'])
lit_dist.sort('logOH')
lit_dist.add_index('name')

# M* for SMC is weird
lit_dist.loc['SMC']['M*']  = cepheids.loc['SMC']['M*']
lit_dist.loc['SMC']['dM*'] = cepheids.loc['SMC']['dM*']

# update the values to the fit
for k,v in Mmax_dict.items():
    lit_dist.loc[k]['M*'] = v

additional metallicities (SMC,LMC and NGC5253)
https://www.aanda.org/articles/aa/pdf/2013/02/aa20580-12.pdf

In [ ]:
from astropy.modeling import models, fitting

logOH = np.linspace(7,10)
logOH_sun = 8.69   # Asplund
deltaM = lambda OH: 0.928*OH**2-0.1091*OH+0.0036

# polynom
model  = models.Polynomial1D(degree=2,c0=0.014, c1=0.225, c2= 0.928)
fitter = fitting.LinearLSQFitter(calc_uncertainties=True)
fit = fitter(model,lit_dist['logOH'][lit_dist['source']=='literature']-logOH_sun,
             lit_dist['dM*'][lit_dist['source']=='literature'],
             weights=1/lit_dist['errM*'][lit_dist['source']=='literature'])
#fit = fitter(model,lit_dist['logOH']-logOH_sun,lit_dist['dM*'],weights=1/lit_dist['errM*'])

fig,ax = plt.subplots(nrows=1,figsize=(single_column,single_column/1.618))

ax.errorbar(lit_dist['logOH'][lit_dist['source']=='literature'],lit_dist['M*'][lit_dist['source']=='literature'],
            yerr=lit_dist['errM*'][lit_dist['source']=='literature'],capsize=0,
            fmt='o',color=tab10[1],mec=tab10[1],ms=2,elinewidth=0.5,label='literature',zorder=3)
ax.errorbar(lit_dist['logOH'][lit_dist['source']=='PHANGS'],lit_dist['M*'][lit_dist['source']=='PHANGS'],
            yerr=lit_dist['errM*'][lit_dist['source']=='PHANGS'],capsize=0,
            fmt='o',color=tab10[0],mec=tab10[0],ms=2,elinewidth=0.5,label='PHANGS',zorder=3)

ax.plot(logOH,-4.47+deltaM(logOH-logOH_sun),ls=':',lw=1.2,label='Ciardullo',color='gray',zorder=1)
#ax.plot(logOH, -4.47+fit(logOH-logOH_sun),ls='-',lw=0.8,label='fit',color='black',zorder=1)

halign = {'SMC':'right','NGC0055':'left','NGC5068':'left','NGC0598':'left',
          'NGC0628':'right','NGC6946':'left','NGC1365':'left','NGC5457':'left'}
valign = {'IC0010':'bottom','NGC5068':'bottom','LMC':'bottom','NGC0598':'bottom',
          'IC5332':'bottom','NGC6946':'bottom','NGC1365':'bottom','NGC3627':'bottom',
          'NGC0628':'bottom','NGC0253':'bottom','SMC':'bottom'}

for row in lit_dist:
    ax.text(row['logOH'],row['M*'],row['name'],
            ha=halign.get(row['name'],'right'),
            va=valign.get(row['name'],'top'),
            fontsize='3',zorder=4,rotation=90)


plt.locator_params(axis='y',nbins=5)

# constant
model  = models.Polynomial1D(degree=1,c0=-4.5, c1=0)
model.c1.fixed=True
sub = lit_dist[lit_dist['logOH']>8.3]

fit = fitter(model,sub['logOH'][sub['source']=='PHANGS'],sub['M*'][sub['source']=='PHANGS'],weights=1/sub['errM*'][sub['source']=='PHANGS'])
print(f'PHANGS: M*={fit.c0.value:.2f}+-{fit._stds.stds[0]:.2f}')

fit = fitter(model,sub['logOH'],sub['M*'],weights=1/sub['errM*'])
print(f'All: M*={fit.c0.value:.2f}+-{fit._stds.stds[0]:.2f}')

ax.axhline(fit.c0.value,ls='-',lw=0.8,color='k',zorder=2,label='fit')

'''
# linear
model  = models.Polynomial1D(degree=1)
fit = fitter(model,lit_dist['logOH'],lit_dist['M*'],weights=1/lit_dist['errM*'])
ax.plot(logOH,fit(logOH),ls='--',lw=0.8,color='black',zorder=1)
'''

ax.set(xlim=[8,8.82],ylim=[-5.2,-3.2],ylabel=r'$M^*$ / mag')
ax.set(xlim=[8,8.82],ylim=[-5.2,-3.2],xlabel=r'$12+\log (\mathrm{O}/\mathrm{H})$',ylabel=r'$M^*$ / mag')
ax.invert_yaxis()
ax.legend(ncol=2,loc=4)

plt.savefig(basedir/'reports'/'zeropoint_literature.pdf',dpi=600)
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(5,5))

for row in lit_dist[lit_dist['source']=='literature']:
    if row['name'] in cepheids['name']:
        ax.scatter(row['M*'],cepheids.loc[row['name']]['M*'])
ax.plot([-4.8,-4.1],[-4.8,-4.1],c='k')
ax.set(xlabel='M* literature',ylabel='M* Ciardullo+2002')
ax.invert_xaxis()
ax.invert_yaxis()
plt.show()

### fix $(m-M)$ and fit $M*$

we fix $(m-M)$ to the TRGB value and leave $M*$ as a free parameter 

IC5332,NGC0628,NGC1365,NGC1433,NGC1512,NGC1566,NGC2835,NGC3351,NGC3627,NGC4321,NGC5068

In [ ]:
catalogue = Table(fits.getdata(basedir/'data'/'catalogues'/f'nebulae.fits',ext=1))
#catalogue['overluminous'] = catalogue['note']=='OL'
#catalogue['exclude'] = catalogue['note']=='EX'
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])

In [ ]:
from pnlf.auxiliary import parsec_to_mu

parsec_to_mu(15.44*u.Mpc, 1.62*u.Mpc)

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, pnlf

name = 'NGC2835'

trgb_distances = {'IC5332': (29.77,0.1),
 'NGC0628': (29.96,0.14),
 'NGC1365': (31.46,0.09),
 'NGC2835': (30.44,0.17),
 'NGC3351': (29.99,0.07),
 'NGC3627': (30.27,0.09),
 'NGC4321': (30.94,0.23),
 'NGC5068': (28.58,0.09)}

# literature value and our measured distance
mu_trgb, mu_trgb_err = trgb_distances[name]
mu_pnlf =  results.loc[name]['(m-M)']
dM = mu_pnlf-mu_trgb
dM_err = np.sqrt(mu_trgb_err**2+results.loc[name]['err-(m-M)']**2)
# the initial guess
Mmax0 = -4.47+dM
# define a range of values to test
Mmax_lst = np.linspace(Mmax0-dM_err,Mmax0+dM_err,6)
#Mmax_lst = np.linspace(-4.75,-4.5,5)
completeness = parameters[name]['completeness_limit']

data = catalogue[(catalogue['gal_name']==name) & (catalogue['type']=='PN') & catalogue['OIII5006_detection'] &
             ~catalogue['exclude'] & ~catalogue['overluminous'] & (catalogue['mOIII']<completeness)]

dM_lst = []
dMerr_lower_lst = []
dMerr_upper_lst = []
for Mmax in Mmax_lst:
    # update zero point
    fitter = MaximumLikelihood1D(pnlf,data['mOIII'],err=data['dmOIII'],mhigh=completeness,Mmax=Mmax)
    mu,mu_p,mu_m = fitter([30])
    dM_lst.append(mu-mu_trgb)
    dMerr_lower_lst.append(mu_m)
    dMerr_upper_lst.append(mu_p)

dM_lst = np.array(dM_lst)
dMerr_lower_lst = np.array(dMerr_lower_lst)
dMerr_upper_lst = np.array(dMerr_upper_lst)

# when we calculate the difference, we need to include the trgb error in the uncertainties
dMerr_lower_lst = np.sqrt(mu_trgb_err**2+dMerr_lower_lst**2)
dMerr_upper_lst = np.sqrt(mu_trgb_err**2+dMerr_upper_lst**2)

Mmax_interp = np.interp(0,dM_lst[::-1],Mmax_lst[::-1])
Mmax_lower = Mmax_interp-np.interp(0,dM_lst[::-1]-dMerr_lower_lst[::-1],Mmax_lst[::-1])
Mmax_upper = np.interp(0,dM_lst[::-1]+dMerr_upper_lst[::-1],Mmax_lst[::-1])-Mmax_interp

print(f'M* = {Mmax_interp:.3f}+{Mmax_upper:.3f}-{Mmax_lower:.3f}')
print(f'M* = ({Mmax_interp:.3f},{Mmax_upper:.3f},{Mmax_lower:.3f})')

take a look at what we are calculating

In [ ]:
fig,ax=plt.subplots()
ax.errorbar(Mmax_lst,dM_lst,yerr=[dMerr_lower_lst,dMerr_upper_lst],color='tab:blue')
ax.fill_between(Mmax_lst,dM_lst-dMerr_lower_lst,dM_lst+dMerr_upper_lst,color='tab:blue',alpha=0.2)
#ax.plot(np.array(Mmax_lst),np.array(dM_lst)-np.array(dMerr_lst),color='tab:blue')
#ax.plot(np.array(Mmax_lst),np.array(dM_lst)+np.array(dMerr_lst),color='tab:blue')

ax.axhline(0,color='black')
ax.plot([Mmax_interp-Mmax_lower,Mmax_interp+Mmax_upper],[0,0],color='tab:red')

#ax.axvline(Mmax0,ls='--')
ax.axvline(Mmax_interp)

ax.set(xlabel='M* / mag',ylabel=r'$(m-M)_\mathrm{PNLF}-(m-M)_\mathrm{TRGB}$')

plt.show()

In [ ]:
better_Mmax = {'IC5332': (-4.503,0.122,0.169),
 'NGC0628': (-4.527,0.133,0.135),
 'NGC1365':  (-4.624,0.071,0.069),
 'NGC2835': (-4.360,0.165,0.184),
 'NGC3351': (-4.139,0.087,0.103),
 'NGC3627':(-4.540,0.096,0.116),
 'NGC4321': (-4.337,0.211,0.194),
 'NGC5068': (-4.570,0.124,0.204)
                }

we use the same fitting algorithm, but fix (m-M) and fit M*

**not really working**

In [ ]:
from pnlf.plot.pnlf import plot_pnlf
from pnlf.analyse import F, MaximumLikelihood1D
from scipy.optimize import minimize

def pnlf_Mmax(m,Mmax,mu,mhigh):

    m = np.atleast_1d(m)
    mlow = Mmax+mu
    
    normalization = 1/(F(mhigh,mu) - F(mlow,mu))    
    out = normalization * np.exp(0.307*(m-mu)) * (1-np.exp(3*(Mmax-m+mu)))
    out[(m>mhigh) | (m<mlow)] = 0
    
    return out

def gaussian(x,mu,sig):
    return 1/np.sqrt(2*np.pi*sig**2) * np.exp(-(x-mu)**2/(2*sig**2))

def prior(param):
    return gaussian(param,-4.47,0.1)

name= 'NGC0628'

mu_trgb = sample_table.loc[name]['(m-M)']
completeness = parameters[name]['completeness_limit']
binsize = parameters[name]['binsize']

data = catalogue[(catalogue['gal_name']=='NGC0628') & (catalogue['type']=='PN') & catalogue['OIII5006_detection'] &
             ~catalogue['exclude'] & ~catalogue['overluminous'] & (catalogue['mOIII']<28)]

fitter = MaximumLikelihood1D(pnlf_Mmax,data['mOIII'],err=data['dmOIII'],
                             prior=None,mu=mu_trgb,mhigh=completeness)
Mmax,mp,mm = fitter.fit(-4.5)
#Mmax = minimize(fitter.likelihood,[-4.47],method=fitter.method).x[0]

mlow = Mmax+mu_trgb
mhigh = 28.5

dMexp =  results.loc[name]['(m-M)']-mu_trgb
print(f'expected: Mmax={-4.47+dMexp:.2f}, dMmax={dMexp:.2f}')
print(f'{name}: Mmax={Mmax:.2f}, dMmax={Mmax+4.47:.2f}')

#axes = plot_pnlf(data['mOIII'],mu_trgb,completeness,binsize=binsize,
#                 mhigh=28.5,Mmax=Mmax,filename=None,color=tab10[0])

In [ ]:
fitter = MaximumLikelihood1D(pnlf_Mmax,data['mOIII'],mu=mu_trgb,mhigh=completeness)

x_arr = np.linspace(-4,-6,100)
evidence = [fitter.evidence(x) for x in x_arr]

In [ ]:
Mmax = np.linspace(-6,-4,100)
evidence = [np.sum(np.log(pnlf_Mmax(data['mOIII'],mmax,mu=29.99,mhigh=28))) for mmax in Mmax]

In [ ]:
nbins = 100
x = np.linspace(-4.4,-4.6,nbins)
y = np.linspace(29.8,30.3,nbins)

X,Y = np.meshgrid(x,y)

evidence = np.zeros((len(x),len(y)))

for i in range(len(x)):
    for j in range(len(y)):
        evidence[i,j] = np.sum(np.log(pnlf(data['mOIII'],Y[i,j],Mmax=X[i,j],mhigh=28)))

In [ ]:
for x in [4.4,4.47,4.54]:
    idx = np.argmin(np.abs(X[0]+x))
    plt.plot(Y[:,idx],evidence[:,idx],label=x)
plt.legend()
plt.show()

In [ ]:
for y in [29.8,29.9,30,30.1,30.2]:
    idx = np.argmin(np.abs(Y[:,1]-y))
    print(idx)
    plt.plot(X[idx,:],evidence[idx,:],label=y)
plt.legend()
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(8,8))

ax.pcolormesh(X,Y,evidence)
ax.axvline(-4.47,color='red')
ax.axhline(mu_trgb,color='red')
plt.show()


In [ ]:
lim = [29.7,30.3]
mu = np.linspace(*lim,100)
evidence = [np.sum(np.log(pnlf(data['mOIII'],x,Mmax=-4.37,mhigh=28))) for x in mu]
fig,ax=plt.subplots()
ax.plot(mu,evidence)
ax.set(xlim=lim)
plt.show()

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, pnlf

name = 'NGC3351'

# get an initial guess
mu_trgb = sample_table.loc[name]['(m-M)']
mu_pnlf =  results.loc[name]['(m-M)']
dM = mu_pnlf-mu_trgb
Mmax0 = -4.47+dM
completeness = parameters[name]['completeness_limit']

data = catalogue[(catalogue['gal_name']==name) & (catalogue['type']=='PN') & catalogue['OIII5006_detection'] &
             ~catalogue['exclude'] & ~catalogue['overluminous'] & (catalogue['mOIII']<completeness)]


fitter = MaximumLikelihood1D(pnlf,data['mOIII'],err=data['dmOIII'],mhigh=completeness,Mmax=Mmax0)
mu,mp,mm = fitter([29])

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, F
from pnlf.plot.pnlf import _plot_pnlf
from scipy.optimize import minimize

names = list(trgb['name'])

names.remove('NGC1433')
names.remove('NGC1512')

nrows = 3
ncols = 3
filename = None #basedir / 'reports' / f'all_galaxies_PNLF'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------

width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

trgb['dM*new'] = 0.0
for name in names:
    
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
    else:
        print(f'no catalogue for {name}')
        continue
        
    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0
        
    # pre-process the data for the plot and read additional parameters
    data = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude'])]['mOIII']
    err = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude'])]['dmOIII']

    mu_trgb = trgb.loc[name]['trgb_(m-M)']
    completeness = parameters[name]['completeness_limit']

    binsize = parameters[name]['binsize']

    fitter = MaximumLikelihood1D(pnlf_Mmax,data[data<completeness],err=err[data<completeness],prior=prior,mu=mu_trgb,mhigh=completeness)
    Mmax = minimize(fitter.likelihood,[-4.47],method=fitter.method).x[0]
    
    trgb.loc[name]['dM*new'] = Mmax+4.47
    
    mlow = Mmax+mu_trgb
    mhigh = 28.5
    
    print(f'{name}: Mmax={Mmax:.2f}, dMmax={Mmax+4.47:.2f}')
    
    ax=_plot_pnlf(data,mu_trgb,completeness,binsize=binsize,mlow=mlow,mhigh=mhigh,ax=ax,ms=3)
    ax.text(0.4,0.08,f'{name}', transform=ax.transAxes,fontsize=7)

    #ax.set_xlim([mu-5,completeness+0.5])
    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'N')
    #ax.set_title(name)
    #ax.set(xlim=[24,28.5])
    
plt.show()

## Radial Trends 

measure (m-M) from inner and outer PNe

the metallicity and with it $M*$ decreases in the outer regions of the galaxies. A smaller $M*$ should lead to a larger distance. Therefore the PNe in the outer parts should yield a larger distance.

In [ ]:
abundance_gradients = ascii.read(basedir/'data'/'external'/'radial_abundance_gradients.txt',
                                names=['name','R0','g_r25'])
abundance_gradients.add_index('name')

In [ ]:
from regions import EllipseSkyRegion

name = 'NGC0628'

logOH_sun = 8.87
deltaM = lambda OH: 0.928*OH**2+0.225*OH+0.014

try:
    mu_trgb = trgb.loc[name]['trgb_(m-M)']
except:
    print(f'no TRGB for {name}')
    mu_trgb=0
completeness = parameters[name]['completeness_limit']
binsize = parameters[name]['binsize']

catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
    catalogue['overluminous'] = catalogue['overluminous'].astype(bool)

catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude']) & (~catalogue['overluminous']) & (catalogue['mOIII']<completeness)]

center = sample_table.loc[name]['SkyCoord']
posang = sample_table.loc[name]['posang']
inclination = sample_table.loc[name]['Inclination']
eccentricity = np.sin(inclination*u.deg).value

r25 = sample_table.loc[name]['r25']*u.arcmin

catalogue['r'] = catalogue['SkyCoord'].separation(center)
rmean = np.mean(catalogue['r']/r25).decompose()
logOH_rmean = abundance_gradients.loc[name]['R0']+rmean*abundance_gradients.loc[name]['g_r25']

print(f'rmean = {rmean:.2f} r25')
print(f'dM*={deltaM(logOH_rmean-logOH_sun):.3f}')

with fits.open(data_ext/'MUSE_DR2.1'/'MUSEDAP'/f'{name}_MAPS.fits') as hdul:
    wcs = WCS(hdul['OIII5006_FLUX '].header)
    OIII = hdul['OIII5006_FLUX'].data
    Halpha = hdul['HA6562_FLUX'].data
    
catalogue['region'] = '     '

### split sample into sub-samples

In [ ]:
def split_radial(catalogue,r25,center,eccentricity,posang,wcs):
    threshold = 0.5
    tried = set()
    while True: 
        width = threshold*r25
        aperture = EllipseSkyRegion(center,
                                width=width,
                                height=np.sqrt((width)**2 * (1-eccentricity**2)),
                                angle=(posang-90)*u.deg)
        inside = aperture.contains(catalogue['SkyCoord'],wcs)
        ratio = np.sum(inside)/np.sum(~inside)

        if threshold in tried:
            break
        tried.add(threshold)

        if np.abs(ratio-1)<0.02:
            break
        elif ratio>1:
            threshold/=1.02
        elif ratio<1:
            threshold*=1.02

    width = threshold*r25
    aperture = EllipseSkyRegion(center,
                            width=width,
                            height=np.sqrt((width)**2 * (1-eccentricity**2)),
                            angle=(posang-90)*u.deg)
    catalogue['region'] = 'inner'
    catalogue['region'][~aperture.contains(catalogue['SkyCoord'],wcs)] = 'outer'
    print(f'width={threshold:.3f} r25')

    return catalogue,threshold,aperture

def split_quadrants(catalogue,wcs,posang):
    
    catalogue['region'] = 'south'
    x0,y0 = center.to_pixel(wcs)
    x,y = catalogue['x'],catalogue['y']

    north = (y>y0-np.sin((posang)/180*np.pi)*(x-x0))
    south = (y<y0-np.sin((posang)/180*np.pi)*(x-x0))
    east = (x<x0+np.cos((posang-90)/180*np.pi)*(y-y0))
    west = (x>x0+np.cos((posang-90)/180*np.pi)*(y-y0))

    catalogue['region'][north & west] = 'nw'
    catalogue['region'][north & east] = 'ne'
    catalogue['region'][south & west] = 'sw'
    catalogue['region'][south & east] = 'se'
    
    return catalogue

split into inner/outer

In [ ]:
catalogue,threshold,aperture = split_radial(catalogue,r25,center,eccentricity,posang,wcs)

logOH1 = abundance_gradients.loc[name]['R0']
logOH2 = abundance_gradients.loc[name]['R0']+threshold*abundance_gradients.loc[name]['g_r25']

print(f'inner dM*={deltaM(logOH1-logOH_sun):.3f}\noutter dM*={deltaM(logOH2-logOH_sun):.3f}')

or split into quadrants

In [ ]:
catalogue['region'] = 'south'

catalogue['region'][(center.ra<catalogue['SkyCoord'].ra) & (center.dec<catalogue['SkyCoord'].dec)] = 'north'
catalogue['region'][(center.ra<catalogue['SkyCoord'].ra) & (center.dec>catalogue['SkyCoord'].dec)] = 'west'
catalogue['region'][(center.ra>catalogue['SkyCoord'].ra) & (center.dec<catalogue['SkyCoord'].dec)] = 'east'
catalogue['region'][(center.ra>catalogue['SkyCoord'].ra) & (center.dec>catalogue['SkyCoord'].dec)] = 'south'


In [ ]:
catalogue['region'] = 'west'

catalogue['region'][(center.ra<catalogue['SkyCoord'].ra)] = 'west'
catalogue['region'][(center.ra>catalogue['SkyCoord'].ra)] = 'east'


In [ ]:
catalogue['region'] = 'south'

catalogue['region'][(center.dec<catalogue['SkyCoord'].dec)] = 'north'
catalogue['region'][(center.dec>catalogue['SkyCoord'].dec)] = 'south'

In [ ]:
# split into 4 quadrants based on position angel
catalogue = split_quadrants(catalogue,wcs,posang)

compare central PNe to outer PNe

In [ ]:
def plot_quadrants(image,catalogue,wcs,aperture,posang,ax=None):

    x0,y0 = center.to_pixel(wcs)
    dx = 1000

    majx1 = x0-np.cos(posang/180*np.pi)*dx
    majx2 = x0+np.cos(posang/180*np.pi)*dx
    majy1 = y0-np.sin(posang/180*np.pi)*dx
    majy2 = y0+np.sin(posang/180*np.pi)*dx

    minx1 = x0-np.cos((posang-90)/180*np.pi)*dx
    minx2 = x0+np.cos((posang-90)/180*np.pi)*dx
    miny1 = y0-np.sin((posang-90)/180*np.pi)*dx
    miny2 = y0+np.sin((posang-90)/180*np.pi)*dx

    if not ax:
        fig = plt.figure(figsize=(single_column,single_column))
        ax = fig.add_subplot(projection=wcs)

    norm = simple_norm(image,clip=False,percent=97)
    ax.imshow(image,norm=norm,cmap=plt.cm.Greys)

    colors = iter(tab10)
    for region in np.unique(catalogue['region']):
        color = next(colors)
        tmp1 = catalogue[catalogue['region']==region]

        tmp = tmp1[aperture.contains(tmp1['SkyCoord'],wcs)]
        sc = ax.errorbar(tmp['x'],tmp['y'],mec=color,mfc=color,fmt='o',ms=3,label=region)

        tmp = tmp1[~aperture.contains(tmp1['SkyCoord'],wcs)]
        ax.errorbar(tmp['x'],tmp['y'],fmt='o',mec=color,mfc='none',ms=3,mew=0.9)


    pixel_aperture = aperture.to_pixel(wcs)
    artist = pixel_aperture.as_artist(ec='black')
    ax.add_artist(artist)

    ax.plot([majx1,majx2],[majy1,majy2],color='black')
    ax.plot([minx1,minx2],[miny1,miny2],color='black')

    ax.set(xlim=(0,OIII.shape[1]),ylim=(0,OIII.shape[0]))
    #ax.set_title(name)

    ax.coords[0].set_ticks_visible(False)
    ax.coords[0].set_ticklabel_visible(False)
    ax.coords[1].set_ticks_visible(False)
    ax.coords[1].set_ticklabel_visible(False)
    
    #ax.legend()
    
    return ax

ax = plot_quadrants(OIII,catalogue,wcs,aperture,posang,ax=None)
#plt.savefig(basedir/'reports'/name/f'{name}_regions.pdf',dpi=600)
plt.show()

### for a single galaxy

In [ ]:
from scipy.stats import ks_2samp

reg = np.unique(catalogue['region'])
data1 = catalogue[catalogue['region']==reg[0]]['mOIII']
data2 = catalogue[catalogue['region']==reg[1]]['mOIII']

ks,pv = ks_2samp(data1,data2)
print(f'{name}: statistic={ks:.3f}, pvalue={pv:.3f}')

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, pnlf, cdf
from pnlf.plot.pnlf import plot_pnlf
from scipy.stats import kstest

regions = np.unique(catalogue['region'])
if 'axes' in locals():
    del axes
pdfs = {}
for i,region in enumerate(regions):
    
    data = catalogue[catalogue['region']==region]['mOIII']
    err  = catalogue[catalogue['region']==region]['dmOIII']
    
    if len(data)<15:
        print(f'not enough data points ({len(data)}) for region {region}')
        continue
        
    fitter = MaximumLikelihood1D(pnlf,
                                 data[data<completeness],
                                 err=err[data<completeness],
                                 mhigh=completeness,Mmax=-4.47)
    mu,mu_p,mu_m = fitter([29])
    pdfs[region] = (fitter.x_arr,fitter.likelihood_arr)
    print('{}: {:.2f} + {:.2f} - {:.2f}'.format(region,mu,mu_p,mu_m))


    #Plot PNLF
    if 'axes' not in locals():
        axes = plot_pnlf(data,mu,completeness,
                 binsize=binsize,mhigh=28.5,Mmax=-4.47,color=tab10[i])
    else:
        axes = plot_pnlf(data,mu,completeness,
                         binsize=binsize,mhigh=28.5,Mmax=-4.47,filename=None,color=tab10[i],axes=axes)
    
ax1,ax2 = axes 
h, l = ax2.get_legend_handles_labels()
ax2.legend(h,regions)
filename=basedir/'reports'/name/f'{name}_PNLF_radial'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')

calculate error of difference with Monte Carlo (the original errors are not gaussian and hence we can not use classical error propagation)

In [ ]:
from pnlf.auxiliary import sample_numerical

x,y=pdfs['inner']
mu_inner = sample_numerical(x,y,100000)

x,y=pdfs['outer']
mu_outer = sample_numerical(x,y,100000)

In [ ]:
from pnlf.analyse import gaussian

fig,(ax1,ax2) = plt.subplots(ncols=2,figsize=(8,3),gridspec_kw={'width_ratios': [1.5, 1]})

ax1.hist(mu_inner,bins=np.arange(29,30.5,0.01),density=True,label='inner',alpha=0.5)
ax1.hist(mu_outer,bins=np.arange(29,30.5,0.01),density=True,label='outer',alpha=0.5)
ax1.set(xlabel='$(m-M)$',xlim=[29,30.5])
ax1.legend(loc=2)

diff = mu_outer-mu_inner
mu = np.mean(diff)
std = np.std(diff)
print(f'{mu:.2f}+-{std:.2f}')
x = np.linspace(0,1)
ax2.hist(diff,bins=np.arange(0,1,0.01),density=True,color='#9E91A8',alpha=0.6)
ax2.plot(x,gaussian(x,mu,std),color='black')
ax2.set(xlabel=r'$\Delta(m-M)$',xlim=[0,1])
plt.show()

NGC0628

inner outer: statistic=0.272, pvalue=0.006
inner: 29.78 + 0.08 - 0.15
outer: 30.21 + 0.07 - 0.13


statistic=0.244, pvalue=0.207
ne: 29.81 + 0.10 - 0.21
nw: 29.82 + 0.11 - 0.25
se: 30.15 + 0.09 - 0.19
sw: 29.94 + 0.10 - 0.23

NGC3351

inner outer: statistic=0.187, pvalue=0.141
inner: 30.35 + 0.08 - 0.13
outer: 30.32 + 0.08 - 0.13

vergleicht nur 2 der 4
ne: 30.22 + 0.10 - 0.19
nw: 30.38 + 0.09 - 0.18
se: 30.40 + 0.10 - 0.21
sw: 30.45 + 0.10 - 0.21


NGC1433

NGC1433: statistic=0.170, pvalue=0.508
inner: 31.37 + 0.07 - 0.10
outer: 31.37 + 0.07 - 0.10


NGC1433: statistic=0.222, pvalue=0.588
ne: 31.29 + 0.09 - 0.19
nw: 31.41 + 0.09 - 0.16
se: 31.42 + 0.09 - 0.16
w: 31.36 + 0.08 - 0.14


### for all galaxies at once

In [ ]:
from pnlf.auxiliary import project

def calc_r25(name,x,y):
    '''calculate deprojected r25
    
    '''

    # get the pixel position of the centre
    with fits.open(data_ext/'MUSE_DR2.1'/'MUSEDAP'/f'{name}_MAPS.fits') as hdul:
        wcs = WCS(hdul['FLUX'].header)
    centre = sample_table.loc[name]['SkyCoord']
    x_cen,y_cen = centre.to_pixel(wcs)
    
    pa  = sample_table.loc[name]['posang']
    inc = sample_table.loc[name]['Inclination']
    r25 = sample_table.loc[name]['r25']*u.arcmin
    
    # deproject
    x_depr,y_depr = project(x-x_cen,y-y_cen,pa,inc)
    skycoord_depr = SkyCoord.from_pixel(x_depr+x_cen,y_depr+y_cen,wcs)
    
    # separation to centre
    sep = skycoord_depr.separation(centre)
    
    return (sep/r25).decompose()

In [ ]:
from regions import EllipseSkyRegion
from scipy.stats import ks_2samp
from pnlf.auxiliary import sample_numerical
from pnlf.analyse import MaximumLikelihood1D,pnlf

sample = results[results['N_PN']>50]['name']
ncols = 4
nrows = np.ceil(len(sample)/ncols)

fig1 = plt.figure(figsize=(two_column,two_column*nrows/ncols))
fig2 = plt.figure(figsize=(two_column,two_column*nrows/ncols))

logOH_sun = 8.87
deltaM = lambda OH: 0.928*OH**2+0.225*OH+0.014

tbl=Table(dtype=['str']+12*[float],names=['name','(m-M)1','err+(m-M)1','err-(m-M)1',
                                            '(m-M)2','err+(m-M)2','err-(m-M)2','d(m-M)','errd(m-M)',
                                            'ks','pv','r1','r2'])
for i,name in enumerate(sample):
    
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
    catalogue['overluminous'] = catalogue['overluminous'].astype(bool)
    catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])

    # some parameters of the galaxy
    completeness = parameters[name]['completeness_limit']
    binsize = parameters[name]['binsize']
    center = sample_table.loc[name]['SkyCoord']
    posang = sample_table.loc[name]['posang']
    inclination = sample_table.loc[name]['Inclination']
    eccentricity = np.sin(inclination*u.deg).value
    #r25 = sample_table.loc[name]['r25']*u.arcmin
    #catalogue['r'] = catalogue['SkyCoord'].separation(center)
    catalogue['r'] = calc_r25(name,catalogue['x'],catalogue['y'])
    
    catalogue = catalogue[(catalogue['type']=='PN') & (catalogue['mOIII']<completeness) & (~catalogue['overluminous']) & (~catalogue['exclude'])]
    print(f'{name}: {len(catalogue)} objects')
    
    with fits.open(data_ext/'MUSE_DR2.1'/'MUSEDAP'/f'{name}_MAPS.fits') as hdul:
        wcs = WCS(hdul['OIII5006_FLUX '].header)
        OIII = hdul['OIII5006_FLUX'].data
        Halpha = hdul['HA6562_FLUX'].data
    
    # split evenly between inner and outer
    catalogue,threshold,aperture = split_radial(catalogue,r25,center,eccentricity,posang,wcs)
    # split into quadrants
    #catalogue = split_quadrants(catalogue,wcs,posang)
    
    # plot with the position of the PN and split into quadrants/radial
    ax1 = fig1.add_subplot(nrows,ncols,i+1,projection=wcs)
    ax1 = plot_quadrants(OIII,catalogue,wcs,aperture,posang,ax=ax1)
    t = ax1.text(0.05,0.9,f'{name}', transform=ax1.transAxes,fontsize=6)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

    # the cumulative luminosity function plot
    ax2 = fig2.add_subplot(nrows,ncols,i+1)
    regions = np.unique(catalogue['region'])
    colors = iter(tab10)
    for j,region in enumerate(regions):
        data = catalogue[catalogue['region']==region]['mOIII']
        N = len(data)        
        color = next(colors)
        data.sort()
        ax2.plot(data,np.arange(1,N+1,1),ls='-',mfc=color,mec=color,ms=1,marker='o',label=region)

    reg = np.unique(catalogue['region'])
    data1 = catalogue[catalogue['region']==reg[0]]['mOIII']
    data2 = catalogue[catalogue['region']==reg[1]]['mOIII']
    ks,pv = ks_2samp(data1,data2)
    print(f'{name}: statistic={ks:.3f}, pvalue={pv:.3f}')
    
    if True:
        # this measures the distance for each subsample
        fitter = MaximumLikelihood1D(pnlf,
                                     data1,
                                     err=catalogue[catalogue['region']==reg[0]]['dmOIII'],
                                     mhigh=completeness,Mmax=-4.47)
        mu1,mu_p1,mu_m1 = fitter([29])
        x,y=fitter.x_arr,fitter.likelihood_arr
        mu_inner = sample_numerical(x,y,100000)
        
        fitter = MaximumLikelihood1D(pnlf,
                                     data2,
                                     err=catalogue[catalogue['region']==reg[1]]['dmOIII'],
                                     mhigh=completeness,Mmax=-4.47)
        mu2,mu_p2,mu_m2 = fitter([29])
        x,y=fitter.x_arr,fitter.likelihood_arr
        mu_outer = sample_numerical(x,y,100000)    

        diff = mu_outer-mu_inner
        dmu = np.mean(diff)
        std = np.std(diff)
        
        center = sample_table.loc[name]['SkyCoord']
        r25 = sample_table.loc[name]['r25']*u.arcmin
        r1 = np.mean(catalogue[catalogue['region']==reg[0]]['r'])
        r2 = np.mean(catalogue[catalogue['region']==reg[1]]['r'])
        tbl.add_row([name,mu1,mu_p1,mu_m1,mu2,mu_p2,mu_m2,dmu,std,ks,pv,r1,r2])
    
    ax2.text(0.57,0.08,f'{name}', transform=ax2.transAxes,fontsize=7)
    ax2.text(0.05,0.78,f'$p$-value$={pv:.2f}$',transform=ax2.transAxes,fontsize=7)
    ax2.text(0.07,0.88,f'$D_{{max}}={ks:.3f}$', transform=ax2.transAxes,fontsize=7)
    ax.set(aspect='equal')
    
    if i%ncols==0:
        ax2.set(ylabel='Cumulative N')
    if i//ncols==nrows-1:
        ax2.set(xlabel=r'$m_{[\mathrm{OIII}]}$ / mag')

    if i==5:
        ax2.set(xlabel=r'$m_{[\mathrm{OIII}]}$ / mag')

# Create the legend

h,l = ax2.get_legend_handles_labels()
ax2 = fig2.add_subplot(nrows,ncols,i+2)
ax2.legend(h,l,
           loc=2
           #ncol=2,
           #loc="upper center",   # Position of legend
           #borderaxespad=0.1,    # Small spacing around legend box
           )
ax2.axis('off')

fig1.tight_layout() 
fig2.tight_layout() 

fig1.savefig(basedir/'reports'/'subsamples_map.png',dpi=600,bbox_inches='tight')
fig2.savefig(basedir/'reports'/'subsamples_PNLF_cum.png',dpi=600,bbox_inches='tight')

plt.show()

In [ ]:
tbl = join(tbl,abundance_gradients,keys='name')
tbl['logOH1'] = tbl['R0'] +tbl['r1']*tbl['g_r25']
tbl['logOH2'] = tbl['R0'] +tbl['r2']*tbl['g_r25']
tbl['dlogOH'] = tbl['logOH2']-tbl['logOH1'] 
    
tbl['dM1'] = deltaM(tbl['logOH1']-logOH_sun)
tbl['dM2'] = deltaM(tbl['logOH2']-logOH_sun)
tbl['d(m-M)'] = tbl['(m-M)2']-tbl['(m-M)1']
for col in tbl.columns[1:]:
    tbl[col].info.format='%.3f'
    
with open(basedir/'data'/'interim'/ 'pnlf_io.txt','w',newline='\n') as f:
    ascii.write(tbl,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')

### Look at dM* as a function of g r25

In [ ]:
from astropy.table import join 

tbl = ascii.read(basedir/'data'/'interim'/'pnlf_io.txt')
tmp = join(tbl,results,keys='name')
#tbl['err_d(m-M)'] = np.sqrt(tbl['err+(m-M)2']**2+tbl['err-(m-M)1']**2)

In [ ]:
ascii.write(tbl[['name','dlogOH','d(m-M)']],sys.stdout, Writer = ascii.Latex)

In [ ]:
ha = {'NGC4535':'left','NGC4321':'right','NGC3351':'right','NGC3627':'right','NGC1433':'right'}

fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

for row in tbl:
    ax.text(row['logOH2']-row['logOH1'],row['d(m-M)'],row['name'],
           ha=ha.get(row['name'],'left'),va='top',fontsize=7)
ax.errorbar(tbl['logOH2']-tbl['logOH1'],tbl['d(m-M)'],yerr=tbl['errd(m-M)'],fmt='o',ms=4)

#ax.scatter(tmp['logOH2']-tmp['logOH1'],deltaM(tmp['logOH2']-logOH_sun)-deltaM(tmp['logOH1']-logOH_sun),color='tab:orange')

ax.set(xlim=[-0.065,0.01],ylim=[-0.5,1],
       xlabel=r'$\Delta \log (\mathrm{O}/\mathrm{H})$',ylabel='$\Delta (m-M)$')
#plt.savefig(basedir/'reports'/'inner_outer_difference.pdf',dpi=600)
plt.show()

In [ ]:
fig,ax=plt.subplots()

for row in tbl:
    ax.text(row['dM1']-row['dM2'],row['d(m-M)'],row['name'])
ax.scatter(tbl['dM1']-tbl['dM2'],tbl['d(m-M)'])
ax.set(xlim=[-0.06,0.02],ylim=[-0.3,0.6],xlabel=r'$\Delta$ M*',ylabel='$\Delta$ (m-M)')

plt.show()

In [ ]:
fig,ax=plt.subplots()

for row in tbl:
    ax.text(row['logOH1'],row['(m-M)1'],row['name'])
ax.scatter(tbl['logOH1'],tbl['(m-M)1'],label='inner')
ax.scatter(tbl['logOH2'],tbl['(m-M)2'],label='outer')

for row in tbl:
    ax.arrow(row['logOH1'],row['(m-M)1'],
             row['logOH2']-row['logOH1'],row['(m-M)2']-row['(m-M)1'],
            )

ax.set(xlim=[8.3,8.6],ylim=[28.2,31.7],xlabel=r'12+logO/H',ylabel='(m-M)')
plt.legend(loc=2)
plt.show()

## luminosity-specific planetary nebula number

In [ ]:
from astropy.table import join 

from pnlf.analyse import F

def NPN(mu,completeness,N_total,deltaM):
    cutoff = mu - 4.47
    p_deltaM = (F(cutoff+deltaM,mu) - F(cutoff,mu)) / (F(completeness,mu) - F(cutoff,mu))
    
    return N_total * p_deltaM


results = ascii.read(basedir/'data'/'interim'/ 'results.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
abundance_gradients = ascii.read(basedir/'data'/'external'/'radial_abundance_gradients.txt',
                                names=['name','R0','g_r25'])
observed_mass = ascii.read(basedir/'data'/'interim'/ 'observed_mass.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
results = join(results,abundance_gradients)
results = join(results,observed_mass)
results.add_index('name')

# from survey paper based on sample table distances
results['spatial_res'] = [43.7,47.7,76.8,92.1,94.9,83.5,
                               90.3,91.3,85.8,94.1,59.2,48.3,54.9,
                               63.5,82.4,73.7,76.5, 25.2,90.8]
# from survey paper
results['PSF'] = [0.72,0.73,0.74,0.63,0.82,0.49,0.65,0.8,0.64,0.72,
                  0.85,0.74,0.77,0.58,0.58,0.64,0.44,0.73,0.79]
results['mass'] = sample_table['mass']
results['SFR']  = sample_table['SFR']
results['Inclination'] = sample_table['Inclination']
results['AO'] = ~sample_table['AO'].mask

# those two galaxies have a lower completeness limit
row = results.loc['NGC3627']
row ['N_PN'] = NPN(row['(m-M)'],27.5,row['N_PN'],28-row['(m-M)']+4.47)
row = results.loc['NGC2835']
row ['N_PN'] = NPN(row['(m-M)'],27.5,row['N_PN'],28-row['(m-M)']+4.47)

# luminosity specific planetary nebula number (detected and not N25)
results['alpha2'] = np.log10(results['N_PN']/results['Lbol'])
results['alpha3'] = np.log10(results['N_PN']/10**results['obs_mass'])
results['resolution'] = results['PSF']*(results['d/Mpc']*u.Mpc*(u.arcsec.to(u.rad))).to(u.pc).value

In [ ]:
# number of PN within 1 mag of the bright end cutoff and extrapolate to completeness
N_PN = []
for name in results['name']:
    with fits.open(basedir/'data'/'catalogues'/f'{name}_classifications.fits') as hdul:
        catalogue = Table(hdul[1].data)    
    threshold = results.loc[name]['(m-M)']-4.47+1
    criteria = (catalogue['type']=='PN') & (catalogue['mOIII']<threshold) & ~catalogue['exclude'] & ~catalogue['overluminous']
    N = NPN(row['(m-M)'],threshold,np.sum(criteria),28-threshold)
    N_PN.append(N)
results['alpha3'] = np.log10(np.array(N_PN)/results['Lbol'])

first we look at the parameter space that our sample covers. We see that more massive galaxies are also more luminous, and that the massive galaxies in our sample are further away

In [ ]:
# double distance means increasing the ditsance modulus by
d = Distance(distmod=27.9)
d2 = Distance(2*d)

print(f'd={d.distmod:.2f}, 2d={d2.distmod:.2f}, dd={d2.distmod-d.distmod:.2f}')

In [ ]:
fig,ax=plt.subplots()
sc = ax.scatter(results['mass'],results['Lbol'])
ax.set(xlabel=r'$\log M/\mathrm{M}_\odot$',ylabel=r'$L_\mathrm{bol}/\mathrm{L}_\odot$')
plt.show()

In [ ]:
halign = {'NGC0628':'right','NGC1300':'right','NGC1512':'right'}
valign = {'NGC1512':'top','NGC1300':'bottom'}

fig,ax=plt.subplots()
ax.scatter(results['mass'],results['(m-M)'])
for name in sample_table['name']:
    ax.text(sample_table.loc[name]['mass'],results.loc[name]['(m-M)'],name,
            horizontalalignment=halign.get(name,'left'),
            verticalalignment=valign.get(name,'center'))
ax.set(xlabel=r'$\log M/\mathrm{M}_\odot$',ylabel='distance',xlim=[9.2,11.5])      
plt.show()

the number of PN decreases with distances because we sample a smaller part of the PNLF

In [ ]:
from pnlf.analyse import F,cdf

mu0 = 28.
mhigh = 28   # also the completeness limit
x = np.linspace(mu0-4.47,mhigh)

fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

for dmu in [0,0.5,1,1.5]:
    xp = x+dmu
    yp = cdf(x,mu0,mhigh)
    ax.plot(xp,yp,label=f'(m-M)={mu0+dmu}')
    # how far above the cutoff until we detect 20%
    p = yp[np.argmax(xp>=28)]
    print(f'mu={mu0+dmu}, cutoff={mu0+dmu-4.47}, p={p:.2f}')

ax.axvline(28,color='black')    
ax.set(xlabel='mOIII',ylabel='cumulative PNLF')
plt.legend()
plt.savefig('cumPNLF.png',dpi=600)
plt.show()

In [ ]:
mu0 = 28.
mhigh = 28
x = np.linspace(mu0-4.47,mhigh)

p = []
distmod = np.arange(28.,33,0.25)
distance = Distance(distmod=distmod).to(u.Mpc).value

for mu in distmod:
    dmu = mu-mu0
    xp = x+dmu
    yp = cdf(x,mu0,mhigh)
    p.append(yp[np.argmax(xp>=28)])
p = np.array(p)

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2))
ax1.plot(distmod,p)
ax1.axvline(mhigh+4.47,color='black')
ax1.set(xlabel='(m-M) / mag',ylabel=r'$N_\mathrm{PN}$')

ax2.plot(distance,p)
ax2.axvline(Distance(distmod=mhigh+4.47).to(u.Mpc).value,color='black')
ax2.set(xlabel='distance / Mpc',ylabel=r'$N_\mathrm{PN}$')

plt.show()

our goal is to predict how many PNe we find a a given galaxy. We start by looking at the number of PN we find as a function of the distance to the galaxy

In [ ]:
from scipy.stats import spearmanr

ref_gal = 'NGC0628'
row = results.loc[ref_gal]

# number of PN at reference point (1-res/250) for decrease due to resolution
Nref = row['N_PN'] / np.interp(row['d/Mpc'],distance,p)
alpha = np.log10(p*Nref/row['Lbol'])


halign = {
 'IC5332': 'left','NGC0628': 'left','NGC1087': 'right','NGC1300': 'right',
 'NGC1365': 'left','NGC1385': 'right','NGC1433': 'left','NGC1512': 'right',
 'NGC1566': 'left','NGC1672': 'right','NGC2835': 'left','NGC3351': 'left',
 'NGC3627': 'left','NGC4254': 'right','NGC4303': 'right','NGC4321': 'right',
 'NGC4535': 'left','NGC5068': 'left','NGC7496': 'right'}

valign = {
 'IC5332': 'bottom','NGC0628': 'bottom','NGC1087': 'bottom','NGC1300': 'center',
 'NGC1365': 'top','NGC1385': 'bottom','NGC1433': 'center','NGC1512': 'top',
 'NGC1566': 'bottom','NGC1672': 'bottom','NGC2835': 'top','NGC3351': 'center',
 'NGC3627': 'bottom','NGC4254': 'bottom','NGC4303': 'top','NGC4321': 'center',
 'NGC4535': 'bottom','NGC5068': 'bottom','NGC7496': 'bottom'}


fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

ax.plot(distance,alpha,color='gray',lw=0.5,zorder=1)

'''
cmap = plt.cm.viridis_r
cmap.set_bad('black')
#tmp = results[np.isin(results['name'],masked)]
tmp = results[results['AO']]
sc = ax.scatter(tmp['d/Mpc'],tmp['alpha2'],s=2,marker='d',
                c=tmp['age_mw'],cmap=cmap,zorder=2)
#tmp = results[~np.isin(results['name'],masked)]
tmp = results[~results['AO']]
sc = ax.scatter(tmp['d/Mpc'],tmp['alpha2'],s=2,marker='o',
                c=tmp['age_mw'],cmap=cmap,zorder=2)
'''

yerr = 1/(np.sqrt(results['N_PN'])*np.log(10))

#ax.errorbar(results['d/Mpc'],results['alpha2'],yerr=yerr,
#                 ms=2,fmt='o',color=tab10[0],zorder=2)
sc = ax.scatter(results['d/Mpc'],results['alpha2'], c=sample_table['12+logOH'], s=4, zorder=3)

for name in results['name']:
    ax.text(results.loc[name]['d/Mpc'],results.loc[name]['alpha2'],name,
            horizontalalignment=halign.get(name,'left'),
            verticalalignment=valign.get(name,'center'),
            fontsize=4)
fig.colorbar(sc,label=r'$12+\log_{10} \mathrm{O/H}$')
#fig.colorbar(sc,label=r'$N_\mathrm{PN} / L_\mathrm{bol}$')
ax.set(xlim=[4,27],
       xlabel='distance / Mpc',
       ylabel=r'$\log_{10} (N_\mathrm{PN} / L_\mathrm{bol})$')
plt.savefig(basedir/'reports'/'specific_PN_number_vs_distance.pdf',dpi=600)
plt.show()

print(spearmanr(results['resolution'],results['alpha2']))

the number of detection does not really depend on the distance but rather on the resolution

In [ ]:
from scipy.stats import spearmanr

ref_gal = 'NGC0628'
row = results.loc[ref_gal]

res = np.mean(results['PSF'])*(Distance(distmod=distmod)*(u.arcsec.to(u.rad))).to(u.pc).value
# number of PN at reference point (1-res/250) for decrease due to resolution
Nref = row['N_PN'] / np.interp(row['resolution'],res,p)
alpha = np.log10(p*Nref/row['Lbol'])

masked = [k for k,v in parameters.items() if 'mask' in v]

halign = {
 'IC5332': 'left','NGC0628': 'left','NGC1087': 'center','NGC1300': 'right',
 'NGC1365': 'right','NGC1385': 'left','NGC1433': 'left','NGC1512': 'left',
 'NGC1566': 'right','NGC1672': 'left','NGC2835': 'left','NGC3351': 'left',
 'NGC3627': 'center','NGC4254': 'right','NGC4303': 'right','NGC4321': 'center',
 'NGC4535': 'right','NGC5068': 'left','NGC7496': 'left'}

valign = {
 'IC5332': 'bottom','NGC0628': 'bottom','NGC1087': 'bottom','NGC1300': 'center',
 'NGC1365': 'bottom','NGC1385': 'top','NGC1433': 'top','NGC1512': 'center',
 'NGC1566': 'center','NGC1672': 'top','NGC2835': 'bottom','NGC3351': 'top',
 'NGC3627': 'bottom','NGC4254': 'bottom','NGC4303': 'top','NGC4321': 'bottom',
 'NGC4535': 'top','NGC5068': 'top','NGC7496': 'bottom'}

print('correlation={:.2f}, pvalue={:.4f}'.format(*spearmanr(results['resolution'],results['alpha2'])))

fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

ax.plot(res,alpha,color='gray',lw=0.5,zorder=1)

#sc = ax.scatter(results['resolution'],results['alpha2'],s=2,
#                c=results['mass'],cmap=plt.cm.viridis_r,zorder=2)

#tmp = results[np.isin(results['name'],masked)]
tmp = results[results['AO']]
sc = ax.scatter(tmp['resolution'],tmp['alpha2'],s=2,marker='d',
                c=tmp['R0'],cmap=plt.cm.viridis_r,zorder=2)
#tmp = results[~np.isin(results['name'],masked)]
tmp = results[~results['AO']]
sc = ax.scatter(tmp['resolution'],tmp['alpha2'],s=2,marker='o',
                c=tmp['R0'],cmap=plt.cm.viridis_r,zorder=2)


for name in results['name']:
    ax.text(results.loc[name]['resolution'],results.loc[name]['alpha2'],name,
            horizontalalignment=halign.get(name,'left'),
            verticalalignment=valign.get(name,'center'),
            fontsize=4)
#fig.colorbar(sc,label=r'$\log_{10} M/\mathrm{M}_\odot$')
fig.colorbar(sc,label=r'$12+\log_{10} \mathrm{O/H}$')

ax.set(xlim=[11,91],
       xlabel='resolution in pc',
       ylabel=r'$\log_{10} N_\mathrm{PN} / L_\mathrm{bol}$')
plt.savefig(basedir/'reports'/'specific_PN_number_vs_resolution.pdf',dpi=600)
plt.show()



can I simply use the number of detected PN? This means that galaxies further away will natuarlly have fewer detections as their cutoff is closer to the completeness limit and hence we expect fewer PNe anyways.

In [ ]:
halign = {
 'IC5332': 'right','NGC0628': 'left','NGC1087': 'left','NGC1300': 'right',
 'NGC1365': 'left','NGC1385': 'left','NGC1433': 'left','NGC1512': 'left',
 'NGC1566': 'right','NGC1672': 'left','NGC2835': 'left','NGC3351': 'left',
 'NGC3627': 'right','NGC4254': 'left','NGC4303': 'right','NGC4321': 'right',
 'NGC4535': 'left','NGC5068': 'left','NGC7496': 'left'}

valign = {
 'IC5332': 'top','NGC0628': 'bottom','NGC1087': 'bottom','NGC1300': 'center',
 'NGC1365': 'top','NGC1385': 'top','NGC1433': 'center','NGC1512': 'top',
 'NGC1566': 'center','NGC1672': 'top','NGC2835': 'bottom','NGC3351': 'top',
 'NGC3627': 'top','NGC4254': 'top','NGC4303': 'top','NGC4321': 'center',
 'NGC4535': 'top','NGC5068': 'top','NGC7496': 'bottom'}

fig,ax=plt.subplots(figsize=(2*single_column,2*single_column/1.618))
sc = ax.scatter(results['resolution'],results['alpha'],s=8,
                c=results['mass'],cmap=plt.cm.viridis_r)
for name in results['name']:
    ax.text(results.loc[name]['resolution'],results.loc[name]['alpha'],name,
            horizontalalignment=halign.get(name,'left'),
            verticalalignment=valign.get(name,'center'),
            fontsize=8)

fig.colorbar(sc,label=r'$\log_{10} M/\mathrm{M}_\odot$')

#fig.colorbar(sc,label=r'$N_\mathrm{PN} / L_\mathrm{bol}$')
ax.set(xlim=[10,90],
       xlabel='resolution / pc',
       ylabel=r'$\log N_\mathrm{PN} / L_\mathrm{bol}$')
#plt.savefig(basedir/'reports'/'specific_PN_number_vs_resolution.pdf',dpi=600)
plt.show()

how many PN do we expect in a given galaxy

In [ ]:
# calculate alpha from a reference galaxy
ref_gal = 'IC5332'
row = results.loc[ref_gal]
Nref = row['N_PN'] / np.interp(row['(m-M)'],distmod,p)
alpha = np.log10(p*Nref/row['Lbol'])

# use alpha to predict the number of PN in another galaxy
gal_name = 'NGC1365'
row = results.loc[gal_name]
a = np.interp(row['(m-M)'],distmod,alpha)
print(f"predicted: {10**a * row['Lbol']:.1f}, observed: {row['N_PN']}")

or based on resolution

In [ ]:
# calculate alpha from a reference galaxy
ref_gal = 'NGC1512'
row = results.loc[ref_gal]
Nref = row['N_PN'] / np.interp(row['resolution'],res,p)
alpha = np.log10(p*Nref/row['Lbol'])

# use alpha to predict the number of PN in another galaxy
gal_name = 'NGC0628'
row = results.loc[gal_name]
a = np.interp(row['resolution'],res,alpha)
print(f"predicted: {10**a * row['Lbol']:.1f}, observed: {row['N_PN']}")

up to what distance can we observe a given galaxy

In [ ]:
alphas = []
for ref_gal in results['name']:
    row = results.loc[ref_gal]
    Nref = row['N_PN'] / np.interp(row['d/Mpc'],distance,p)
    alpha = np.log10(p*Nref/row['Lbol'])
    alphas.append(alpha[0])
    alpha_min = np.log10(20/row['Lbol'])
    max_dist = np.interp(-alpha_min,-alpha,distance)
    
    print(f'{ref_gal}: alpha={alpha[0]:.2f}, max_dist={max_dist:.2f}')

In [ ]:
10**min(alphas)/10**max(alphas)

NGC5068 has the largest $\alpha=-7.56$ and NGC1433 is the most luminous. Up to what distance could we observe a combination of both. NGC1365 has the smallest $\alpha=-8.32$.

In [ ]:
row = results.loc['NGC5068']
Nref = row['N_PN'] / np.interp(row['d/Mpc'],distance,p)
alpha = np.log10(p*Nref/row['Lbol'])

row = results.loc['NGC1433']
alpha_min = np.log10(20/row['Lbol'])
max_dist = np.interp(-alpha_min,-alpha,distance)

print(f'alpha={alpha[0]:.2f}, max_dist={max_dist:.2f}')

In [ ]:
row = results.loc['NGC1365']
Nref = row['N_PN'] / np.interp(row['d/Mpc'],distance,p)
alpha = np.log10(p*Nref/row['Lbol'])

row = results.loc['NGC1433']
alpha_min = np.log10(20/row['Lbol'])
max_dist = np.interp(-alpha_min,-alpha,distance)

print(f'alpha={alpha[0]:.2f}, max_dist={max_dist:.2f}')

In [ ]:
alst = []
for ref_gal in results['name']:
    row = results.loc[ref_gal]
    Nref = row['N_PN'] / np.interp(row['(m-M)'],distmod,p)
    alpha = np.log10(p*Nref/row['Lbol'])
    alst.append(alpha[0])

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column))

ax.scatter(sample_table['12+logOH'],10**results['alpha'])
ax.set(ylim=[1e-9,1.5e-8])

plt.show()

In [ ]:


plt.scatter(results['mass'],results['obs_mass'])
plt.show()

## Redo 

this section can be used to redo certain steps

### Extinction correction

In [ ]:
import pyneb
import re

with fits.open(basedir/'data'/'catalogues'/f'nebulae_full.fits') as hdul:
    catalogue = Table(hdul[1].data)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])

rc = pyneb.RedCorr(R_V = 3.1, law = 'CCM89')
rc.setCorr(obs_over_theo= catalogue['HA6562']/catalogue['HB4861'] / 2.86, wave1=6562.81, wave2=4861.33)
rc.E_BV[(rc.E_BV<0) | (catalogue['HB4861']<3*catalogue['HB4861_err']) |  (catalogue['HA6562']<3*catalogue['HA6562_err'])] = 0
catalogue['EBV_balmer'] = rc.E_BV
catalogue['A5007'] =  -2.5*np.log10(rc.getCorr(5007))
    
catalogue['type_old'] = catalogue['type']
catalogue['mOIII_old'] = catalogue['mOIII']

for line in ['HB4861', 'OIII5006', 'HA6562', 'NII6583', 'SII6716', 'SII6730']:
    wavelength = int(re.findall(r'\d{4}', line)[0])
    catalogue[f'{line}_old'] = catalogue[f'{line}']
    catalogue[f'{line}_old_err'] = catalogue[f'{line}_err']
    catalogue[line] = catalogue[line] * rc.getCorr(wavelength)
    catalogue[f'{line}_err'] = catalogue[f'{line}_err'] * rc.getCorr(wavelength)
catalogue['mOIII'] = -2.5*np.log10(catalogue['OIII5006']*1e-20) - 13.74

print(f"{np.sum((catalogue['type']=='PN') & (catalogue['A5007']<0))} objects have Av<0")
print(f"mean A5007: {np.nanmean(catalogue[(catalogue['type']=='PN') & (catalogue['A5007']<0)]['A5007']):.2f}")

In [ ]:
gal_name='NGC1433'
print(f"bright end: {results.loc[gal_name]['(m-M)']-4.47:.2f}")
sub = catalogue[(catalogue['gal_name']==gal_name) & (catalogue['type']=='PN') & (catalogue['A5007']<0)]
sub.sort('mOIII_old')
sub[['id','mOIII_old','mOIII','A5007']]

In [ ]:
for gal_name in np.unique(catalogue['gal_name']):
    a = catalogue['A5007'][(catalogue['gal_name']==gal_name) & (catalogue['type']=='PN') & (catalogue['A5007']<0)]
    print(f'{gal_name}: {np.nanmean(a):.2f} ({len(a)})')

In [ ]:
np.median(A5007[(catalogue['type']=='PN') & (catalogue['mOIII']<28) & (A5007<0)])

In [ ]:
fig,ax=plt.subplots()
ax.hist(A5007[(catalogue['type']=='PN') & (catalogue['mOIII']<28)],bins=np.arange(-2,-0.01,0.2))
ax.set(xlabel=r'$A_{5007}$',ylabel='N')
plt.show()

In [ ]:
from pnlf.analyse import emission_line_diagnostics
catalogue['HA6562_SIGMA'] = 0
lst = []
for name in results['name']:
    lst.append(emission_line_diagnostics(catalogue[catalogue['gal_name']==name],results.loc[name]['(m-M)'],28))
tbl = vstack(lst)          

In [ ]:
tbl['HA6562_detection'] = tbl['HA6562_detection']=='True'
tbl['SII_detection'] = tbl['SII_detection']=='True'
tbl['overluminous'] = tbl['overluminous'].astype(bool)

In [ ]:
sample = tbl[((tbl['type']!='PN') & (tbl['type_old']=='PN')) | ((tbl['type']=='PN') & (tbl['type_old']!='PN'))]

In [ ]:
from pnlf.plot import plot_emission_line_ratio

sample = sample[sample['mOIII']<28]

plot_emission_line_ratio(sample,mu=30,completeness=28)

In [ ]:
problem = tbl[(tbl['type']=='PN') & (tbl['type_old']!='PN')] #[['mOIII','mOIII_old']]
problem[problem['mOIII']<28]

In [ ]:
fig,ax=plt.subplots(figsize=(8,8))

for t in ['HII','SNR','PN']:
    tmp=catalogue[catalogue['type']==t]
    ax.scatter(tmp['mOIII_old'],tmp['mOIII'],label=t)
ax.plot([22,29],[22,29],color='black')
ax.legend()
ax.set(xlim=[22,29],ylim=[22,29])
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(8,8))


ax.scatter(catalogue['EBV_balmer'],catalogue['Ebv'])
ax.plot([0,2],[0,2],color='black')
ax.legend()
ax.set(xlim=[0,2],ylim=[0,2],xlabel='E(B-V) Balmer',ylabel='E(B-V) Stars')
plt.show()

In [ ]:
rc.setCorr(obs_over_theo= catalogue['HA6562']/catalogue['HB4861'] / 2.86, wave1=6562.81, wave2=4861.33)
EBV = rc.E_BV

rc.setCorr(obs_over_theo= (catalogue['HA6562']-catalogue['HA6562_err']) / (catalogue['HB4861']+catalogue['HB4861_err']) / 2.86, wave1=6562.81, wave2=4861.33)
EBV_minus = rc.E_BV

rc.setCorr(obs_over_theo= (catalogue['HA6562']+catalogue['HA6562_err']) / (catalogue['HB4861']-catalogue['HB4861_err']) / 2.86, wave1=6562.81, wave2=4861.33)
EBV_plus = rc.E_BV

criteria  = (catalogue['type']=='PN')
criteria &= (catalogue['HB4861']>3*catalogue['HB4861_err'])
criteria &= (catalogue['HA6562']>3*catalogue['HA6562_err']) 

np.sum(criteria & (EBV_minus>0))

In [ ]:
criteria  = (catalogue['type']=='PN')
criteria &= (catalogue['HB4861']>3*catalogue['HB4861_err'])
criteria &= (catalogue['HA6562']>3*catalogue['HA6562_err']) 
criteria &= (catalogue['EBV']>0) 




fig,ax=plt.subplots(figsize=(4,4))
ax.hist(EBV[criteria],bins=np.arange(0,1,0.1),alpha=0.5)
ax.hist(EBV_minus[criteria],bins=np.arange(0,1,0.1),alpha=0.5)


ax.set(xlim=[0,1],xlabel='E(B-V) balmer')
plt.show()


In [ ]:
fig,ax=plt.subplots(figsize=(4,4))
ax.hist(catalogue[criteria]['EBV'],bins=np.arange(0,1,0.1))
ax.set(xlim=[0,1],xlabel='E(B-V) balmer')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(4,4))
ax.scatter(catalogue[criteria]['mOIII'],catalogue[criteria]['mOIII_corr'])
ax.plot([25,29],[25,29],color='black')
ax.set(xlim=[25,29],ylim=[25,29])
plt.show()

In [ ]:
plt.hist(catalogue[criteria]['mOIII']-catalogue[criteria]['mOIII_corr'],bins=np.arange(0,5,0.5))
plt.show()


In [ ]:
for name in np.unique(catalogue['gal_name']):
    completeness = 28
    
    criteria  = (catalogue['gal_name']==name) 
    criteria &= (catalogue['type']=='PN') 
    criteria &= (catalogue['note']=='') 
    criteria &= (catalogue['HB4861']>3*catalogue['HB4861_err'])
    sub = catalogue[criteria]
    
    print(f'{name}: min mOIII: {np.nanmin(sub["mOIII"])}, {np.nanmin(sub["mOIII_corr"])}')


#### With CO

In [ ]:
rc = pyneb.RedCorr(R_V = 3.1, law = 'CCM89')
rc.setCorr(obs_over_theo= tmp['HA6562']/tmp['HB4861'] / 2.86, wave1=6562.81, wave2=4861.33)
rc.E_BV[(rc.E_BV<0) | (tmp['HB4861']<3*tmp['HB4861_err']) |  (tmp['HA6562']<3*tmp['HA6562_err'])] = 0
tmp['EBV_balmer'] = rc.E_BV
tmp['A5007'] = -2.5*np.log10(rc.getCorr(5007))

In [ ]:
from photutils import SkyCircularAperture, aperture_photometry

fig,ax=plt.subplots()


for name in np.unique(catalogue['gal_name']):
    tmp = catalogue[(catalogue['gal_name']==name) & (catalogue['type']=='PN')]
    if np.sum((tmp['A5007']<0) & (tmp['type']=='PN'))<2:
        print(f'no PN for {name}')
        continue
    
    with fits.open(data_ext/'ALMAv4p0'/f'{name.lower()}_12m+7m+tp_co21_broad_mom0.fits') as hdul:
        CO = NDData(data=hdul[0].data,
                    meta=hdul[0].header,
                    wcs=WCS(hdul[0].header))

    aperture = SkyCircularAperture(tmp['SkyCoord'],r=0.6*u.arcsec)
    CO_flux = aperture_photometry(CO,aperture)
    tmp['CO'] = CO_flux['aperture_sum']

    ax.scatter(tmp['A5007'],tmp['CO'],label=name)

ax.legend()
ax.set(xlim=[-4,-0.1])
ax.set(xlabel=r'$A_{5007}$',ylabel='CO')
plt.show()

### PNLF fit

this cell fits all galaxies and saves the resulting figures and distances

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, PNLF, pnlf, cdf
from pnlf.plot.pnlf import plot_pnlf
from scipy.stats import kstest

name = 'NGC5068'

tbl = Table(fits.getdata(basedir/'data'/'catalogues'/'PN_catalogue.fits'))
tbl['overluminous'] = tbl['note']=='OL'

completeness = parameters[name]['completeness_limit']
binsize = parameters[name]['binsize']

criteria = (tbl['type']=='PN') & ~tbl['overluminous'] & (tbl['gal_name']==name)
data = tbl[criteria]
print(f'{len(data)} PNe in sample')

In [ ]:
import re
from dust_extinction.parameter_averages import O94, CCM89
from cluster.extinction import balmer_decrement

extinction_model = CCM89(Rv=3.1)

data['EBV'] = balmer_decrement(data['HA6562'],data['HB4861'],
                               extinction_model=extinction_model)

for line in ['HB4861','OIII5006','HA6562','NII6583','SII6716','SII6730']:
    wavelength = int(re.findall(r'\d{4}', line)[0])
    print(f'working on {line}: {wavelength} A')
    extinction_int = extinction_model.extinguish(wavelength*u.angstrom,Ebv=data['EBV'])
    
    data[f'{line}_corr'] = data[line] / extinction_int

data['mOIII_corr'] = -2.5*np.log10(data['OIII5006_corr']*1e-20) - 13.74
#data['dmOIII'] = np.abs( 2.5/np.log(10) * flux['OIII5006_err'] / flux['OIII5006'] )

if True:
    fig,ax=plt.subplots(figsize=(4,4))
    ax.scatter(data['mOIII'],data['mOIII_corr'])
    ax.plot([25,29],[25,29],color='black')
    ax.set(xlim=[25,29],ylim=[25,29])
    plt.show()

In [ ]:
Mmax = -4.53

fitter = MaximumLikelihood1D(pnlf,data['mOIII'],err=data['dmOIII'],mhigh=completeness,Mmax=Mmax)
mu,dp,dm = fitter([28])
print('{:.2f} + {:.2f} - {:.2f}'.format(mu,dp,dm))

ks,pv = kstest(data['mOIII'],cdf,args=(mu,completeness))
print(f'statistic={ks:.3f}, pvalue={pv:.3f}')

filename = None #basedir / 'reports' / galaxy.name / f'{galaxy.name}_PNLF_with_SNR'
ax = plot_pnlf(data['mOIII'],mu,completeness,binsize=binsize,mhigh=30,color=tab10[0])
#plt.show()

In [ ]:

data['EBV'] = balmer_decrement(data['HA6562'],data['HB4861'])

plt.hist(data['EBV'],bins=np.arange(0,1,0.1))
plt.show()

for all galaxies at once

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, PNLF, pnlf, cdf
from pnlf.plot.pnlf import plot_pnlf
from scipy.stats import kstest
from pnlf.utils import get_bolometric_luminosity
from pnlf.analyse import N25
import datetime
date = datetime.date.today().strftime('%Y.%m.%d')

results = ascii.read(basedir/'data'/'interim'/ 'results.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
results.add_index('name')  
    
for name in results['name']:
    
    print(f'working on {name}')
    
    tbl = ascii.read(basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt')
    tbl['SNRorPN'] = tbl['SNRorPN'] == 'True'
    tbl['exclude'] = tbl['exclude'].astype(bool)
    tbl['overluminous'] = tbl['overluminous'].astype(bool)

    completeness = parameters[name]['completeness_limit']
    binsize = parameters[name]['binsize']

    criteria1 = (tbl['type']=='PN') & ~tbl['exclude'] & ~tbl['overluminous'] & (tbl['mOIII']<completeness)
    data1 = tbl[criteria1]['mOIII']
    err1  = tbl[criteria1]['dmOIII']

    criteria2 = ((tbl['type']=='PN')|((tbl['type']=='SNR')&(tbl['SNRorPN']))) & ~tbl['exclude'] & ~tbl['overluminous'] & (tbl['mOIII']<completeness)
    data2 = tbl[criteria2]['mOIII']
    err2  = tbl[criteria2]['dmOIII']
    
    fitter = MaximumLikelihood1D(pnlf,data1,err=err1,mhigh=completeness)
    mu1,dp1,dm1 = fitter([28])

    ks1,pv1 = kstest(data1,cdf,args=(mu1,completeness))
    print(f'without SNR: statistic={ks1:.3f}, pvalue={pv1:.3f}')

    fitter = MaximumLikelihood1D(pnlf,data2,err=err2,mhigh=completeness)
    mu2,dp2,dm2 = fitter([28])

    ks2,pv2 = kstest(data1,cdf,args=(mu2,completeness))
    print(f'with SNR: statistic={ks2:.3f}, pvalue={pv2:.3f}')

    print(f'without SNR: {mu1:.2f}+{dp1:.2f}-{dm1:.2f}\nwith SNR:    {mu2:.2f}+{dp2:.2f}-{dm2:.2f} ({mu1-mu2:.2f})')

    #filename = basedir / 'reports' / galaxy.name / f'{galaxy.name}_PNLF_with_SNR'
    #axes = plot_pnlf(tbl[criteria1]['mOIII'],mu1,completeness,binsize=binsize,mhigh=30,color=tab10[0])
    #axes = plot_pnlf(tbl[criteria2]['mOIII'],mu2,completeness,binsize=binsize,mhigh=30,filename=filename,color='grey',alpha=0.7,axes=axes)
    #plt.show()

    results.loc[name]['(m-M)'] = mu1
    results.loc[name]['err+(m-M)'] = dp1
    results.loc[name]['err-(m-M)'] = dm1
    results.loc[name]['mu_SNR'] = mu2
    results.loc[name]['mu_SNR+'] = dp2
    results.loc[name]['mu_SNR-'] = dm2
    results.loc[name]['d/Mpc'] = Distance(distmod=mu1).to(u.Mpc).value
    results.loc[name]['err+d/Mpc'] = 2*np.log(10)*10**(mu1/5) * dp1 / 1e6
    results.loc[name]['err-d/Mpc'] = 2*np.log(10)*10**(mu1/5) * dm1 / 1e6
    
    # save results to output table
    for col in results.colnames[2:]:
        if col.startswith('N_'):
            results[col].info.format = '%.0f'
        else:
            results[col].info.format = '%.3f'
    results['Lbol'].info.format = '%.2e'    
    
with open(basedir/'data'/'interim'/ 'results_new.txt','w',newline='\n') as f:
    ascii.write(results,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')

### literature distance plot

in case something has changed, this cell creates the plot with the comparison

In [ ]:
from pnlf.plot import compile_distances, plot_distances

name = 'NGC4303'
#for name in results['name']:

plt.close('all')    
mu,mu_m,mu_p = results.loc[name][['(m-M)','err-(m-M)','err+(m-M)']]
print(name)
filename = basedir / 'reports' / name / f'{name}_distances'
distances = compile_distances(name)
plot_distances(name,mu,mu_p,mu_m,distances,filename=filename)


### Emission line diagnostics

In [ ]:
# remove all objects defined here from the sample
# define masks as slices
exclude = {
 'IC5332'  : [1338],
 'NGC0628' : [886],
 'NGC1300' : [2702,2578,2466,2523],
 'NGC1365' : [730],
 'NGC1385' : [88,44,585,606,677,106,29],
 'NGC1512' : [272],
 'NGC1566' : [25,168],
 'NGC1672' : [275,199,],
 'NGC2835' : [179,416,287,807],
 'NGC4254' : [496,1948,1947,624,1941,473,206,285,573,482],
 'NGC4303' : [411,380,327],
 'NGC5068' : [554,557],
 'NGC7496' : [353],
}

overluminous = {
 'NGC1087' : [878],
 'NGC1512' : [277],
 'NGC1566' : [199,103],
 'NGC1672' : [204,42],
 'NGC2835' : [],
 'NGC4254' : [447,345,1104],
 'NGC4303' : [370],
 'NGC4321' : [2571],
 'NGC7496' : [408,350],
}

slow  = .2 #galaxy.sharplo  
shigh = 1. #galaxy.sharphi 
r     = .8 #galaxy.roundness 

In [ ]:
name = 'NGC1672'

distance_modulus = parameters[name]['mu']
completeness_limit = parameters[name]['completeness_limit']

with fits.open(basedir/'data'/'interim'/'fluxes'/f'{name}_fluxes.fits') as hdul:
    flux = Table(hdul[1].data)
flux['SkyCoord'] = SkyCoord(flux['RaDec'])

# create additional columns that are needed for the classification
flux['exclude'] = False
flux['overluminous'] = False

# flag the objects from the dictionary
indices = np.where(np.in1d(flux['id'], exclude.get(name,[])))[0]
flux['exclude'][indices]=True
indices = np.where(np.in1d(flux['id'], overluminous.get(name,[])))[0]
flux['overluminous'][indices]=True

# use mean for background
for line in [x.split('_')[0] for x in flux.columns if x.endswith('_flux')]:
    flux[f'{line}_flux'] = flux[f'{line}_flux_raw'] - flux[f'{line}_bkg_mean']
    
print(f'{name}: mu={distance_modulus}, cl={completeness_limit}')

In [ ]:
from pnlf.analyse import emission_line_diagnostics
from pnlf.analyse import MaximumLikelihood1D, pnlf, cdf
from scipy.stats import kstest

Mmax = -4.47

print(f'emission line diagnostics for {name}')

distance_modulus_old = distance_modulus
N_PN_old = 0
n_iter = 0
while True:
    n_iter += 1
    print(f'\nitteration {n_iter}')
    
    tbl = emission_line_diagnostics(flux,distance_modulus=distance_modulus,
                                    completeness_limit=completeness_limit,
                                    detection_limit=5) 

    # table contains all detected objects. here we mask all undesired objects.
    c_shape = ((tbl['sharp']>slow) & (tbl['sharp']<shigh) & (np.abs(tbl['round'])<r)) 
    c_PN    = (tbl['type']=='PN')
    c_SNR   = (tbl['SNRorPN'] & (tbl['type']=='SNR'))
    c_detec = tbl['OIII5006_detection'] 
    c_limit = (tbl['mOIII']<completeness_limit) 

    criteria = c_shape & (c_PN) & ~tbl['exclude'] & ~tbl['overluminous'] & ~np.isnan(tbl['mOIII'])
    data = tbl[np.where(criteria & c_limit)]['mOIII']
    err = tbl[np.where(criteria & c_limit)]['dmOIII']

    fitter = MaximumLikelihood1D(pnlf,data,err=err,mhigh=completeness_limit,Mmax=Mmax)
    distance_modulus,mu_p,mu_m = fitter([29])
    print('{}: {:.2f} + {:.2f} - {:.2f}'.format(name,distance_modulus,mu_p,mu_m))
    ks,pv = kstest(data,cdf,args=(distance_modulus,completeness_limit))
    print(f'{name}: statistic={ks:.3f}, pvalue={pv:.3f}')
    
    N_PN = len(data)
    if (np.abs((distance_modulus-distance_modulus_old)/distance_modulus_old) > 0.01 or \
       N_PN_old != N_PN) and n_iter < 5:
        N_PN_old = N_PN
        distance_modulus_old = distance_modulus
    else:
        print('\nN_PN and (m-M) did not change!')
        break

In [ ]:
from pnlf.plot.pnlf import plot_pnlf

binsize = (completeness_limit-Mmax-distance_modulus) / 3
filename = None #basedir / 'reports' / f'{name}' / f'{name}_PNLF'
axes = plot_pnlf(tbl[criteria]['mOIII'],distance_modulus,completeness_limit,
                 binsize=binsize,mhigh=28.5,Mmax=Mmax,filename=filename,color=tab10[0])

In [ ]:
fig,ax=plt.subplots()

lim = [1,1e4]
ax.scatter(flux['OIII5006_flux'],flux['OIII5006_flux_mean'])
ax.plot(lim,lim,color='black')
ax.set(xlim=lim,ylim=lim,xscale='log',yscale='log')
plt.show()

## PN Number vs Mass etc.

In [ ]:
sample=ascii.read(basedir/'reports'/'sample.txt')
sample['SkyCoord'] = SkyCoord(sample['R.A.'],sample['Dec.'])
sample.add_index('Name')

In [ ]:
galaxies = {}

for n in result['name']:
    print('are you sure that you want to run this?')
    break
    filename = data_raw / 'MUSEDAP' / f'{n}_MAPS.fits'

    with fits.open(filename) as hdul:
        d=hdul['STELLAR_MASS_DENSITY'].data
        galaxies[n]= np.nansum(d)

In [ ]:
result['mass']=sample['mass']
result['Survey Area'] = sample['Survey Area']

Calculate the survey area from number of pixels and distance

In [ ]:
for row in result:
    N_pix = row['N_pixel']
    d = Distance(distmod=parameters[row['name']]['mu'])
    A_pix = ((d*(0.2/206265))**2).to(u.kpc**2)
    
    print(f'{row["name"]}: {d:.2f}, {N_pix*A_pix:.2f}')

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,nrows=1,figsize=(12,4))

for row in result:
    if row['N_PN']>1:
        ax1.scatter(row['mass'],row['N_PN'])
        ax1.text(row['mass'],row['N_PN']+2,row['name'],horizontalalignment='center')
        
        
        ax2.scatter(row['Survey Area'],row['N_PN'])
        ax2.text(row['Survey Area'],row['N_PN']+2,row['name'],horizontalalignment='center') 

        ax3.scatter(row['N_pixel'],row['N_PN'])
        ax3.text(row['N_pixel'],row['N_PN']+2,row['name'],horizontalalignment='center') 
        

ax1.set(xlabel='stellar mass density',ylabel='N PN')
ax2.set(xlabel='Survey Area')
ax3.set(xlabel='Npixel')
#ax3.set(xlabel='Npixel')
plt.show()

## Compare to literature

### Offsets to different methods

In [ ]:
# combine the different 
rows = ['gal_name,(m-M),err(m-M),D(Mpc),Method,Refcode,Notes,SN Name,Redshift,H0,LMCModulus']
for name in results['name']: 
    with open(basedir / 'data' / 'literature distances' / f'{name}.csv') as f:
        raw = f.read()
    for row in raw.split('\n')[1:]:
        if row:
            rows.append(name+','+row.strip('#'))
#with open(basedir / 'data' / 'literature distances' / f'literature_combined.csv','w+') as f:
#    f.write('\n'.join(rows))
distances = ascii.read('\n'.join(rows))
distances['(m-M)_PHANGS'] = np.nan
distances['(m-M)_PHANGS_err+'] = np.nan
distances['(m-M)_PHANGS_err-'] = np.nan

for name in results['name']: 
    distances['(m-M)_PHANGS'][distances['gal_name']==name] = results.loc[name]['(m-M)'] 
    distances['(m-M)_PHANGS_err+'][distances['gal_name']==name] = results.loc[name]['err+(m-M)'] 
    distances['(m-M)_PHANGS_err-'][distances['gal_name']==name] = results.loc[name]['err-(m-M)'] 

distances['d(m-M)'] =  distances['(m-M)']-distances['(m-M)_PHANGS']

del distances[['Notes','SN Name','Redshift','H0','LMCModulus']]
distances['year'] = [int(row['Refcode'][:4]) for row in distances]

# to combine multiple values from once source
'''
gal_name = []
dm = []
err_dm = []
methods = []
dm_PHANGS = []
dm_PHANGS_err_p = []
dm_PHANGS_err_m = []
marker = []
for g in distances.group_by(['Refcode','Method','gal_name']).groups:
    gal_name.append(g['gal_name'][0])
    dm.append(g['(m-M)'].mean())
    err_dm.append(np.sqrt(np.sum(g['err(m-M)']**2)))
    methods.append(g['Method'][0])
    dm_PHANGS.append(g['(m-M)_PHANGS'][0])
    dm_PHANGS_err_p.append(g['(m-M)_PHANGS_err+'][0])
    dm_PHANGS_err_m.append(g['(m-M)_PHANGS_err-'][0])

    if len(g)==1:
        marker.append('o')
    else:
        marker.append('D')

distances = Table([gal_name,dm,err_dm,methods,dm_PHANGS,dm_PHANGS_err_p,dm_PHANGS_err_m,marker],
                  names=['gal_name','(m-M)','err(m-M)','Method','(m-M)_PHANGS','(m-M)_PHANGS_err+','(m-M)_PHANGS_err-','marker'])
distances['d(m-M)'] = distances['(m-M)_PHANGS'] - distances['(m-M)']
''';

In [ ]:
np.std(distances[(distances['Method']=='TRGB') & ~np.isin(distances['gal_name'],['NGC1433','NGC1512'])]['d(m-M)'])

In [ ]:
fig,axes = plt.subplots(nrows=7,figsize=(two_column,1.25*two_column))

colors = {
'TRGB':'#f28e2b',
'Cepheids':'#59a14e',
'IRAS':'#ff9da7',
'NAM' : '#edc949',
'PNLF':'#e15759',
'SNII optical':'#4e79a7',
'SNIa':'#76b7b2',
'Tully-Fisher':'#b07aa2'
}

m_labels = {'Tully-Fisher':'Tully--Fisher','SNIa':'Type Ia Supernova',
            'SNII optical':'Type II Supernova optical',
            'PNLF':'Planetary Nebula Luminosity Function',
            'TRGB':'Tip of the Red Giant Branch',
            'NAM':'Numerical Action Method'}

distances.sort(['(m-M)_PHANGS','year'])

for ax,Method in zip(axes,['PNLF','TRGB','Cepheids','Tully-Fisher','NAM','SNII optical','SNIa']):
    sample = distances[distances['Method']==Method]
    sample['x'] = np.arange(1,len(sample)+1)

    #if Method=='TRGB':
    #    sample=sample[~np.isin(sample['gal_name'],['NGC1433','NGC1512'])]
    
    label = f"{m_labels.get(Method,Method)}: ${np.mean(sample['d(m-M)']):.2f}\pm{np.std(sample['d(m-M)']):.2f}$"
    #ax.errorbar(np.arange(len(sample)),sample['d(m-M)'],yerr=sample['err(m-M)'],
    #             fmt='o',color=colors[Method])
    ax.axhline(0,color='black')
    
    galaxy_ticks = []
    galaxy_labels = np.unique(sample['gal_name'])
    for i,group in enumerate(sample.group_by('gal_name').groups):
        m = group[0]['gal_name']
        for row in group:
            ax.errorbar(row['x'],row['d(m-M)'],yerr=row['err(m-M)'],color=colors[Method],ls='none',fmt='o',ms=2,zorder=2)
        galaxy_ticks.append(np.mean(group['x']))
        ax.axvline(np.max(group['x'])+0.5,color='gray',lw=0.5)
        ax.fill_between([np.min(group['x'])-0.5,np.max(group['x'])+0.5],
                        y1=2*[-row['(m-M)_PHANGS_err-']],
                        y2=2*[row['(m-M)_PHANGS_err+']],facecolor='black', alpha=0.2,zorder=1)
        ax.fill_between([np.min(group['x'])-0.5,np.max(group['x'])+0.5],
                        y1=2*[-row['(m-M)_PHANGS_err-']],
                        y2=2*[row['(m-M)_PHANGS_err+']],facecolor='black', alpha=0.2,zorder=1)
    
    if Method == 'Tully-Fisher':
        ax.set(ylim=[-4,4],xlim=[0.5,len(sample)+0.5],ylabel=r'$\Delta(m-M)$')        
    elif Method == 'NAM':
        ax.set(ylim=[-2,2],xlim=[0.5,len(sample)+0.5],ylabel=r'$\Delta(m-M)$')        

    else:
        ax.set(ylim=[-1.5,1.5],xlim=[0.5,len(sample)+0.5],ylabel=r'$\Delta(m-M)$')

    ax.set_xticks(galaxy_ticks,minor=False)
    ax.set_xticklabels(galaxy_labels,rotation=30,ha='right',fontsize=6)
  
    ax.set_title(label,fontsize=8)

plt.tight_layout()
plt.savefig(basedir/'reports'/'literature_distances.pdf',dpi=600)
plt.show()

### measured fluxes

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)

from pnlf.load_references import NGC628, \
                                 pn_NGC628_kreckel, \
                                 snr_NGC628_kreckel, \
                                 pn_NGC628_herrmann, \
                                 NGC628_kreckel, \
                                 pn_NGC5068_herrmann, \
                                 pn_NGC3351_ciardullo, \
                                 pn_NGC3627_ciardullo

In [ ]:
fig,(ax1,ax2) = plt.subplots(nrows=2,ncols=1,figsize=(single_column,single_column*1.8))

'''
   NGC0628
'''

name = 'NGC0628'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[(catalogue['type']=='PN') | ((catalogue['type']=='SNR') & (catalogue['SNRorPN']==True)) ]
#catalogue = catalogue[np.isin(catalogue['type'],['PN','SNR'])]
catalogue['R2'] = catalogue['OIII5006'] / (catalogue['HA6562']+catalogue['NII6583'])
catalogue['dR2'] = catalogue['R2']  * np.sqrt(catalogue['OIII5006_err']/catalogue['OIII5006_err']**2 + 1/(catalogue['HA6562']+catalogue['NII6583'])**2 * (catalogue['HA6562_err']**2+catalogue['NII6583_err']**2) )                                  

matchcoord = NGC628.copy()

ID, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogue['SkyCoord'])
matchcoord['mOIII_measured'] = catalogue[ID]['mOIII']
matchcoord['dmOIII_measured'] = catalogue[ID]['dmOIII']
matchcoord['R_measured'] = catalogue[ID]['R2']
matchcoord['dR_measured'] = catalogue[ID]['dR2']
crit = sep.__lt__(Angle("0.5s"))

for s,c in zip(['Kreckel PN','Kreckel SNR','Herrmann PN'],['tab:red','tab:orange','tab:blue']):
    tmp = matchcoord[(matchcoord['source']==s) & crit]
    ax1.errorbar(tmp['mOIII'],tmp['mOIII_measured'],
                 yerr = tmp['dmOIII_measured'],
                 marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label=s)
    tmp = matchcoord[(matchcoord['source']==s) & crit]
    ax2.errorbar(tmp['R'],tmp['R_measured'],
                 #xerr = tmp['dR'],
                 #yerr = tmp['dR_measured'],
                 marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label=s)


'''
   NGC5068
'''
name = 'NGC5068'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[np.isin(catalogue['type'],['PN','SNR'])]
catalogue['R2'] = catalogue['OIII5006'] / (catalogue['HA6562']+catalogue['NII6583'])
catalogue['dR2'] = catalogue['R2']  * np.sqrt(catalogue['OIII5006_err']/catalogue['OIII5006_err']**2 + 1/(catalogue['HA6562']+catalogue['NII6583'])**2 * (catalogue['HA6562_err']**2+catalogue['NII6583_err']**2) )                                  

matchcoord = pn_NGC5068_herrmann.copy()

ID, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogue['SkyCoord'])
matchcoord['mOIII_measured'] = catalogue[ID]['mOIII']
matchcoord['dmOIII_measured'] = catalogue[ID]['dmOIII']
matchcoord['R_measured'] = catalogue[ID]['R2']
matchcoord['dR_measured'] = catalogue[ID]['dR2']
crit = sep.__lt__(Angle("0.5s"))

tmp = matchcoord[(crit)]
c = 'tab:green'
ax1.errorbar(tmp['mOIII'],tmp['mOIII_measured'],
             yerr = tmp['dmOIII_measured'],
             #xerr = tmp['dmOIII'],
             marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label=s)
ax2.errorbar(tmp['R'],tmp['R_measured'],
             #xerr = tmp['sigma_R'],
             #yerr = tmp['dR_measured'],
             marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label=s)


'''
   NGC3627
'''
name = 'NGC3627'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[np.isin(catalogue['type'],['PN','SNR'])]

matchcoord = pn_NGC3627_ciardullo.copy()

ID, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogue['SkyCoord'])
matchcoord['mOIII_measured'] = catalogue[ID]['mOIII']
matchcoord['dmOIII_measured'] = catalogue[ID]['dmOIII']
crit = sep.__lt__(Angle("0.5s"))

tmp = matchcoord[crit]
c = 'cyan'
ax1.errorbar(tmp['mOIII'],tmp['mOIII_measured'],
             yerr = tmp['dmOIII_measured'],
             marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label='NGC3627')


'''
   NGC3351
'''
name = 'NGC3351'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[np.isin(catalogue['type'],['PN','SNR'])]
catalogue['R2'] = catalogue['OIII5006'] / (catalogue['HA6562']+catalogue['NII6583'])
catalogue['dR2'] = catalogue['R2']  * np.sqrt(catalogue['OIII5006_err']/catalogue['OIII5006_err']**2 + 1/(catalogue['HA6562']+catalogue['NII6583'])**2 * (catalogue['HA6562_err']**2+catalogue['NII6583_err']**2) )                                  

matchcoord = pn_NGC3351_ciardullo.copy()

ID, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogue['SkyCoord'])
matchcoord['mOIII_measured'] = catalogue[ID]['mOIII']
matchcoord['dmOIII_measured'] = catalogue[ID]['dmOIII']
crit = sep.__lt__(Angle("0.5s"))

tmp = matchcoord[crit]
c = 'purple'
ax1.errorbar(tmp['mOIII'],tmp['mOIII_measured'],
             yerr = tmp['dmOIII_measured'],
             marker='o',ms=2,ls='none',mec=c,mfc=c,ecolor=c,label='NGC3351')


ax1.plot([25,27.5],[25,27.5],color='black',lw=0.4)
ax1.set(xlim=[25,27.5],ylim=[25,27.5])
ax1.set_xlabel(r'$\mathrm{m}_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ existing studies')
ax1.set_ylabel(r'$\mathrm{m}_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ this work')
ax1.legend()


xmin,xmax = 0,7
ymin,ymax = 0,7
ax2.plot([xmin,xmax],[xmin,xmax],color='black',lw=0.4)
ax2.set_xlim([xmin,xmax])
ax2.set_ylim([ymin,ymax])
ax2.set_xlabel(r'$I_{[\mathrm{O}\,\tiny{\textsc{iii}}]}\;/\;(I_{\mathrm{H}\,\alpha}+I_{[\mathrm{N}\,\tiny{\textsc{ii}}]})$ existing studies')
ax2.set_ylabel(r'$I_{[\mathrm{O}\,\tiny{\textsc{iii}}]}\;/\;(I_{\mathrm{H}\,\alpha}+I_{[\mathrm{N}\,\tiny{\textsc{ii}}]})$ this work')
#ax2.legend(loc=2)

plt.savefig(basedir / 'reports' / f'flux_comparison.pdf',dpi=600)
plt.show()


In [ ]:
name = 'NGC0628'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
catalogue = catalogue[np.isin(catalogue['type'],['PN','SNR'])]
catalogue['R2'] = catalogue['OIII5006'] / (catalogue['HA6562']+catalogue['NII6583'])
catalogue['dR2'] = catalogue['R2']  * np.sqrt(catalogue['OIII5006_err']/catalogue['OIII5006_err']**2 + 1/(catalogue['HA6562']+catalogue['NII6583'])**2 * (catalogue['HA6562_err']**2+catalogue['NII6583_err']**2) )                                  

matchcoord = NGC628.copy()

ID, sep, _  = match_coordinates_sky(matchcoord['SkyCoord'],catalogue['SkyCoord'])
matchcoord['IDf'] = ID
matchcoord['mOIII_measured'] = catalogue[ID]['mOIII']
matchcoord['dmOIII_measured'] = catalogue[ID]['dmOIII']
matchcoord['R_measured'] = catalogue[ID]['R2']
matchcoord['dR_measured'] = catalogue[ID]['dR2']
crit = sep.__lt__(Angle("0.5s"))


### Other PNLF studies

In [ ]:
distances = ascii.read(basedir/'data'/'literature distances'/'latest.csv',delimiter=',',header_start=12,data_start=14)
distances['year'] = distances['Date (Yr. - 1980)']+1980
distances.rename_column('Galaxy ID','name')
distances['name'] = [n.rstrip('a').rstrip('b') for n in distances['name']]

results = ascii.read(basedir/'data'/'interim'/'results.txt')
print(f"intial cagalogue has {len(np.unique(distances[distances['Method']=='PNLF']['name']))} objects")

In [ ]:
tmp=distances[distances['Method']=='Cepheids']

plt.hist(tmp['D (Mpc)'],bins=np.arange(0,50,5))
plt.show()

In [ ]:
pnlf_distances = distances[distances["Method"]=='PNLF']
print(f"excluding {len(np.unique(pnlf_distances[(pnlf_distances['m-M']<22.5) | (pnlf_distances['m-M']>32)]['Galaxy ID']))} objects")
#pnlf_distances = pnlf_distances[(pnlf_distances['m-M']>22.5) & (pnlf_distances['m-M']<32)]  # exclude the many measurments of the LMC and SMC

alias = {
    'NGC 0628': 'MESSIER 074',
    'NGC 3351': 'MESSIER 095',
    'NGC 3627': 'MESSIER 066',
    'NGC 4254': 'MESSIER 099',
    'NGC 4303': 'MESSIER 061',
    'NGC 4321': 'MESSIER 100'
}

alias_back = {v:k for k,v in alias.items()}

phangs_sample = []
for row in results:
    name = row['name'].replace('NGC','NGC ').replace('IC','IC ')
    name = alias.get(name,name)
    phangs_sample.append(name)
    new = ['',0,0,name,row['(m-M)'],row['err-(m-M)'],0,'PNLF','Schmnn+2020','',0,0,0,40,'',2020]
    pnlf_distances.add_row(new)
    
galaxies = list(np.unique(pnlf_distances['name']))

In [ ]:
from pnlf.auxiliary import mu_to_parsec

d,d_err = mu_to_parsec(distances['m-M'],distances['err'])

distances['d/Mpc'] = d
distances['err d/Mpc'] = d_err

In [ ]:
distances['ep'] = distances['err d/Mpc']/distances['d/Mpc']
group = distances.group_by('Method')

means = group.groups.aggregate(np.mean)
for key, mean in zip(group.groups.keys, means):
    print('mean for {0} = {1}'.format(key, mean))

In [ ]:
fig,ax=plt.subplots()
for method in ['Cepheids','PNLF','SBF','SNII optical','SNIa','TRGB','Tully-Fisher']:
    tmp = distances[distances['Method']==method]
    ax.scatter(tmp['year'],100*tmp['err d/Mpc']/tmp['d/Mpc'],label=method)
ax.set(xlim=[1900,None],ylim=[0,100])
ax.legend()
plt.show()

In [ ]:
fig,(axes)=plt.subplots(nrows=2,ncols=2,figsize=(two_column,two_column))
axes_iter = iter(axes.flatten())
for method in ['PNLF','TRGB','Cepheids','SBF']:
    ax=next(axes_iter)
    tmp = distances[distances['Method']==method]
    ax.scatter(tmp['year'],100*tmp['err d/Mpc']/tmp['d/Mpc'])
    ax.set(xlim=[1980,None],ylim=[0,40],xlabel='year',ylabel='d/err in percent')
    ax.set_title(method)
plt.tight_layout()
plt.show()

In [ ]:
from pnlf.auxiliary import mu_to_parsec

d,d_err = mu_to_parsec(pnlf_distances['m-M'],pnlf_distances['err'])

pnlf_distances['d/Mpc'] = d
pnlf_distances['err d/Mpc'] = d_err

In [ ]:
fig,ax=plt.subplots()
ax.scatter(pnlf_distances['year'],100*pnlf_distances['err d/Mpc']/pnlf_distances['d/Mpc'])
ax.set(ylim=[1,40],xlabel='year',ylabel='err/distance')
plt.show()

In [ ]:
set(pnlf_distances[pnlf_distances['err d/Mpc']/pnlf_distances['d/Mpc']>0.15]['name'])

In [ ]:
fig,ax=plt.subplots()
ax.hist(100*pnlf_distances['err d/Mpc']/pnlf_distances['d/Mpc'],bins=np.arange(0,40,5))
ax.set(xlabel='err/distance')
plt.show()

In [ ]:
trgb_sample = set(distances[distances['Method']=='TRGB']['name'])
pnlf_sample = set(distances[distances['Method']=='PNLF']['name'])
cepheid_sample = set(distances[distances['Method']=='Cepheids']['name'])

In [ ]:
differences = []
for name in list(pnlf_sample & (cepheid_sample|trgb_sample)):
    d_pnlf = distances[(distances['name']==name) & (distances['Method']=='PNLF')]['D (Mpc)'].mean()
    d_ref = distances[(distances['name']==name) & ((distances['Method']=='TRGB')|(distances['Method']=='Cepheids'))]['D (Mpc)'].mean()
    
    dif = abs((d_ref-d_pnlf)/d_ref)
    print(f'{name}: {dif}')
    differences.append(dif)
differences = np.array(differences)


In [ ]:
np.mean(differences[differences<0.5])

In [ ]:
mu = pnlf_distances.group_by('name')['m-M'].groups.aggregate(np.mean)
dis = Distance(distmod=mu).value
plt.hist(dis,bins=np.arange(10,100,2))

plt.show()

In [ ]:
# we sort by mean distance
mean_dis = []
for gal in galaxies:
    mean_dis.append(pnlf_distances[pnlf_distances['name']==gal]['m-M'].mean())

In [ ]:
pnlf_distances.sort('year')
important_papers = dict()
is_measured = []
for row in pnlf_distances:
    #if row['name'] not in is_measured:
    #    is_measured.append(row['name'])
    if row['REFCODE'] in important_papers:
        important_papers[row['REFCODE']] += 1
    else:
        important_papers[row['REFCODE']] = 1


In [ ]:

fig, ax = plt.subplots(1,figsize=(10,6/1.618))

x_pos  = []
gal_names = [x for _,x in sorted(zip(mean_dis,galaxies))]

print(f'{len(gal_names)} galaxies in sample')
color_grid=[]
for i,gal in enumerate(gal_names):
    tmp = pnlf_distances[pnlf_distances['name']==gal]
    ax.scatter(len(tmp)*[i+1],tmp['m-M'],marker="_",color='gray')
    # color the measured by me red
    if len(tmp[tmp['REFCODE']=='Schmnn+2020'])>0:
        ax.scatter([i+1],tmp[tmp['REFCODE']=='Schmnn+2020']['m-M'],marker="_",color='tab:red')
        color_grid.append(i)

# create a legend with numbers only
#legend_elements = [mpl.lines.Line2D([0], [0], color=tab10[i], lw=2, label=str(i+1)) for i in range(10)]
#plt.legend(handles=legend_elements, loc='upper center',ncol=10)
    
ymin,ymax = 23,32
# set the galaxy names as x-ticklabels
ax.set(xticks=np.arange(1,len(galaxies)+1),
       ylabel='($m-M$) / mag',
       title='Galaxies with PNLF distances',
       xlim=[0.5,len(galaxies)+0.5],
       ylim=[ymin,ymax])
ax.set_xticklabels(gal_names,rotation=90,color='gray')    

# color the galaxies which are in the phangs sample 
for n in phangs_sample:
    i = gal_names.index(n)
    ax.get_xticklabels()[i].set_color("tab:red")
#for n in ['MESSIER 066','MESSIER 074','MESSIER 095','NGC 5068']:
#    i = gal_names.index(n)
#    ax.get_xticklabels()[i].set_color("tab:red")    

ax.grid(axis='x')
#a = ax.get_xgridlines()
#for i in color_grid:
#    a[i].set_color('tab:red')

yticks_mpc = np.logspace(np.log10(Distance(distmod=ymin).to(u.Mpc).value),np.log10(Distance(distmod=ymax).to(u.Mpc).value),10)
yticks_mu  = Distance(yticks_mpc*u.Mpc).distmod
    
ax2 = ax.twinx()
ax2.set_yticks(yticks_mu.value,minor=False)
ax2.set_yticklabels([f'{x:.2f}' for x in yticks_mpc],ha="left")
ax2.set(ylim=[ymin,ymax],ylabel='$D$ / Mpc')

plt.tight_layout()

plt.savefig(basedir/'reports'/'PNstudies.png',dpi=600)
plt.show()

In [ ]:

fig, (ax2,ax1) = plt.subplots(nrows=2,figsize=(two_column,two_column))

gal_names = [x for _,x in sorted(zip(mean_dis,galaxies))]

def literature_distances(labels,ax,ymin=23,ymax=32):
    
    # set the galaxy names as x-ticklabels
    ax.set(xticks=np.arange(1,len(labels)+1),
           ylabel='($m-M$) / mag',
           xlim=[0.5,len(labels)+0.5],
           ylim=[ymin,ymax])
    labels_new = [alias_back.get(l,l) for l in labels]
    ax.set_xticklabels(labels_new,rotation=90,color='gray')    
    ax.grid(axis='x',ls='--',lw=0.4)
    grid = ax.get_xgridlines()

    for i,label in enumerate(labels):
        tmp = pnlf_distances[pnlf_distances['name']==label]
        ax.scatter(len(tmp)*[i+1],tmp['m-M'],marker="_",color='gray')
        # color the measured by me red
        if len(tmp[tmp['REFCODE']=='Schmnn+2020'])>0:
            ax.get_xticklabels()[i].set(color="tab:red",fontweight='black')
            ax.scatter([i+1],tmp[tmp['REFCODE']=='Schmnn+2020']['m-M'],marker="_",color='tab:red')
            grid[i].set(ls='-',lw=0.5)
    
    
    yticks_mpc = np.logspace(np.log10(Distance(distmod=ymin).to(u.Mpc).value),np.log10(Distance(distmod=ymax).to(u.Mpc).value),10)
    yticks_mu  = Distance(yticks_mpc*u.Mpc).distmod

    axt = ax.twinx()
    axt.set_yticks(yticks_mu.value,minor=False)
    axt.set_yticklabels([f'{x:.2f}' for x in yticks_mpc],ha="left")
    axt.set(ylim=[ymin,ymax],ylabel='$D$ / Mpc')
    
    return ax
    
n = len(gal_names)
ax1 = literature_distances(gal_names[:int(n/2)],ax1,ymin=23,ymax=30.5)
ax2 = literature_distances(gal_names[int(n/2):],ax2,ymin=29.5,ymax=32.2)

#ax2.set_title('Galaxies with PNLF distances')
plt.tight_layout()


plt.savefig(basedir/'reports'/'PNstudies.pdf',dpi=600)
plt.savefig(basedir/'reports'/'PNstudies.png',dpi=600)

plt.show()

In [ ]:
from pnlf.auxiliary import filter_table

virgo = ['MESSIER 098','NGC 4216','MESSIER 099','MESSIER 061','MESSIER 100',
         'NGC 4365','MESSIER 084','MESSIER 085','MESSIER 086','NGC 4429',
         'NGC 4438','NGC 4442','NGC 4450','NGC 4459','MESSIER 049',
         'NGC 4473','NGC 4477','MESSIER 087','MESSIER 088','NGC 4526',
         'NGC 4535','MESSIER 091','MESSIER 089','MESSIER 090','MESSIER 058',
         'NGC 4596','MESSIER 059','NGC 4654','MESSIER 060','NGC 4762']
alias = {'MESSIER 099': 'NGC4254','MESSIER 061': 'NGC4303',
         'MESSIER 100': 'NGC4321','NGC 4535': 'NGC4535'}

fig,ax = plt.subplots(figsize=(4,8))

for i,name in enumerate(virgo):
    sub = distances[distances['name']==name]
    
    ax.scatter(sub['m-M'],len(sub)*[i],color='gray')
    
    tmp = filter_table(sub,Method=['Cepheids','TRGB','PNLF'])
    ax.scatter(tmp['m-M'],len(tmp)*[i],color='black')

    if alias.get(name,None):
        ax.scatter(results.loc[alias.get(name)]['(m-M)'],[i],color='red')
        
ax.set_yticks(np.arange(len(virgo)))
ax.set_yticklabels(virgo)
ax.set(ylim=[-0.5,len(virgo)-0.5],xlabel='(m-M) / mag')
ax.set_title('Galaxies in the Virgo Cluster')
plt.savefig(basedir/'reports'/'virgo_cluster.pdf',dpi=600)
plt.show()

### How many potential galaxies for PNLF

In [ ]:
distances = ascii.read(basedir/'data'/'literature distances'/'latest.csv',delimiter=',',header_start=12,data_start=14)
distances['year'] = distances['Date (Yr. - 1980)']+1980
distances.rename_column('Galaxy ID','name')
distances['name'] = [n.rstrip('a').rstrip('b') for n in distances['name']]

results = ascii.read(basedir/'data'/'interim'/'results.txt')
print(f"intial cagalogue has {len(np.unique(distances[distances['Method']=='PNLF']['name']))} objects")

In [ ]:
tmp = distances[(distances['D (Mpc)']<25) & np.isin(distances['Method'],['Tully-Fisher','Cepheids','SBF','TRGB','PNLF'])]
subsample = np.unique(tmp["name"])
tmp2 = tmp[~np.isin(tmp['Method'],['Cepheids','TRGB','PNLF'])]
subsample2 = np.unique(tmp2["name"])

print(f'{len(subsample)} objects in full sample (<10 Mpc)')
print(f'{len(subsample2)} objects without good distance')

In [ ]:
fig,ax = plt.subplots()
for method in ['Tully-Fisher','TRGB','Cepheids','SBF','Faber-Jackson','PNLF']:
    sub = distances[(distances['Method']==method) & (distances['D (Mpc)']<40)]
    group = sub.group_by('name')
    dis = group['D (Mpc)'].groups.aggregate(np.mean)
    
    un = np.unique(sub['name'])
    print(f'{method}: {len(un)}')
    
    ax.hist(dis,bins=np.linspace(1,31,30),label=method)
    
ax.legend()
ax.set(xlabel='Distance / Mpc',ylabel='number of galaxies')
plt.savefig(basedir/'reports'/'distances_by_method.pdf',dpi=300)
plt.show()



### Compare with Deep

In [ ]:
deep   = ascii.read(basedir/'data'/'literature distances'/'deep_distances.csv')
result = ascii.read(basedir/'data'/'interim'/'results.txt')

deep.add_index("galaxy")
deep['d'] = [float(x[:-4]) for x in deep['distance']]
deep['e'] = [float(x[:-4]) for x in deep['error']]

In [ ]:
from pnlf.auxiliary import mu_to_parsec

fig,ax = plt.subplots(figsize=(two_column,two_column/1.618))
result.sort('d/Mpc')
for i, row in enumerate(result):

    tmp = deep.loc[row['name']]
    if tmp['method'] == 'TRGB':
        d,(dp,dm) = mu_to_parsec(row['(m-M)'],[row['err+(m-M)'],row['err-(m-M)']])

        ax.errorbar(i-0.1,row['d/Mpc'],yerr=([dm.value],[dp.value]),fmt='o',color='tab:red')

        ax.errorbar(i+0.1,tmp['d'],yerr=tmp['e'],fmt='o',color='black')
        
        diff = row['d/Mpc']-tmp['d']
        if diff>0:
            err = dm
        else:
            err = dp
            
        print(f"{row['name']}: {diff/err:.2f}")
        
        
ax.set(xticks=np.arange(0,len(result)),
       ylabel='$D$ / Mpc',
       title='PHANGS distances',
       xlim=[-0.5,len(result)-0.5])
ax.set_xticklabels(result['name'],rotation=90)  
ax.grid(axis='x')

legend_elements = [mpl.lines.Line2D([0], [0], color=col, lw=2, label=l) for col,l in zip(['tab:red','black'],['PNLF','TRGB'])]
plt.legend(handles=legend_elements, loc='lower center',ncol=10)
#plt.savefig(basedir/'reports'/'PNLF_vs_TRGB.png',dpi=600)

plt.show()

In [ ]:
from pnlf.auxiliary import parsec_to_mu


distances = ascii.read(basedir/'data'/'literature distances'/'latest.csv',delimiter=',',header_start=12,data_start=14)
trgb_distances = distances[distances["Method"]=='TRGB']
trgb_distances.rename_column('Galaxy ID','name')

trgb_distances['year'] = trgb_distances['Date (Yr. - 1980)']+1980
trgb_distances['name'] = [n.rstrip('a').rstrip('b') for n in trgb_distances['name']]

alias = {
'MESSIER074' : 'NGC0628',
'MESSIER095' : 'NGC3351',
'MESSIER066' : 'NGC3627',
'MESSIER099' : 'NGC4254',
'MESSIER061' : 'NGC4303',
'MESSIER100' : 'NGC4321'
}

trgb_distances['name'] = [x.replace(' ','') for x in trgb_distances['name']]

for k,v in alias.items():
    trgb_distances['name'] = [x.replace(k,v) for x in trgb_distances['name']]
trgb_distances = trgb_distances[np.isin(trgb_distances['name'],results['name'])]

# add the new distances from deep
for name in 'IC5332', 'NGC2835', 'NGC4321':
    d = deep.loc[name]['d']
    e = deep.loc[name]['e']
    mu,err = parsec_to_mu(d*u.Mpc,e*u.Mpc)
    
    row = {'name':name,'m-M':mu,'err':err,'Method':'TRGB','D (Mpc)':d}
    trgb_distances.add_row(row)

In [ ]:
deep.loc['NGC4321']

In [ ]:
Distance(distmod=trgb['m-M']).mean()

In [ ]:
from pnlf.auxiliary import mu_to_parsec

fig,ax = plt.subplots(figsize=(two_column,two_column/1.618))

sample  = np.unique(trgb_distances['name'])

results = ascii.read(basedir/'data'/'interim'/ 'results.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
results = results[np.isin(results['name'],sample)]
results.sort('d/Mpc')
results.add_index('name')

for i, name in enumerate(results['name']):
    
    trgb = trgb_distances[trgb_distances['name']==name]
    for row in trgb:   
        d,(dp,dm) = mu_to_parsec(row['m-M'],[row['err'],row['err']])
        ax.errorbar(i-0.2,d.value,yerr=([dm.value],[dp.value]),fmt='o',color='black')
    
    pnlf = results.loc[name]
    ax.errorbar(i+0.2,pnlf['d/Mpc'],yerr=([pnlf['err-d/Mpc']],[pnlf['err+d/Mpc']]),fmt='o',color='tab:red')

    diff = pnlf['d/Mpc'] - np.mean(Distance(distmod=trgb['m-M'])).value
    if diff>0:
        err = pnlf['err-d/Mpc']
    else:
        err = pnlf['err+d/Mpc']

    print(f"{row['name']}: {diff/err:.2f}") 

for x in np.arange(-0.5,len(results)+0.5):
    ax.axvline(x,color='gray',zorder=0)
ax.set(xticks=np.arange(0,len(results)),
       ylabel='$D$ / Mpc',
       title='PNLF vs TRGB',
       xlim=[-0.5,len(results)-0.5])
ax.set_xticklabels(results['name'],rotation=90)  
#ax.grid(axis='x')

legend_elements = [mpl.lines.Line2D([0], [0], color=col, lw=2, label=l) for col,l in zip(['tab:red','black'],['PNLF','TRGB'])]
plt.legend(handles=legend_elements, loc='lower center',ncol=10)
plt.savefig(basedir/'reports'/'PNLF_vs_TRGB.png',dpi=600)

plt.show()

## PSF uncertainties

the uncertainty of the PSF is ~0.1"=0.5px. Here we explore how this impacts the measured fluxes

In [ ]:
from pnlf.photometry import measure_flux 

class DisableLogger():
    def __enter__(self):
       logging.disable(logging.CRITICAL)
    def __exit__(self, exit_type, exit_value, exit_traceback):
       logging.disable(logging.NOTSET)
        
def est_err(tbl,galaxy,aperture_size=2):
    '''estimate the flux error resulting from an uncertainy PSF
    
    The fwhm of the PSF is difficult to measure and the values we and for
    the values we use we assume an uncertainty of dFWHM = 0.1" = 0.5 px.
    '''
    tbl = tbl.copy()
    delta = 0.5
    print(f'aperture size = {aperture_size}')
    
    with DisableLogger():
        
        #print('using measured PSF')
        flux1 = measure_flux(galaxy,tbl,lines=['OIII5006'],alpha=galaxy.power_index,Rv=3.1,Ebv=galaxy.Ebv,
                            extinction='MW',background='local',aperture_size=aperture_size)
        mOIII = -2.5*np.log10(flux1['OIII5006']*1e-20) - 13.74
        dmOIII = 2.5/np.log(10) * flux1['OIII5006_err']/flux1['OIII5006']

        #print(f'using PSF-{delta}')
        tbl['fwhm'] -= delta
        flux3 = measure_flux(galaxy,tbl,lines=['OIII5006'],alpha=galaxy.power_index,Rv=3.1,Ebv=galaxy.Ebv,
                            extinction='MW',background='local',aperture_size=aperture_size)
        mOIIIm = -2.5*np.log10(flux3['OIII5006']*1e-20) - 13.74
        
        #print(f'using PSF+{delta}')
        tbl['fwhm'] += 2*delta
        flux2 = measure_flux(galaxy,tbl,lines=['OIII5006'],alpha=galaxy.power_index,Rv=3.1,Ebv=galaxy.Ebv,
                            extinction='MW',background='local',aperture_size=aperture_size)
        mOIIIp = -2.5*np.log10(flux2['OIII5006']*1e-20) - 13.74

 
    
    #print(f'PSF-(PSF+delta): {np.nanmean(mOIII[mOIII<28]-mOIIIp[mOIII<28]):.3f}')
    #print(f'PSF-(PSF-delta): {np.nanmean(mOIII[mOIII<28]-mOIIIm[mOIII<28]):.3f}')
    print(f'(PSF-delta)+(PSF-delta): {np.nanmean(mOIIIm[mOIII<28]-mOIIIp[mOIII<28])/2:.3f}')

    return mOIII,dmOIII,mOIIIp,mOIIIm


In [ ]:
from pnlf.io import ReadLineMaps

# the catalogue with the PNe (and SNRs)
with fits.open(basedir/'data'/'catalogues'/'nebulae.fits') as hdul:
    catalogue = Table(hdul[1].data)
catalogue['overluminous'] = catalogue['note']=='OL'
catalogue['exclude'] = catalogue['note']=='EX'
catalogue['SkyCoord'] = SkyCoord(catalogue['RA'],catalogue['DEC'])

with open(basedir / 'data' / 'interim' / 'parameters.yml') as yml_file:
    parameters = yaml.load(yml_file,Loader=yaml.FullLoader)
    
DR = 'DR2.1'
lines = ['OIII5006']

    
aperture_size = [0.5,1,1.5,2,2.5,3,3.5]
error_phot = {}
error_psf = {}

for gal_name in np.unique(catalogue['gal_name']):
    print(gal_name)
    
    # read in the data we will be working with and print some information
    galaxy = ReadLineMaps(data_ext/f'MUSE'/DR/'MUSEDAP',gal_name,extensions=lines,**parameters[gal_name])
    #galaxy.center = sample_table.loc[name]['SkyCoord'].to_pixel(galaxy.wcs)
    galaxy.Ebv = sample_table.loc[gal_name]['E(B-V)']
    #galaxy.posang = sample_table.loc[name]['posang']
    #galaxy.inclination = sample_table.loc[name]['Inclination']
    #galaxy.r25 = sample_table.loc[name]['r25']*u.arcmin
    
    sources = catalogue[catalogue['gal_name']==gal_name]

    error_phot_lst = []
    error_psf_lst = []
    
    for a in aperture_size:
        mOIII,dmOIII,mOIIIp,mOIIIm = est_err(sources,galaxy,aperture_size=a)

        error_phot_lst.append(np.nanmean(dmOIII[mOIII<28]))
        error_psf_lst.append(np.nanmean(mOIIIm[mOIII<28]-mOIIIp[mOIII<28])/2)
    
    error_phot[gal_name] = error_phot_lst
    error_psf[gal_name] = error_psf_lst
    

In [ ]:
error_phot_lst = error_phot[gal_name]
error_psf_lst = error_psf[gal_name]

fig,ax=plt.subplots(figsize=(single_column,single_column))

ax.plot(aperture_size,error_psf_lst,label='error from PSF',color='black',ls=':')
ax.plot(aperture_size,error_phot_lst,label='error from photometry',color='black',ls='--')
ax.plot(aperture_size,np.sqrt(np.power(error_phot_lst,2)+np.power(error_psf_lst,2)),label='total error',
        color='tab:red')
ax.legend()
ax.set_title(f'error\_PSF(fwhm=2.5)={error_psf_lst[aperture_size.index(2.5)]:.3f}')
ax.set(xlabel='aperture size / fwhm',ylabel='dmOIII / mag')
#plt.savefig(basedir/'reports'/galaxy.name/f'{galaxy.name}_aperture_size.pdf')
plt.show()

In [ ]:
from pnlf.plot.pnlf import _plot_pnlf

names = results['name']
nrows = 5
ncols = 4
filename = basedir / 'reports' / f'all_galaxies_psf_uncertainties_delte01'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

# loop over the galaxies we want to plot
for gal_name in np.unique(catalogue['gal_name']):
    
    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0
        
        
    error_phot_lst = error_phot[gal_name]
    error_psf_lst = error_psf[gal_name]

    ax.plot(aperture_size,error_psf_lst,label='error from PSF',color='black',ls=':')
    ax.plot(aperture_size,error_phot_lst,label='error from photometry',color='black',ls='--')
    ax.plot(aperture_size,np.sqrt(np.power(error_phot_lst,2)+np.power(error_psf_lst,2)),label='total error',
            color='tab:red')

    
    dPSF = error_psf_lst[aperture_size.index(2.5)]
    ax.set_title(f'$\Delta m_\mathrm{{PSF}}={dPSF:.3f}$ mag',fontsize=7)     
    ax.text(0.55,0.9,f'{gal_name}', transform=ax.transAxes,fontsize=7)
    #ax.text(0.05,0.88,f'$\Delta m_\mathrm{{PSF}}={dPSF:.3f}$', transform=ax.transAxes,fontsize=6)     

    xlim=ax.get_xlim()
    ylim=ax.get_ylim()
    ax.plot([xlim[0],2.5,2.5],[dPSF,dPSF,ylim[0]],color='black',ls=':',lw=0.5)
    ax.set(xlim=xlim,ylim=ylim)
    if i==nrows-1:
        ax.set_xlabel('aperture size / fwhm')
    if j==0:
        ax.set_ylabel(ylabel='dmOIII / mag')
    #ax.set_title(name)
    #ax.set(xlim=[24,28.5])
 
        
axes[3,3].set_xlabel('aperture size / fwhm')
ax = next(axes_iter)
#ax.remove()
h,l = fig.axes[0].get_legend_handles_labels()
ax.axis('off')
ax.legend(h,l,fontsize=7,loc='center left',frameon=False)

plt.subplots_adjust(wspace=0.15, hspace=0.15)
plt.tight_layout()
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
#plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

In [ ]:
dPSF = [v[aperture_size.index(2.5)] for k,v in error_psf.items()]

plt.hist(dPSF,bins=np.linspace(0.05,0.35,6))
plt.show()

In [ ]:
np.median(dPSF)

## Misc

### Extinction two LF

In [ ]:
from pnlf.analyse import sample_pnlf
from pnlf.plot import plot_pnlf
from pnlf.analyse import MaximumLikelihood1D, pnlf

N_PN = 100
mu = 30
cl = 28
Mmax= -4.47
nbins = 6

sample = sample_pnlf(N_PN,mu,cl)
#extinction = np.random.randint(0,2,N_PN)
extinction = np.isin(np.arange(N_PN),np.random.choice(np.arange(N_PN),size=int(N_PN/2),replace=False)).astype(float)
print(f'extinction: {np.sum(extinction)} of {N_PN}')
binsize = (cl-Mmax-mu) / nbins
ax1,ax2 = plot_pnlf(sample,mu,cl,binsize=binsize)
#ax1.set(yscale='linear')
plt.show()

In [ ]:
data = sample + 1 * extinction

fitter = MaximumLikelihood1D(pnlf,sample,mhigh=cl,Mmax=Mmax)
mu_fit,mu_p,mu_m = fitter([29])
print('without extinction: {:.2f} + {:.2f} - {:.2f}'.format(mu_fit,mu_p,mu_m))

fitter = MaximumLikelihood1D(pnlf,data[data<cl],mhigh=cl,Mmax=Mmax)
mu_fit,mu_p,mu_m = fitter([29])
print('with extinction: {:.2f} + {:.2f} - {:.2f}'.format(mu_fit,mu_p,mu_m))
binsize = (cl-Mmax-mu) / 6

fig,((ax1,ax2),(ax3,ax4)) = plt.subplots(nrows=2,ncols=2,figsize=(single_column,0.9*single_column))

ax1,ax2=plot_pnlf(sample,mu,cl,binsize=binsize,axes=(ax1,ax2))
#ax3,ax4=plot_pnlf(data[extinction>0],mu_fit,cl,binsize=binsize,axes=(ax3,ax4),color='grey')
ax3,ax4=plot_pnlf(data,mu_fit,cl,binsize=binsize,axes=(ax3,ax4))

#ax1.set_yticks([10,100])
#ax2.set_ylim([0,1000])
#ax1.set(yscale='linear')
plt.savefig(basedir/'reports'/'extinction_pnlf.pdf',dpi=600)
plt.show()

In [ ]:
from pnlf.plot.pnlf import _plot_pnlf, _plot_cum_pnlf

data1=data[extinction==0]
data2=data[extinction>0]

fig = plt.figure(figsize=(two_column,two_column/2))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

ax1 = _plot_pnlf(data1,mu,cl,binsize=binsize,mhigh=28.5,Mmax=Mmax,color=tab10[0],ax=ax1)
ax2 = _plot_cum_pnlf(data1,mu,cl,binsize=None,mhigh=28.5,Mmax=Mmax,color=tab10[0],ax=ax2)

ax1 = _plot_pnlf(data2,mu,cl,binsize=binsize,mhigh=28.5,Mmax=Mmax,color='grey',ax=ax1)
ax2 = _plot_cum_pnlf(data2,mu,cl,binsize=None,mhigh=28.5,Mmax=Mmax,color='grey',ax=ax2)

#ax1.set(yscale='linear')

plt.show()

now do it for a large sample

In [ ]:
mu   = 30
cl   = 28
N_PN = 100

mu_dict = {0.1:[],0.5:[],1:[]}

for i in range(1000):
    sample = sample_pnlf(N_PN,mu,cl)
    extinction = np.random.randint(0,2,N_PN)
    fitter = MaximumLikelihood1D(pnlf,sample,mhigh=cl,Mmax=Mmax)
    mu_ref,mu_p,mu_m = fitter([29])

    for A5007 in [0.1,0.5,1]:
        data = sample + A5007 * extinction
        fitter = MaximumLikelihood1D(pnlf,data[data<cl],mhigh=cl,Mmax=Mmax)
        mu_fit,mu_p,mu_m = fitter([29])
        #print('{}: {:.2f} + {:.2f} - {:.2f}'.format(A5007,mu_fit,mu_p,mu_m))
        mu_dict[A5007].append(mu_fit-mu_ref)

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(two_column,two_column/2.5))

ax1.hist(mu_dict[0.1],np.arange(-0.1,0.5,0.02),density=True)
ax2.hist(mu_dict[0.5],np.arange(-0.1,0.5,0.02),density=True)
ax3.hist(mu_dict[1],np.arange(-0.1,0.5,0.02),density=True)

for A5007, ax in zip((0.1,0.5,1.0),(ax1,ax2,ax3)):
    ax.set(xlabel=r'$\Delta (m-M)$',ylim=[0,15])
    ax.set_title(f'${np.mean(mu_dict[A5007]):.3f}\pm{np.std(mu_dict[A5007]):.3f}$',fontsize=7)
    label = f'$A_{{5007}}={A5007}$'
    ax.text(0.65,0.9,label, transform=ax.transAxes,fontsize=8)

ax2.set_yticklabels([])
ax3.set_yticklabels([])
    
plt.tight_layout()
plt.savefig(basedir/'reports'/'extinction_pnlf_diff.pdf')
plt.show()

take a look at uncertainty as a function of sample size

In [ ]:
# it's better to run this in the python script sample_pnlf.py

from pnlf.analyse import sample_pnlf
from pnlf.analyse import MaximumLikelihood1D, pnlf
import pickle
import logging

logging.basicConfig(stream=sys.stdout,format='%(levelname)s: %(message)s',level=logging.WARNING)

N_iter = 10
mu   = 30
Mmax = -4.47

mu_dict = {}
for cl in [26.5,27,27.5,28]:
    for N_PN in [20,50,100,150]:
        mu_dict[(N_PN,cl)] = []
        for i in range(N_iter):
            sample = sample_pnlf(N_PN,mu,cl)
            fitter = MaximumLikelihood1D(pnlf,sample,mhigh=cl,Mmax=Mmax)
            mu_fit,mu_p,mu_m = fitter([29])
            mu_dict[(N_PN,cl)].append(mu_fit)

In [ ]:
import pickle

with open(basedir/'scripts'/'sampled_pnlf.pkl','rb') as f:
    mu_dict = pickle.load(f)

In [ ]:
fig, axes = plt.subplots(ncols=4,nrows=4,figsize=(two_column,1.1*two_column))

for j, cl in enumerate([26.5,27,27.5,28]):
    for i, N_PN in enumerate([20,50,100,150]):
        ax = axes[i,j]
        ax.hist(mu_dict[(N_PN,cl)],bins=np.arange(29.76,30.26,0.02),density=True)
        ax.set(xlim=(29.75,30.25),ylim=[0,15])
        ax.set_xticks([29.8,29.9,30,30.1,30.2])
        ax.set_xticklabels([29.8,None,30.0,None,30.2])

        #ax.set_title(f'cl={cl},NPN={N_PN}')
        ax.set_title(f'${np.mean(mu_dict[(N_PN,cl)]):.3f}\pm{np.std(mu_dict[(N_PN,cl)]):.3f}$',fontsize=7)
        ax.set_yticklabels([])
        
        if j%4!=0:
            ax.set_yticklabels([])
        if i!=3:
            ax.set_xticklabels([])
        else:
            ax.set(xlabel=r'$(m-M)$')

axes[1,0].annotate(r'$N_{\mathrm{PN}}$', xy=(-0.4, -0.2), xycoords='axes fraction',rotation=90,fontsize=8)
axes[3,2].annotate(r'completeness limit / mag', xy=(-0.6,-0.55), xycoords='axes fraction',fontsize=8)


axes[0,0].annotate(r'20', xy=(-0.2,0.5), xycoords='axes fraction',rotation=90,fontsize=8)
axes[1,0].annotate(r'50', xy=(-0.2,0.5), xycoords='axes fraction',rotation=90,fontsize=8)
axes[2,0].annotate(r'100', xy=(-0.2,0.5), xycoords='axes fraction',rotation=90,fontsize=8)
axes[3,0].annotate(r'150', xy=(-0.2,0.5), xycoords='axes fraction',rotation=90,fontsize=8)


axes[3,0].annotate(r'26.5', xy=(0.5,-0.4), xycoords='axes fraction',fontsize=8)
axes[3,1].annotate(r'27.0', xy=(0.5,-0.4), xycoords='axes fraction',fontsize=8)
axes[3,2].annotate(r'27.5', xy=(0.5,-0.4), xycoords='axes fraction',fontsize=8)
axes[3,3].annotate(r'28.0', xy=(0.5,-0.4), xycoords='axes fraction',fontsize=8)


# the horizontal one (completeness limit)
con = mpl.patches.ConnectionPatch(xyA=(0.9,-0.45), xyB=(0.1,-0.45), 
                                  coordsA="axes fraction", coordsB="axes fraction",
                                  axesA=axes[3,3], axesB=axes[3,0], color="black",linewidth=0.6,
                                  arrowstyle='<|-',)
axes[3,3].add_artist(con)


# the vertical one (N PN)
con = mpl.patches.ConnectionPatch(xyA=(-0.26,0.9), xyB=(-0.26,0.1), 
                                  coordsA="axes fraction", coordsB="axes fraction",
                                  axesA=axes[0,0], axesB=axes[3,0], color="black",linewidth=0.6,
                                  arrowstyle='-|>',)
axes[0,0].add_artist(con)

plt.tight_layout()
plt.subplots_adjust(hspace=0.2,wspace=0.1)

plt.savefig(basedir/'reports'/'sample_PNLF.pdf',dpi=600)
plt.show()

In [ ]:
results['range'] = 28-(results['(m-M)']-4.47)

fig,ax=plt.subplots()

ax.scatter(results['range'],results['N_PN'])
for name in results['name']:
    ax.text(results.loc[name]['range'],results.loc[name]['N_PN'],name)
       
plt.show()

In [ ]:
data = sample[10:60]

print(f'{len(data)} data points (min={np.min(data):.2f},max={np.max(data):.2f})')
fitter = MaximumLikelihood1D(pnlf,data[data<cl],mhigh=cl,Mmax=Mmax)
mu_fit,mu_p,mu_m = fitter([29])
print('{:.2f} + {:.2f} - {:.2f}'.format(mu_fit,mu_p,mu_m))

ax1,ax2=plot_pnlf(data,mu_fit,cl,binsize=binsize)
#ax1.set(yscale='linear')
plt.show()

### PSF uncertainties

for each galaxy we estimate the impact of the PSF uncertainties on the final magnitude uncertainty. This section is to showcase the impact that the individual errors have on the final uncertainty

In [ ]:
tmp = catalogue[(catalogue['type']=='PN') & (catalogue['mOIII']<28)].copy()

tmp['dPSF'] = np.nan
for gal_name in np.unique(catalogue['gal_name']):
    tmp['dPSF'][tmp['gal_name']==gal_name] = parameters[gal_name]['dPSF']
tmp['dmOIII_old'] = np.sqrt(tmp['dmOIII']-tmp['dPSF'])
for gal_name in np.unique(catalogue['gal_name']):
    t = tmp[tmp['gal_name']==gal_name]
    print(f"{gal_name}: {np.mean(t['dmOIII_old']):.3f}, {parameters[gal_name]['dPSF']:.3f}, {np.mean(t['dmOIII']):.3f}")
print(f"\nmedian dPSF: {np.median(tmp['dPSF']):.2f}\nmedian dmOIII: {np.median(tmp['dmOIII_old']):.2f}")


### Change the classification criteria

In [ ]:
from pnlf.analyse import emission_line_diagnostics

# the table with the measured distances
results = ascii.read(basedir/'data'/'interim'/ 'results_median_bkg.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
results.add_index('name')

name = 'IC5332'

completeness_limit = parameters[name]['completeness_limit']
distance_modulus = results.loc[name]['(m-M)']

catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
    catalogue['overluminous'] = catalogue['overluminous'].astype(bool)

tbl = emission_line_diagnostics(catalogue,distance_modulus,completeness_limit)

In [ ]:
pn_original = catalogue[(catalogue['type']=='PN') & (catalogue['mOIII']<completeness_limit)]
pn_new = tbl[(tbl['type']=='PN') & (tbl['mOIII']<completeness_limit)]

print(f'original: {len(pn_original)}, new: {len(pn_new)}')

### Median vs Mean Background

In [ ]:
results_mean = ascii.read(basedir/'data'/'interim'/ 'results_submitted.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
results_mean.add_index('name')
results_median = ascii.read(basedir/'data'/'interim'/ 'results.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
results_median.add_index('name')

fig,ax=plt.subplots(figsize=(two_column,two_column))

lim = [28,32.5]
ax.errorbar(results_mean['(m-M)'],results_median['(m-M)'],
            xerr=[results_mean['err+(m-M)'],results_mean['err-(m-M)']],
            yerr=[results_median['err+(m-M)'],results_median['err-(m-M)']],fmt='o')
ax.plot(lim,lim,color='black')
ax.set(xlim=lim,ylim=lim,xlabel='(m-M) mean background',ylabel='(m-M) median background')

for gal_name in ['NGC1566','NGC7496','NGC4254','NGC1087','NGC1385']:
    ax.text(results_mean.loc[gal_name]['(m-M)'],results_median.loc[gal_name]['(m-M)'],gal_name,
         ha='right',fontsize='7',zorder=4)
    
    
plt.show()


### Bootstrapping for stat error

In [ ]:
results.add_column(0.0,index=9,name='err_stat')

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, pnlf

sample = catalogue[(catalogue['type']=='PN') & (catalogue['note']=='')]

for gal_name in np.unique(catalogue['gal_name']):
    
    cl = parameters[gal_name]['completeness_limit']
    subsample = sample[(sample['gal_name']==gal_name) & (sample['mOIII']<cl)]['mOIII']
    
    print(gal_name)
    fitter = MaximumLikelihood1D(pnlf,subsample,mhigh=28,Mmax=-4.47)
    mu_fit,mu_p,mu_m = fitter([29])
    print('           {:.2f} + {:.2f} - {:.2f}'.format(mu_fit,mu_p,mu_m))

    mu_boot, std_boot = fitter.bootstrap([29],N_boot=1000)
    results.loc[gal_name]['err_stat'] = std_boot
    print('Bootstrap: {:.2f} +- {:.2f}'.format(mu_boot,std_boot))

In [ ]:
results['err+'] = np.sqrt(results['err+(m-M)']**2 + results['err_stat']**2)
results['err-'] = np.sqrt(results['err-(m-M)']**2 + results['err_stat']**2)

results['err_stat'].info.format='%.2f'
results['err+'].info.format='%.3f'
results['err-'].info.format='%.3f'

In [ ]:
print(results[['name','(m-M)','err_stat','err+(m-M)','err-(m-M)']])

In [ ]:
from pnlf.plot import plot_pnlf

gal_name = 'NGC1433'
cl = parameters[gal_name]['completeness_limit']
subsample = sample[(sample['gal_name']==gal_name) & (sample['mOIII']<cl)]['mOIII']
mu = results.loc[gal_name]['(m-M)']
binsize = (cl+4.47-mu)/3
plot_pnlf(subsample,mu,cl,binsize=binsize)
plt.show()

### Plot entire sky

In [ ]:
filename = basedir / 'data' / 'interim' / 'sample.txt'
sample = ascii.read(filename,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
sample['SkyCoord'] = SkyCoord(sample['R.A.'],sample['Dec.'])

ra = sample['SkyCoord'].ra
ra = ra.wrap_at(180*u.degree)
dec = sample['SkyCoord'].dec

In [ ]:
#mpl.use('pdf')
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(ra.radian,dec.radian,marker='.')
#ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.grid(True)
ax.set_xticklabels([])
ax.set_yticklabels([])

for x,y,s in zip(ra,dec,sample['Name']):
    ax.annotate(s,(x.radian,y.radian),xycoords='data',size='x-small')

fig.savefig("map.pdf")


In [ ]:
# https://github.com/henrysky/milkyway_plot/blob/master/mw_plot/mw_plot_classes.py
image_filename = basedir/'data'/'interim'/'MW_edgeon_unannotate.jpg'
img = plt.imread(image_filename)
img = img[1625:4875]  # so there are 3250px there

center=(0, 0) * u.deg
radius=(180, 90) * u.deg
    
y_img_center = 1625 - int((3250 / 180) * center[1].value)
y_radious_px = int((3250 / 180) * radius[1].value)
x_img_center = int((6500 / 360) * center[0].value) + 3250
x_radious_px = int((6500 / 360) * radius[0].value)

img = img[(y_img_center - y_radious_px):(y_img_center + y_radious_px),
             (x_img_center - x_radious_px):(x_img_center + x_radious_px), :]

'''
fig = plt.figure()
ax = fig.add_subplot(111, projection='mollweide')

lon = np.linspace(-np.pi, np.pi, 6500)
lat = np.linspace(np.pi / 2., -np.pi / 2., 3250)
Lon, Lat = np.meshgrid(lon, lat)

plt.show()
'''

In [ ]:
positions = {'IC5332':(4,2),
'NGC0628':(0,1),
'NGC1087':(0,2),
'NGC1300':(0,3),
'NGC1365':(4,3),
'NGC1385':(0,4),
'NGC1433':(4,5),
'NGC1512':(4,4),
'NGC1566':(3,6),
'NGC1672':(4,6),
'NGC2835':(2,6),
'NGC3351':(1,6),
'NGC3627':(0,6),
'NGC4254':(1,0),
'NGC4303':(3,0),
'NGC4321':(0, 0),
'NGC4535':(2,0),
'NGC5068':(4,0),
'NGC7496':(4,1)}

va = {
'IC5332':'bottom',
'NGC0628':'bottom',
'NGC1087':'bottom',
'NGC1300':'center',
'NGC1365':'center',
'NGC1385':'center',
'NGC1433':'center',
'NGC1512':'center',
'NGC1566':'center',
'NGC1672':'center',
'NGC2835':'center',
'NGC3351':'top',
'NGC3627':'bottom',
'NGC4254':'top',
'NGC4303':'top',
'NGC4321':'bottom',
'NGC4535':'center',
'NGC5068':'bottom',
'NGC7496':'top'}
ha = {
'IC5332':'center',
'NGC0628':'center',
'NGC1087':'center',
'NGC1300':'right',
'NGC1365':'right',
'NGC1385':'left',
'NGC1433':'right',
'NGC1512':'left',
'NGC1566':'right',
'NGC1672':'left',
'NGC2835':'right',
'NGC3351':'right',
'NGC3627':'right',
'NGC4254':'left',
'NGC4303':'left',
'NGC4321':'left',
'NGC4535':'left',
'NGC5068':'center',
'NGC7496':'center'}

In [ ]:
from pnlf.plot.plot import create_RGB


fig, axs = plt.subplots(ncols=7, nrows=5,figsize=(1.5*two_column,two_column))
gs = axs[1, 2].get_gridspec()

path = data_raw / 'MUSE_DR2' / 'filterImages' 

# remove the underlying axes
for ax in axs[1:-1,1:-1].flatten():
    ax.remove()
ax = fig.add_subplot(gs[1:-1,1:-1],projection="mollweide")
ax.pcolormesh(Lon, Lat,img[:, :, 0], cmap='gray', zorder=2, alpha=0.85, rasterized=True)
ax.plot(ra.radian,dec.radian,'.r',ms=1)

#ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.grid(True)
ax.set_xticklabels([])
ax.set_yticklabels([])

for x,y,name in zip(ra,dec,sample['Name']):
    #ax.annotate(s,(x.radian,y.radian),xycoords='data',size='x-small',position='top')
    ax.text(x.radian,y.radian,name,
            horizontalalignment=ha[name],
            verticalalignment=va[name],
            fontsize=6,
            color='white')

for name,idx in positions.items():

    sdss_g, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_g_WCS_Pall_mad.fits',header=True)
    sdss_r, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_r_WCS_Pall_mad.fits',header=True)
    sdss_i, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_i_WCS_Pall_mad.fits',header=True)
    
    #ax=axs[idx]
    axs[idx].remove()
    ax = fig.add_subplot(gs[idx],projection=WCS(h))
    
    gri = create_RGB(sdss_i,sdss_r,sdss_g,weights=[1,1,1],percentile=[99,99,99])
    gri[sdss_g==0] = (1,1,1)
    ax.imshow(gri)
    
    #ax.annotate(f'{k}',(0.1, 0.5),xycoords='axes fraction', va='center')
    ax.set_title(name,fontsize=6)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_axis_off()
    
axs[(0,5)].remove()

#fig.tight_layout()
plt.subplots_adjust(wspace=0,hspace=0.3)

plt.savefig(basedir/'reports'/'all_galaxies_sky.pdf',dpi=600)
plt.show()

### Angular resolution of all galaxies

In [ ]:
res = []
ang = []
for name in results['name']:
    with fits.open(data_raw/'MUSE_DR2.1'/'AUXILIARY'/'seeing_maps'/f'{name}_seeing.fits') as hdul:
        PSF = hdul[0].data
    
    res_min = np.nanmin(PSF)/206265*results.loc[name]['d/Mpc']*1e6
    res_max = np.nanmax(PSF)/206265*results.loc[name]['d/Mpc']*1e6
        
    ang_min = np.nanmin(PSF)
    ang_max = np.nanmax(PSF)
    
    res.append(np.nanmean(PSF)/206265*results.loc[name]['d/Mpc']*1e6)
    ang.append(np.nanmean(PSF))
    
    print(f'{name}: min={ang_min:.2f}", max={ang_max:.2f}"')
    print(f'{name}: min={res_min:.2f} pc, max={res_max:.2f} pc')

In [ ]:
pointings = ascii.read(basedir/'data'/'external'/'PHANGS_pointings.csv',delimiter=';')
pointings = pointings[~pointings['PSF'].mask]
pointings['name'] = [x.split(' ')[0] for x in pointings['pointing']]

fig,ax=plt.subplots()

ax.hist(pointings['PSF'],bins=np.linspace(0.3,1.2,20))
ax.set(xlabel='FWHM / arcsec',xlim=[0.3,1.2])
plt.show()

In [ ]:
for name in np.unique(pointings['name']):
    print(name, np.mean(pointings['PSF'][pointings['name']==name]))

### Match catalogue

In [ ]:
name = 'NGC0628'
catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
if catalogue_file.is_file():
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['exclude'] = catalogue['exclude'].astype(bool)
catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])


In [ ]:
RA=24.1888
DEC=15.7968

coord = SkyCoord(RA*u.degree,DEC*u.degree)

sep = coord.separation(catalogue['SkyCoord'])

catalogue[np.argmin(sep)][['id','type']]

compare with Francesco's nebula catalogue

In [ ]:
pn = Table(fits.getdata(basedir/'data'/'catalogues'/'PN_catalogue.fits'))
pn['SkyCoord'] = SkyCoord(pn['RA'],pn['DEC'])

with fits.open(basedir /'..'/'cluster' / 'data' / 'interim' / 'Nebulae_Catalogue_v2p1.fits') as hdul:
    nebulae = Table(hdul[1].data)
nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg,frame='icrs')
 

region_ID = []
for gal_name in np.unique(pn['gal_name']):
    print(gal_name)
    
    sub = pn[pn['gal_name']==gal_name]
    
    filename = data_ext/'Products'/'Nebulae catalogue' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
    with fits.open(filename) as hdul:
        nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
        nebulae_mask.data[nebulae_mask.data==-1] = np.nan
    
    for row in sub:
        x,y = row[['x','y']]
        region_ID.append(nebulae_mask.data[int(y),int(x)])
p = 100*np.sum(~np.isnan(region_ID))/len(region_ID)
p = 100*np.sum(~np.isnan(region_ID) & (pn['type']=='PN'))/np.sum(pn['type']=='PN')
print(f'nebulae catalogue contains {p:.2f}% of PNe')

In [ ]:
from astropy.coordinates import match_coordinates_sky

pn = Table(fits.getdata(basedir/'data'/'catalogues'/'PN_catalogue.fits'))
pn['SkyCoord'] = SkyCoord(pn['RA'],pn['DEC'])

with fits.open(basedir /'..'/'cluster' / 'data' / 'interim' / 'Nebulae_Catalogue_v2p1.fits') as hdul:
    nebulae = Table(hdul[1].data)
nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg,frame='icrs')
 
match = 0
for gal_name in np.unique(pn['gal_name']):
    
    sub_pn  = pn[(pn['gal_name']==gal_name) & (pn['type']=='PN')]
    sub_hii = nebulae[nebulae['gal_name']==gal_name]
    
    idx,sep,_ = match_coordinates_sky(sub_pn['SkyCoord'],sub_hii['SkyCoord'])
    
    match+=len(sub_pn[sep<Angle('1"')])
    
print(f'nebulae catalogue matches {100*match/np.sum(pn["type"]=="PN"):.2f}% of PNe')

In [ ]:
pn['OIII5007'] = 10**(-(pn['mOIII']+13.74)/2.5)
pn['HA6562'] = pn['OIII5007'] / 10**pn['logOIII/Ha']
xlim = [np.min(pn['HA6562']),np.max(pn['HA6562'])]

xlim =[2e-19,2e-16]
bins = np.logspace(*np.log10(xlim),20)

fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

#ax.hist(nebulae['HA6562_FLUX']*1e-20,bins=bins,label='HII regions',alpha=0.6)
ax.hist(pn[np.isnan(region_ID) & (pn['type']=='PN')]['HA6562'],bins=bins,label='not in nebula catalogue',alpha=0.6)
ax.hist(pn[~np.isnan(region_ID) & (pn['type']=='PN')]['HA6562'],bins=bins,label='in nebula catalogue',alpha=0.6)
ax.legend()

ax.set(xscale='log',xlim=xlim,xlabel='HA6562 / erg s-1 cm-2')
plt.savefig(basedir/'reports'/'pn_Halpha_histogram.pdf',dpi=400)
plt.show()

In [ ]:
xlim =[1e-17,1e-15]
bins = np.logspace(*np.log10(xlim),20)

fig,ax=plt.subplots()

ax.hist(pn[np.isnan(region_ID) & (pn['type']=='PN')]['OIII5007'],bins=bins,label='not in nebula catalogue',alpha=0.6)
ax.hist(pn[~np.isnan(region_ID) & (pn['type']=='PN')]['OIII5007'],bins=bins,label='in nebula catalogue',alpha=0.6)
ax.legend()

ax.set(xscale='log',xlim=xlim,xlabel='OIII5007')

plt.show()

In [ ]:
np.mean(pn[np.isnan(region_ID) & (pn['type']=='PN')]['HA6562'])

### Measure mass in mask

In [ ]:
from pnlf.io import ReadLineMaps
from regions import PixCoord,EllipsePixelRegion

observed_mass = {}
age_mw = []


with open(basedir / 'data' / 'interim' / 'parameters.yml') as yml_file:
    parameters = yaml.load(yml_file,Loader=yaml.FullLoader)
        
for name in results['name']:

    lines = ['OIII5006', 'HA6562']

    # read in the data we will be working with and print some information
    galaxy = ReadLineMaps(data_ext/'MUSE_DR2'/'MUSEDAP',name,extensions=lines,**parameters[name])
    galaxy.center = sample_table.loc[name]['SkyCoord'].to_pixel(galaxy.wcs)
    galaxy.Ebv = sample_table.loc[name]['E(B-V)']
    galaxy.posang = sample_table.loc[name]['posang']
    galaxy.inclination = sample_table.loc[name]['Inclination']
    galaxy.r25 = sample_table.loc[name]['r25']*u.arcmin

    eccentricity = np.sin(galaxy.inclination*u.deg).value
    width = 0.2*(galaxy.r25/u.arcmin*300).value  # convert arcmin to pixel
    # angle uses x-axis but posang is defined from north pole (y-axis)
    aperture = EllipsePixelRegion(PixCoord(*galaxy.center),
                                  width=width,
                                  height=np.sqrt((width)**2 * (1-eccentricity**2)),
                                  angle=(galaxy.posang-90)*u.deg)
    center_mask = aperture.to_mask().to_image(galaxy.shape).astype(bool)

    mask = np.zeros(galaxy.shape,dtype=bool)
    mask |= galaxy.star_mask.astype(bool)
    if hasattr(galaxy,'mask'):
        print('masking parts of the image')
        mask[galaxy.HA6562>getattr(galaxy,'HAmask',np.nanpercentile(galaxy.HA6562,95))]=True
        mask |=center_mask
    if name=='NGC1566':
        # this galaxy has one extremely noise pointing
        mask[galaxy.PSF==3.11]=True

    with fits.open(galaxy.filename) as hdul:
        data = hdul['AGE_MW'].data
    age_mw.append(np.nanmedian(data[~mask]))
        
    area_per_pixel =  (0.2*Distance(distmod=sample_table.loc[name]['(m-M)'])*u.arcsec.to(u.rad)).to(u.pc)**2
    m = np.log10(area_per_pixel.value * np.nansum(galaxy.stellar_mass[~mask]))
    observed_mass[name] = m
    

In [ ]:
a,b = [],[]

for k,v in observed_mass.items():
    a.append(k)
    b.append(v)
t = Table([a,b],names=['name','obs_mass'])
t['age_mw'] = age_mw
t['age_mw'].info.format = '%.3f'
t['obs_mass'].info.format = '%.3f'

with open(basedir/'data'/'interim'/'observed_mass.txt','w',newline='\n') as f:
    ascii.write(t,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')


### ...

In [ ]:
from pathlib import Path
import numpy as np
import astropy.units as u
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import match_coordinates_sky, Angle, SkyCoord

# nebulae catalogue from Francesco (mostly HII-regions)
filename = data_ext / 'MUSE_DR2.0' / 'Nebulae catalogue' / f'Nebulae_Catalogue_DR2_native.fits'
with fits.open(filename) as hdul:
    nebulae = Table(hdul[1].data)
nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg)
nebulae.rename_columns(['cen_x','cen_y'],['x','y'])
nebulae['mOIII'] = -2.5*np.log10(nebulae['OIII5006_FLUX']*1e-20) - 13.74
nebulae.add_index('region_ID')

# planetary nebulae catalogue
filename = basedir / 'data' / 'catalogues' / f'nebulae.fits'
with fits.open(filename) as hdul:
    PN_candidates = Table(hdul[1].data)
#PN_candidates['SkyCoord'] = SkyCoord(PN_candidates['RA'],PN_candidates['DEC'])
PN_candidates = PN_candidates[(PN_candidates['mOIII']<28) & (PN_candidates['type']=='HII')]

tolerance = Angle('1"')

# match the two catalogues
idx, sep, _  = match_coordinates_sky(nebulae['SkyCoord'],PN_candidates['SkyCoord'])
within_tolerance = len(sep[sep.__lt__(tolerance)])
print(f'{within_tolerance} nebulae of {len(sep)} are also in PNe catalogue ({100*within_tolerance/len(sep):.2f}%)')

idx, sep, _  = match_coordinates_sky(PN_candidates['SkyCoord'],nebulae['SkyCoord'])
within_tolerance = len(sep[sep.__lt__(tolerance)])
print(f'{within_tolerance} PN of {len(sep)} are also in nebulae catalogue ({100*within_tolerance/len(sep):.2f}%)')

In [ ]:
criteria = sep<tolerance
fig,ax=plt.subplots()
ax.scatter(PN_candidates[criteria]['mOIII'],nebulae[idx[criteria]]['mOIII'])
ax.plot([20,28],[20,28],zorder=2,color='black')
ax.set(xlabel='mOIII (Fabian)',ylabel='mOIII (Francesco)')
plt.show()

In [ ]:
from skimage.measure import find_contours

name = 'NGC4254'
subsample = PN_candidates[(sep>tolerance) & (PN_candidates['gal_name']==name)]

# DAP linemaps (Halpha and OIII)
filename = data_ext / 'MUSE_DR2.0' / 'MUSEDAP' / f'{name}_MAPS.fits'
with fits.open(filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))
    OIII = NDData(data=hdul['OIII5006_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['OIII5006_FLUX_ERR'].data),
                    mask=np.isnan(hdul['OIII5006_FLUX'].data),
                    meta=hdul['OIII5006_FLUX'].header,
                    wcs=WCS(hdul['OIII5006_FLUX'].header))
# nebulae spatial masks
filename = data_ext / 'MUSE_DR2.0' / 'Nebulae catalogue' /'spatial_masks'/f'{name}_HIIreg_mask.fits'
with fits.open(filename) as hdul:
    nebulae_mask = NDData(hdul[0].data-1,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
    
ncols = 6
nrows = int(np.ceil(len(subsample)/ncols))

width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())
    
for row in subsample:  

    ax = next(axes_iter)
    
    data_cutout = Cutout2D(OIII.data,(row['x'],row['y']),size=16)
    mask_cutout = Cutout2D(nebulae_mask.data,(row['x'],row['y']),size=16)
    
    norm = simple_norm(data_cutout.data,'linear',clip=False,percent=99)
    ax.imshow(data_cutout.data,origin='lower',norm=norm,cmap=plt.cm.gray_r)
    
    region_ID = np.unique(mask_cutout.data[~np.isnan(mask_cutout.data)])

    contours = []
    for i in region_ID:
        blank_mask = np.zeros_like(mask_cutout.data)
        blank_mask[mask_cutout.data==i] = 1
        contours += find_contours(blank_mask, 0.5)

    for coords in contours:
        ax.plot(coords[:,1],coords[:,0],color='tab:red',lw=0.5)
    
    x,y=data_cutout.input_position_cutout 
    aperture = CircularAperture((x,y), r=2.5)
    aperture.plot(color='tab:blue',lw=.5, alpha=1,axes=ax)

    ax.axis('off')
    
for i in range(nrows*ncols-len(subsample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()
fig.suptitle(name,y=0.9)
plt.subplots_adjust(wspace=0.1, hspace=.0)

plt.show()

use spatial masks

In [ ]:
from astropy.wcs import WCS
from astropy.table import join 

def get_value(data,x,y):
    '''get the value of data at position x/y
    
    Parameters
    ----------
    data : ndarray
    x : array 
    y : array
    '''
    
    shape = data.shape
    
    out = []
    for j,i in zip(x,y):
        if 0<i<shape[0] and 0<j<shape[1]:
            out.append(data[int(i),int(j)])
        else:
            out.append(np.nan)
    return out


PN_candidates['region_ID'] = np.nan

for name in np.unique(PN_candidates['gal_name']):
    #print(f'working on {name}')
    
    # read in the nebulae spatial mask
    filename = data_ext / 'MUSE_DR2.0' / 'Nebulae catalogue' /'spatial_masks'/f'{name}_HIIreg_mask.fits'
    with fits.open(filename) as hdul:
        nebulae_mask = NDData(hdul[0].data-1,meta=hdul[0].header,wcs=WCS(hdul[0].header))
        nebulae_mask.data[nebulae_mask.data==-1] = np.nan
    
    criteria = PN_candidates['gal_name']==name
    PN_candidates['region_ID'][criteria] = get_value(nebulae_mask.data,x=PN_candidates['x'][criteria],y=PN_candidates['y'][criteria])
    
joined_catalogue = join(PN_candidates,nebulae,keys=['gal_name','region_ID'])

### BPT

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))


tmp = catalogue[catalogue['type']=='SNR']
ax.scatter(np.log10((tmp['SII6716']+tmp['SII6730'])/tmp['HA6562']),np.log10(tmp['OIII5006']/tmp['HB4861']),
           color='tab:orange',label='SNR',s=0.8)

tmp = catalogue[catalogue['type']=='HII']
ax.scatter(np.log10((tmp['SII6716']+tmp['SII6730'])/tmp['HA6562']),np.log10(tmp['OIII5006']/tmp['HB4861']),
           color='tab:blue',label='HII',s=0.8)

tmp = catalogue[catalogue['type']=='PN']
ax.scatter(np.log10((tmp['SII6716']+tmp['SII6730'])/tmp['HA6562']),np.log10(tmp['OIII5006']/tmp['HB4861']),
           color='tab:red',label='PN',s=0.8)

ax.legend()
ax.set(xlim=[-2.,1],ylim=[-1.5,2])

plt.show()

In [ ]:
with fits.open(data_ext/'MUSE_DR2.1'/'datacubes'/'IC5332_DATACUBE_FINAL_P01.fits') as hdul:
    primary_header = hdul[0].header
    data_header = hdul[1].header

In [ ]:
fig,ax=plt.subplots()

ax.hist(catalogue['v_SIGMA'],bins=np.arange(0,200,10))
ax.set(xlim=[0,200])

plt.show()

In [ ]:
tbl = catalogue[catalogue['mOIII']<28]
print(f"all:     {np.nanmean(tbl['v_SIGMA']):.2f},  {np.nanstd(tbl['v_SIGMA']):.2f}")

for name in results['name']:
    tmp = tbl[tbl['gal_name']==name]
    
    print(f"{name}: {np.nanmean(tmp['v_SIGMA']):.2f},  {np.nanmedian(tmp['v_SIGMA']):.2f}")

In [ ]:
np.nanmedian(catalogue['v_SIGMA'])

### Use Enricos catalgoue for the fit

In [ ]:
with fits.open(basedir/'data'/'external'/'PNe_enrico.fits') as hdul:
    catalogue = Table(hdul[1].data)
catalogue['gal_name'] = [x.strip() for x in catalogue['gal_name']]
catalogue.rename_columns(['cen_x','cen_y'],['x','y'])

catalogue['mOIII'] = -2.5*np.log10(catalogue['OIII5006_FLUX_DIG']*1e-20) - 13.74
catalogue['dmOIII'] = np.abs( 2.5/np.log(10) * catalogue['OIII5006_FLUX_ERR_DIG'] / catalogue['OIII5006_FLUX_DIG'] )
catalogue = catalogue[np.isfinite(catalogue['mOIII'])]

for name in results['name']:
    data = catalogue[catalogue['gal_name']==name]['mOIII']
    print(name,len(data[data<28]))
    
print(f'N_PN={len(catalogue[catalogue["mOIII"]<28])}')


# to use with `pnlf.photometry.measure_flux`
#sources['fwhm'] = [galaxy.PSF[int(y),int(x)] for x,y in sources[['x','y']]]
#sources['sharpness'] = 0.5
#sources['roundness2'] = 0.5

In [ ]:
from pnlf.analyse import MaximumLikelihood1D, pnlf, cdf
from pnlf.plot.pnlf import plot_pnlf
from pnlf.auxiliary import mu_to_parsec
from scipy.stats import kstest

name = 'NGC4303'
completeness_limit = 28

Mmax = -4.47

data = catalogue[catalogue['gal_name']==name]['mOIII']
err = catalogue[catalogue['gal_name']==name]['dmOIII']

if name in ['NGC2835','NGC4303']:
    #data,err = data[data>26.8],err[data>26.8]
    mask = data<26.
else:
    mask = np.zeros(len(data),dtype=bool)
    
print(f'analysing {name} (sample table: {parameters[name]["mu"]})')
print(f'completeness limit = {completeness_limit}')
fitter = MaximumLikelihood1D(pnlf,data[(data<completeness_limit) & ~mask],err=err[(data<completeness_limit) & ~mask],
                             mhigh=completeness_limit,Mmax=Mmax)
mu,mu_p,mu_m = fitter([29])

d,(dp,dm)=mu_to_parsec(mu,[mu_p,mu_m])
print('{:.2f} + {:.2f} - {:.2f}'.format(mu,mu_p,mu_m))
print('{:.2f} + {:.2f} - {:.2f}'.format(d,dp,dm))

ks,pv = kstest(data[(data<completeness_limit) & ~mask],cdf,args=(mu,completeness_limit))
print(f'{name}: statistic={ks:.3f}, pvalue={pv:.3f}')

binsize = (completeness_limit-Mmax-mu) / 2

axes = plot_pnlf(data,mu,completeness_limit,mask=mask,
                 binsize=binsize,mhigh=28.5,Mmax=Mmax,filename=None,color=tab10[0])

In [ ]:
from pnlf.plot.pnlf import _plot_pnlf

nbins = {'IC5332':3,'NGC0628':3,'NGC1087':3,'NGC1300':2,'NGC1365':3,
         'NGC1385':4,'NGC1433':3,'NGC1512':3,'NGC1566':3,'NGC1672':2,
         'NGC2835':3,'NGC3351':3,'NGC3627':3,'NGC4254':4,'NGC4303':2,
         'NGC4321':3,'NGC4535':3,'NGC5068':3,'NGC7496':3}

completeness = { 'NGC1566':28,'NGC3351':28,'NGC4303':28,'NGC4535':28,'NGC5068':27}    

sample = ['IC5332','NGC0628','NGC1433','NGC1566','NGC2835','NGC3351','NGC3627','NGC4303','NGC4535','NGC5068']

cuts = {'NGC2835':26,'NGC3351':25.5,'NGC4303':26}


Enrico_distances = {}

names = results['name']
nrows = 3
ncols = 4
filename = basedir / 'reports' / f'pnlf_enrico'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())
Mmax = -4.47

# loop over the galaxies we want to plot
#for name in names:  
for name in sample:
    
    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0
        
    data = catalogue[catalogue['gal_name']==name]['mOIII']
    err = catalogue[catalogue['gal_name']==name]['dmOIII']
    completeness_limit = completeness.get(name,27.5)
    
    if name in cuts.keys():
        #data,err = data[data>26.8],err[data>26.8]
        mask = data<cuts.get(name,0)
    else:
        mask = np.zeros(len(data),dtype=bool)
    
    
    print(f'analysing {name} (sample table: {parameters[name]["mu"]})')
    fitter = MaximumLikelihood1D(pnlf,data[(data<completeness_limit) & ~mask],
                                 err=err[(data<completeness_limit) & ~mask],
                                 mhigh=completeness_limit,Mmax=Mmax)
    mu,mu_p,mu_m = fitter([29])
    d,(dp,dm)=mu_to_parsec(mu,[mu_p,mu_m])
    print('{:.2f} + {:.2f} - {:.2f}'.format(mu,mu_p,mu_m))
    print('{:.2f} + {:.2f} - {:.2f}'.format(d,dp,dm))

    Enrico_distances[name] = (mu,mu_p,mu_m)
 
        
    ks,pv = kstest(data[data<completeness_limit],cdf,args=(mu,completeness_limit))
    print(f'{name}: statistic={ks:.3f}, pvalue={pv:.3f}')

    mlow = Mmax+mu
    binsize = (completeness_limit-mlow) / nbins[name]
    mhigh = completeness_limit+1.5*binsize
    
    ax=_plot_pnlf(data,mu,completeness_limit,mask=mask,binsize=binsize,mhigh=mhigh,ax=ax,ms=3)
        
    ylim=ax.get_ylim()
    y2 = ylim[1]*1.7
    if y2>100:y2=99
    ax.set_ylim([0.7,y2])
    if name in ['NGC1433','NGC1512']:
        ax.set_ylim([None,99])
    if name=='NGC1385':
        ax.set_ylim([None,12])        
    
    if name in ['NGC5068','IC5332']:
        ax.text(0.2,0.07,f'{name}', transform=ax.transAxes,fontsize=7)        
    else:
        ax.text(0.63,0.07,f'{name}', transform=ax.transAxes,fontsize=7)
    
    label = f'$(m-M)={mu:.2f}^{{+{mu_p:.2f}}}_{{-{mu_m:.2f}}}$'
    ax.text(0.05,0.88,label, transform=ax.transAxes,fontsize=6)
    
    #ax.set_xlim([mu-5,completeness+0.5])
    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'$N_\mathrm{PN}$')
    #ax.set_title(name)
    #ax.set(xlim=[24,28.5])
axes[1,2].set_xlabel(r'$m_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ / mag')
axes[1,3].set_xlabel(r'$m_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ / mag')

ax = next(axes_iter)
h,l = fig.axes[0].get_legend_handles_labels()
ax.axis('off')
ax.legend(h,l,fontsize=7,loc='center left',frameon=False)

ax = next(axes_iter)
ax.remove()

plt.subplots_adjust(wspace=0.15, hspace=0.15)
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')

plt.show()

In [ ]:
from pnlf.plot.pnlf import _plot_pnlf

nbins = {'IC5332':4,'NGC0628':4,'NGC1087':3,'NGC1300':2,'NGC1365':3,
         'NGC1385':4,'NGC1433':3,'NGC1512':3,'NGC1566':3,'NGC1672':2,
         'NGC2835':3,'NGC3351':4,'NGC3627':3,'NGC4254':4,'NGC4303':3,
         'NGC4321':3,'NGC4535':3,'NGC5068':4,'NGC7496':3}

cuts = {'NGC1433':26.8,'NGC1512':26,'NGC3351':25.5}

Enrico_distances = {}

names = results['name']
nrows = 3
ncols = 3
filename = basedir / 'reports' / f'pnlf_enrico_dig'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())
Mmax = -4.47

# loop over the galaxies we want to plot
for name in 'IC5332','NGC0628','NGC1365','NGC1433', 'NGC1512', 'NGC2835','NGC3351','NGC3627','NGC5068':  

    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0
        
    if name in ['IC5332','NGC2835','NGC3627']:
        completeness_limit=27.5
    else:
        completeness_limit=28
        
    data = catalogue[catalogue['gal_name']==name]['mOIII']
    err = catalogue[catalogue['gal_name']==name]['dmOIII']


    cut = cuts.get(name,0)
    data,err = data[data>cut],err[data>cut]

    print(f'analysing {name} (sample table: {parameters[name]["mu"]})')
    fitter = MaximumLikelihood1D(pnlf,data[data<completeness_limit],err=err[data<completeness_limit],
                                 mhigh=completeness_limit,Mmax=Mmax)
    mu,mu_p,mu_m = fitter([29])
    d,(dp,dm)=mu_to_parsec(mu,[mu_p,mu_m])
    print('{:.2f} + {:.2f} - {:.2f}'.format(mu,mu_p,mu_m))
    print('{:.2f} + {:.2f} - {:.2f}'.format(d,dp,dm))

    Enrico_distances[name] = (mu,mu_p,mu_m)
    
    ks,pv = kstest(data[data<completeness_limit],cdf,args=(mu,completeness_limit))
    print(f'{name}: statistic={ks:.3f}, pvalue={pv:.3f}')

    mlow = Mmax+mu
    binsize = (completeness_limit-mlow) / nbins[name]
    mhigh = completeness_limit+1.5*binsize
    
    ax=_plot_pnlf(data,mu,completeness_limit,binsize=binsize,mhigh=mhigh,ax=ax,ms=3)
        
    ylim=ax.get_ylim()
    y2 = ylim[1]*1.7
    if y2>100:y2=99
    ax.set_ylim([0.7,y2])
    if name in ['NGC1433','NGC1512']:
        ax.set_ylim([None,99])
    if name=='NGC1385':
        ax.set_ylim([None,12])        
    
    if name=='NGC2835':
        ax.text(0.2,0.07,f'{name}', transform=ax.transAxes,fontsize=7)        
    else:
        ax.text(0.63,0.07,f'{name}', transform=ax.transAxes,fontsize=7)
    
    label = f'$(m-M)={mu:.2f}^{{+{mu_p:.2f}}}_{{-{mu_m:.2f}}}$'
    ax.text(0.05,0.88,label, transform=ax.transAxes,fontsize=6)
    
    #ax.set_xlim([mu-5,completeness+0.5])
    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'$N_\mathrm{PN}$')
    #ax.set_title(name)
    #ax.set(xlim=[24,28.5])
    
#axes[3,3].set_xlabel(r'$m_{[\mathrm{O}\,\tiny{\textsc{iii}}]}$ / mag')
#ax = next(axes_iter)
#ax.remove()
#h,l = fig.axes[0].get_legend_handles_labels()
#ax.axis('off')
#ax.legend(h,l,fontsize=7,loc='center left',frameon=False)

plt.subplots_adjust(wspace=0.15, hspace=0.15)
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')

plt.show()

In [ ]:
fig,ax=plt.subplots()

for i,(k,v) in enumerate(Enrico_distances.items()):
    
    ax.errorbar(i,v[0]-results.loc[k]['(m-M)'],yerr=v[2],color='tab:red',fmt='o')
    
ax.set_xticks(np.arange(len(Enrico_distances.keys())))
ax.set_xticklabels(Enrico_distances.keys(),rotation=90)
ax.axhline(0,color='black')
ax.set(ylabel=r'$\Delta(m-M)$ (Enrico$-$Fabian)')

plt.savefig(basedir/'reports'/'pnlf_distances_enrico.pdf')

plt.show()


## Playground

In [ ]:
x = np.linspace(0,1)

R = 8.5
g = -0.1

y = R+g*x

x_data = np.random.uniform(0,1,100)
y_data = R+g*x_data + np.random.normal(scale=0.05,size=100)

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(8,4))

ax1.plot(x,y,color='black')
ax1.scatter(x_data,y_data)

delta = y_data - (R+g*x_data)
ax2.scatter(y_data,delta)

plt.show()

In [ ]:
from pnlf.auxiliary import mu_to_parsec, parsec_to_mu

d,err = mu_to_parsec(29,0.109)

err/d*100

In [ ]:
d = 10*u.Mpc
parsec_to_mu(d,d*0.1)